In [2]:
import numpy as np
import scipy.constants as cst
import scipy.linalg as lg
from scipy.integrate import quad
import matplotlib.pyplot as plt
import os.path
import subprocess
import sys
from ipywidgets import Output, Layout
import ipywidgets as widgets
from IPython.display import display,clear_output,HTML,Latex,Math,Markdown
import vegas

style = {'description_width': '250px'}
layout = Layout(width='480px')

#__________________________________________________
fgHbarc = cst.physical_constants["reduced Planck constant times c in MeV fm"][0]
Mp = cst.physical_constants["proton mass energy equivalent in MeV"][0]
Mn = cst.physical_constants["neutron mass energy equivalent in MeV"][0]
MD = cst.physical_constants["deuteron mass energy equivalent in MeV"][0]
alpha_em = 1./cst.physical_constants["inverse fine-structure constant"][0]

MN=(Mp+Mn)/2 #In Mev
epsilon_bind = 2*MN-MD #In MeV

# TYukawaPWF(): 
## Class that creates the non relativistic (NR) S and D wave functions $(U_{nr}(k), W_{nr}(k))$
***
### Input:
1. order: &nbsp;         (int)&emsp;&emsp;&emsp;&emsp; The amount of parameters that have to be taken into account (max = 12)
2. c: &emsp;&emsp;&nbsp; (array[float])                An array with the c parameters in units $MeV^{-3}fm^{-1/2}$
3. d: &emsp;&emsp;&nbsp; (array[float])                An array with the d parameters in units $MeV^{-3}fm^{-1/2}$
4. m: &emsp;&emsp;       (array[float])                An array with the mass parameters in units $fm^{-1}$
***
### Functions:
1. GetUp():
<br> **Input**:
    - k: (float) The proton momentum k (in $MeV$).

 **Returns**
    - The NR S wavefunction, $U_{nr}(p)$ in units $MeV^{-3/2}$
<br>
<br>
2. GetWp(): 
<br> **Input**:
    - k: (float) The proton momentum k (in $MeV$).

 **Returns**
    - The NR D wavefunction, $W_{nr}(p)$ in units $MeV^{-3/2}$
***
### Remarks:
1. The NR wave functions are normalized as follows: $4\pi \int dk k^2\left[U_{nr}^2(k) + W_{nr}^2(k) \right] = 1$ where k is in MeV.
2. The NR wave functions are related to the full wave functions as follows: $U(k) = \sqrt{m}U_{nr}(k)$ where m is the neutron/proton mass.

In [3]:
class TYukawaPWF:
    
    def Mass(self, i):
        # Mass formula:
        # BEGIN_LATEX
        # m_{i} = #alpha + (i-1) m_{0}
        # END_LATEX
        # Return value in units [fm^-1]
        
        return self.fM[i-1]
    
    #NR Wafefunctions__________________________________
    def GetUp(self, k):
        # U_nr non relativistic S wave
        # L=0 wave in momentum space.
        # p = momentum in [MeV]
        # Return value in units [MeV^-3/2]
        
        Mass_move = np.array([self.Mass(i+1) for i in range(self.fOrder)])
        radial = np.sum(self.fC/((k/fgHbarc)**2 + Mass_move**2))
        
        #4\pi normalisation leads to: np.sqrt(2./np.pi)/np.sqrt(4*np.pi) --> 1./np.sqrt(2*np.pi**2)
        return radial/np.sqrt(2.*np.pi**2)/(fgHbarc**(3./2.))
    
    def GetWp(self, k):
        # W_nr non relativistic D wave
        # L=2 wave in momentum space.
        # p = momentum in [MeV]
        # Return value in units [MeV^-3/2]

        Mass_move = np.array([self.Mass(i+1) for i in range(self.fOrder)])
        radial = -np.sum(self.fD/((k/fgHbarc)**2 + Mass_move**2))
        
        #4\pi normalisation leads to: np.sqrt(2./np.pi)/np.sqrt(4*np.pi) --> 1./np.sqrt(2*np.pi**2)
        return radial/np.sqrt(2.*np.pi**2)/(fgHbarc**(3./2.))
    
    def __init__(self,order,c,d,m):
        self.fM = m
        self.fOrder = order
        
        #C boundaries______________________________________
        self.fC = np.array(c[:self.fOrder])
        self.fC[-1] = -np.sum(c[:-1])
        
        #D boundaries______________________________________
        self.fD = np.array(d[:self.fOrder])
        Mass_move = np.array([self.Mass(i+1) for i in range(self.fOrder-3)])
        sumD = -np.sum(self.fD[:-3])
        sumDm = -np.sum(self.fD[:-3]*Mass_move**2)
        sumD_m = -np.sum(self.fD[:-3]/(Mass_move**2))
        sol = np.array([sumD_m, sumD, sumDm])

        A = np.array([[1/(self.Mass(self.fOrder)**2),1/(self.Mass(self.fOrder-1)**2),1/(self.Mass(self.fOrder-2)**2)], \
                      [1.,1.,1.], \
                      [self.Mass(self.fOrder)**2,self.Mass(self.fOrder-1)**2,self.Mass(self.fOrder-2)**2]])
        x = lg.solve(A,sol)
        
        self.fD[-3] = x[2]
        self.fD[-2] = x[1]
        self.fD[-1] = x[0]


# TDeuteron():
## Class that creates a TYukawaPWF element for the AV18 deuteron wavefunction parametrization
***
### Input:
1. Nothing
***
### Functions:
1. wf_av18():
<br> **Input**:
    - nothing.

 **Returns**
    - A TYukawaPWF element for the first AV18 parametrization.
<br>
<br>
2. wf_av18b():
<br> **Input**:
    - nothing.

 **Returns**
    - A TYukawaPWF element for the second AV18 parametrization.


In [4]:
class TDeuteron:
    #returns a non relativistic wavefunction obtained from the AV18 NN potential.
    
    def CreateAV18Wavefunction(self):
        c = np.array([0.706699E+00/np.sqrt(2./np.pi), -0.169743E+00/np.sqrt(2./np.pi), \
                      0.112368E+01/np.sqrt(2./np.pi), -0.852995E+01/np.sqrt(2./np.pi), \
                      0.195033E+02/np.sqrt(2./np.pi), -0.757831E+02/np.sqrt(2./np.pi), \
                      0.283739E+03/np.sqrt(2./np.pi), -0.694734E+03/np.sqrt(2./np.pi), \
                      0.885257E+03/np.sqrt(2./np.pi), -0.720739E+03/np.sqrt(2./np.pi), \
                      0.412969E+03/np.sqrt(2./np.pi), -0.103336E+03/np.sqrt(2./np.pi)])
        
        d = np.array([0.176655E-01/np.sqrt(2./np.pi), -0.124551E+00/np.sqrt(2./np.pi), \
                      -0.108815E+01/np.sqrt(2./np.pi), 0.384848E+01/np.sqrt(2./np.pi), \
                      -0.852442E+01/np.sqrt(2./np.pi), 0.209435E+02/np.sqrt(2./np.pi), \
                      -0.490728E+02/np.sqrt(2./np.pi), 0.577382E+02/np.sqrt(2./np.pi), \
                      -0.127114E+01/np.sqrt(2./np.pi),-0.628361E+02/np.sqrt(2./np.pi), \
                      0.581016E+02/np.sqrt(2./np.pi), -0.177062E+02/np.sqrt(2./np.pi)])
        
        m = np.array([0.2316, 1.0, 1.5, 2.0, 2.5, 3.5, 4.5, 5.5, 6.5, 8.0, 9.5, 11.0]) 
        
        return TYukawaPWF(12,c,d,m)
    
    def CreateAV18bWavefunction(self):
        c = np.array([0.105252223e+02/(4.*np.pi),  0.124352529e+02/(4.*np.pi), \
                      -0.687541641e+02/(4.*np.pi), 0.239111042e+03/(4.*np.pi), \
                      -0.441014422e+03/(4.*np.pi), 0.300140328e+03/(4.*np.pi), \
                      -0.230639939e+03/(4.*np.pi), 0.409671540e+03/(4.*np.pi), \
                      -0.733453611e+03/(4.*np.pi), 0.123506081e+04/(4.*np.pi), \
                      -0.120520606e+04/(4.*np.pi), 0.])
        
        d = np.array([0.280995496e+00/(4.*np.pi),  0.334117629e-01/(4.*np.pi), \
                      -0.727192237e+00/(4.*np.pi),-0.302809607e+01/(4.*np.pi), \
                      -0.903824982e+01/(4.*np.pi), 0.496045967e+01/(4.*np.pi), \
                      -0.271985613e+02/(4.*np.pi), 0.125334598e+03/(4.*np.pi), \
                      -0.346742235e+03/(4.*np.pi), 0.,
                       0.,                        0.])
        
        m = np.array([0.232500e+00, 0.500000e+00, 0.800000e+00, 0.120000e+01, 0.160000e+01, 0.200000e+01, \
                      0.400000e+01, 0.600000e+01, 0.100000e+02, 0.140000e+02, 0.180000e+02, 0.220000e+02])
        
        return TYukawaPWF(12,c,d,m)
    
    def __init__(self):
        self.wf_av18 = self.CreateAV18Wavefunction()
        self.wf_av18b = self.CreateAV18bWavefunction()


# fd_unpol():
## Function to calculate the unpolarized helicity independent LF momentum distribution
***
### Input:
1. alpha_p:&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;           (float)&ensp; The proton light cone momentum fraction.
2. p_pt: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;&nbsp; (float)&ensp; The transverse proton momentum in $MeV$.
3. WF_parametrization:                                    (string) The used AV18 wave function parametrization (first, second). If not specified the first one is used.
***
### Returns:
The unpolarized helicity independent LF momentum distribution $f_d[unpol]$ in $MeV^{-2}$.
***
### Remarks:
1. The light cone components of the proton momentum in the colinear frame are given by: 
\begin{equation}
    \alpha_p = \frac{2p_p^+}{p_d^+} \quad \bf{p}_{pT}
\end{equation} 
<br> Where $\alpha_p$ is the proton light cone momentum fraction between the proton and deuteron light-front components defined as: $p^+ = p^0+p^3$ and $p_{pT}$ the transverse proton momentum. 
<br> These are used to calculate the proton CM momentum k:
\begin{equation}
    |\bf{k}|^2 = \frac{|\bf{p}_{pT}^2|+m^2}{\alpha_p(2-\alpha_p)} - m^2
\end{equation}
With m the mass of the proton. This can than be used to calculate the S and D wavefunction.

2. The unpolarized helicity independent LF momentum distribution is given by:
\begin{equation}
    f_d[unpol](\alpha_p, p_{pT}) = \frac{U^2(k)+W^2(k)}{2-\alpha_p} = m\frac{U_{nr}^2(k)+W_{nr}^2(k)}{2-\alpha_p} = m\frac{U_{nr}^2(\alpha_p, p_{pT})+W_{nr}^2(\alpha_p, p_{pT})}{2-\alpha_p}
\end{equation}
Where we used the fact that we can write the CM momentum k in function of $\alpha_p$ and $p_{pT}$ using the previous equation and the relation between the normal and non relativistic wave functions.

In [5]:
#helicity independent unpolarised momentum distribution
def fd_unpol(alpha_p, p_pt, WF_parametrization = "first"):
    """
    The unpolarized helicity-independent LF momentum distribution of neutrons in the deuteron ensemble
    
    alpha_p (float)     : the proton light cone momentum fraction between the proton and deuteron light-front 
                          component in the colinear frame
    p_pt    (float)(MeV): the norm of the LF transverse momentum of the proton in the colinear frame
    WF_parametrization (string): "first" or "second" AV18 parametrization
    
    returns $f_d(alpha_p,p_{pT})[unpol] = \frac{U^2(k)+W^2(k)}{2-alpha_p}
                                        = \sqrt{k^2+m^2}\frac{U_{nr}^2(k) + W_{nr}^2(k)}{2-alpha_p}  [In MeV^{-2}]
    """
    
    #AV18 WF___________________________________________
    a = TDeuteron()
    if WF_parametrization == "first":
        WF = a.wf_av18
    elif WF_parametrization == "second":
        WF = a.wf_av18b
    else:
        raise ValueError("No parametrization called: {}".format(WF_parametrization))
    
    #momentum function_________________________________
    #calucating the absolute value of the momentum in function of the proton light cone momentum fraction
    #and the transverse momentum
    k2 = (p_pt**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2 #in MeV^2
    
    return np.sqrt(k2+MN**2)*(WF.GetUp(np.sqrt(k2))**2 + WF.GetWp(np.sqrt(k2))**2)/(2-alpha_p) #in MeV^-2


In [6]:
#helicity independent tensor polarized momentum distribution
def fd_tensor(alpha_p, p_pt, WF_parametrization = "first"):
    """
    The tensor polarized helicity-independent LF momentum distribution of neutrons in the deuteron ensemble
    
    alpha_p (float)     : the proton light cone momentum fraction between the proton and deuteron light-front 
                          component in the colinear frame
    p_pt    (float)(MeV): the norm of the LF transverse momentum of the proton in the colinear frame
    WF_parametrization (string): "first" or "second" AV18 parametrization
    
    R_T = 1 - \frac{3|k_T|^2}{2|k|^2}
    
    returns $f_d(alpha_p,p_{pT})[tensor] = -\frac{1}{2-alpha_p}R_T(2U-\frac{W}{\sqrt{2}})
                                            \frac{W}{sqrt{2}} 
                                           -E\frac{1}{2-alpha_p}R_T(2U_{nr}-\frac{W_{nr}}{\sqrt{2}})
                                            \frac{W_{nr}}{sqrt{2}}  [In MeV^{-2}]
    """
    
    #AV18 WF___________________________________________
    a = TDeuteron()
    if WF_parametrization == "first":
        WF = a.wf_av18
    elif WF_parametrization == "second":
        WF = a.wf_av18b
    else:
        raise ValueError("No parametrization called: {}".format(WF_parametrization))
        
    #kinematic values
    M_pn2 = 4.*(p_pt**2 + MN**2)/(alpha_p*(2.-alpha_p)) #in MeV^2
    k2 = M_pn2/4.-MN**2   #in MeV^2
    E = np.sqrt(M_pn2)/2. #in MeV
    
    R_T = 1. - 3.*p_pt**2/(2.*k2)
    
    return -E/(2.-alpha_p)*R_T*(2.*WF.GetUp(np.sqrt(k2)) + WF.GetWp(np.sqrt(k2))/np.sqrt(2.))*\
WF.GetWp(np.sqrt(k2))/np.sqrt(2.)

In [7]:
#helicity dependent vector polarized momentum distribution
def delta_fd(alpha_p, p_pt, WF_parametrization = "first"):
    """
    The vector polarized helicity-dependent LF momentum distribution of neutrons in the deuteron ensemble
    
    alpha_p (float)     : the proton light cone momentum fraction between the proton and deuteron light-front 
                          component in the colinear frame
    p_pt    (float)(MeV): the norm of the LF transverse momentum of the proton in the colinear frame
    WF_parametrization (string): "first" or "second" AV18 parametrization
    
    returns $\Dleta f_d(alpha_p,p_{pT})[vector] = \frac{1}{2-alpha_p}(U-\frac{W}{\sqrt{2}})
                                                  (UR_{U}-\frac{WR_{W}}{\sqrt{2}})
                                                = E\frac{1}{2-alpha_p}(U_{nr}-\frac{W_{nr}}{\sqrt{2}})
                                                  (U_{nr}R_{U}-\frac{W_{nr}R_{W}}{\sqrt{2}})    [In MeV^{-2}]
    """
    
    #AV18 WF___________________________________________
    a = TDeuteron()
    if WF_parametrization == "first":
        WF = a.wf_av18
    elif WF_parametrization == "second":
        WF = a.wf_av18b
    else:
        raise ValueError("No parametrization called: {}".format(WF_parametrization))
        
    #kinematic values
    M_pn2 = 4.*(p_pt**2 + MN**2)/(alpha_p*(2.-alpha_p)) #in MeV^2
    kz = np.sqrt(M_pn2)*(alpha_p-1.)/2. #in MeV
    k2 = M_pn2/4.-MN**2 #in MeV^2
    E = np.sqrt(M_pn2)/2.
    
    R_U = 1.-((E+kz)*p_pt**2)/((E+MN)*(MN**2+p_pt**2)) #MN/((2.-alpha_p)*E)*(1.-kz/MN+(kz)**2/(MN*(E+MN)))
    R_W = 1.-((E+2.*MN)*(E+kz)*p_pt**2)/((MN**2+p_pt**2)*k2) 
                                                       #MN/((2.-alpha_p)*E)*(-2.-kz/MN+(E+2.*MN)*(kz)**2/(MN*k2))
   
    return E/(2.-alpha_p)*(WF.GetUp(np.sqrt(k2)) - WF.GetWp(np.sqrt(k2))/np.sqrt(2.))*\
(WF.GetUp(np.sqrt(k2))*R_U - WF.GetWp(np.sqrt(k2))*R_W/np.sqrt(2.))

# Plotting
***

In [98]:
def plot_momentum_density(k_min,k_max,step_size, WF_parametrization = "first", log = True, save = ''):
    """
    plot the momentum density of both the S wave only and the S + D wave in function of the norm of the 
    proton/neuteron C.O.M. 3 momentum in the pn configuration.
    
    k_min (float)(MeV)    : start of the momentum axis
    k_max (float)(MeV)    : end of the momentum axis
    step_size (float)(MeV): interval size of the momentum
    WF_parametrization (string): "first" or "second" AV18 parametrization
    log   (bool)          : use logaritmic y axis, standard is True
    save  (string)        : save the figure, if empty figure is not saved
    
    returns figure
    """
    
    #AV18 parametrisation______________________________
    a = TDeuteron()
    if WF_parametrization == "first":
        WF = a.wf_av18
    elif WF_parametrization == "second":
        WF = a.wf_av18b
    else:
        raise ValueError("No parametrization called: {}".format(WF_parametrization))
        
    #curve values______________________________________
    k = np.arange(k_min,k_max+step_size,step_size) #momentum in MeV 
    md_S  = np.array([WF.GetUp(k_i)**2 for k_i in k]) #momentum density for pure S waves in MeV^-3
    md_SD = np.array([WF.GetUp(k_i)**2 + WF.GetWp(k_i)**2 for k_i in k]) #momentum density for S and D waves 
                                                                         #in MeV^-3

    #plotting__________________________________________
    #plot the momentum in GeV so multiply with 10^3^-, plot the momentum density in GeV^-3 so multiply with 10^9
    fig,ax = plt.subplots(figsize=(8,4))
    ax.plot(k*10**-3,md_S*10**9,label = "S-wave only (a)")
    ax.plot(k*10**-3,md_SD*10**9, label = "S + D waves (a)")
    
    #ax.set_title("Momentum density of non-relativistic deuteron WF")
    ax.legend(loc=2,bbox_to_anchor=(1.,1.))
    ax.set_ylabel(r"$|\tilde{\psi}_D(\mathbf{k})|^2 \quad [GeV^{-3}]$")
    ax.set_xlabel(r"$k \quad [GeV]$")
    plt.tight_layout()
    if log:
        ax.set_yscale("log")
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    plt.show()
    
    return fig

def make_figure_momentum_density(out):
    intro = widgets.HTMLMath(r"""<center><b><h1>Momentum density</h1></b></center>
    <hr>
    <figure style="float: right;">
        <img src="figures/figure_momentum_density.png" width ="400">
        <figcaption style="text-align:center">fig: An example figure of the <b>Momentum density</b> program
        </figcaption>
    </figure>
    
    <p>&emsp;This cell can create a figure of the momentum density for both the non relativistic $S_{nr}$ and 
    $S_{nr}+D_{nr}$ waves in function of the proton 3 momentum norm k from <b>$k_{min}$</b> to <b>$k_{max}$</b> 
    in steps of <b>$\Delta k$</b>, given in MeV.</p>
    
    <p>&emsp;There are 2 AV18 Wave function parametrizations, AV18a and AV18b, this can be chosen by use of the <b>"Wave 
    function"</b> radio buttons. The figure can be displayed on a logaritmic y-axis by checking the <b>"logaritmic 
    y-axis"</b> checkbox. If a name is given in the box titled <b>"Figure name"</b>, the figure is stored as 
    this name.png. If no name is given the figure is not saved but there is the option to save the figure after it 
    is displayed. After displaying, and when no name was given, a box titled <b>"Figure name"</b> appears in which 
    a name of the figure can be provided as which the figure will be saved after pressing the <b>"Save figure"</b> 
    button.</p>
    
    <p>&emsp;The figure will be made after pressing the <b>"Make momentum density figure"</b> button.</p><hr>""")
    
    k_min_val    =widgets.Text(value='0',placeholder=r'minimum momentum k',
                           description=r'$k_{min} (MeV)$',disabled=False,style=style,layout=layout)
    k_max_val    =widgets.Text(value='1000',placeholder=r'maximum momentum k',
                           description=r'$k_{max} (MeV)$',disabled=False,style=style,layout=layout)
    step_size_val=widgets.Text(value='1',placeholder=r'momentum step size',
                           description=r'$\Delta k (MeV)$',disabled=False,style=style,layout=layout)
    WF_parametrization_val=widgets.RadioButtons(options=[("AV18a","first"), ("AV18b","second")],
                                                description='Wave function:',disabled=False,style = style,
                                                layout = layout)
    logaritmic    =widgets.Checkbox(value=True,description='logaritmic y axis',disabled=False,style=style,
                                    layout=layout)
    save          =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                                disabled=False,style=style,layout=layout)
    make_figure_md=widgets.Button(description='Make momentum density figure',disabled=False,button_style='',
                                  tooltip='make figure',icon='check',layout=Layout(width="initial"))
    
    display(intro)
    
    display(widgets.HBox((k_min_val,k_max_val)))
    display(widgets.HBox((step_size_val,WF_parametrization_val)))
    display(widgets.HBox((logaritmic,save)))
    display(make_figure_md)

    def momentum_density_fig(_):
        with out:
            clear_output()
            k_min     = float(k_min_val.value)
            k_max     = float(k_max_val.value)
            step_size = float(step_size_val.value)
            figure    = plot_momentum_density(k_min,k_max,step_size,WF_parametrization_val.value,logaritmic.value,
                                          save.value)
            if save.value:
                save.value = ""
            else:
                save_after_t=widgets.Text(value='',placeholder=r"Name of figure for saving",
                                          description=r'Figure name:',disabled=False,
                                          style={'description_width': 'initial'},layout=Layout(width="initial"))
                save_after_b=widgets.Button(description='Save figure',disabled=False,button_style='',
                                            tooltip='save',icon='check',style={'description_width': 'initial'},
                                            layout=Layout(width="initial"))
                display(widgets.HBox((save_after_t,save_after_b)))
                def on_clicked(_):
                    if not "png" in save_after_t.value and not "jpg" in save_after_t.value and\
                    not "pdf" in save_after_t.value:
                        save_after_t.value += ".png"
                    saving = os.path.join(os.getcwd(),"output",save_after_t.value)
                    figure.savefig(saving,dpi=600)
                    print("saved as {}".format(saving))
                    save_after_t.value=""
                save_after_b.on_click(on_clicked)
    make_figure_md.on_click(momentum_density_fig)


In [99]:
def plot_fd_unpol(alpha_p_min, alpha_p_max, step_size, p_pt, WF_parametrization="first", save = "", log = True):
    """
    plot the helicity independent unpolarised momentum distribution in funtion of alpha_p values for different 
    fixed transvere momenta values, p_pt.
    
    alpha_p_min (float)     : start of the alpha_p axis
    alpha_p_max (float)     : end of the alpha_p axis
    step_size   (float)     : interval size for alpha_p
    p_pt        (array[float])(MeV): array of transverse momenta values p_pt
    WF_parametrization (string): "first" or "second" AV18 parametrization
    save        (bool)      : save the figure
    log         (bool)      : use a logaritmic y-axis
    
    returns None
    """
    
    #curve values______________________________________
    alpha_p_vals = np.arange(alpha_p_min, alpha_p_max+step_size,step_size)
    fd_vals =np.array([[fd_unpol(alpha_p_i,p_pt_i,WF_parametrization) for alpha_p_i in alpha_p_vals]\
                      for p_pt_i in p_pt])  #in MeV^-2

    #plotting__________________________________________
    linestyles = [(0, ()), (0, (5, 1)), (0, (5, 5)), (0, (5, 10)), (0, (1, 1)), (0, (1, 1)), (0, (1, 10))]
    fig,ax = plt.subplots(figsize=(8,4))
    for i,fd_val in enumerate(fd_vals):
        ax.plot(alpha_p_vals,fd_val*10**6,linestyle = linestyles[i],\
                 label=r"$p_{{pT}} = {} \quad MeV$".format(p_pt[i])) #*10**6 to get it into GeV^-2
    #ax.set_title("Deuteron light-front spectral function (AV18)")
    ax.set_ylabel(r"$f_d(\alpha_R,p_{RT})[unpol] \quad [GeV^{-2}]$")
    ax.legend(loc=2,bbox_to_anchor=(1.,1.))
    ax.set_xlabel(r"$\alpha_R$")
    plt.tight_layout()
    if log:
        ax.set_yscale("log")
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    plt.show()
    
    return fig

def make_figure_fd_unpol(out):
    intro = widgets.HTMLMath(r"""<center><b><h1>Helicity independent unpolarised momentum distribution</h1></b>
    </center>
    <hr>
    <figure style="float: right;">
        <img src="figures/figure_fd_unpol.png" width ="400">
        <figcaption style="text-align:center">fig: An example figure of the <b>Helicity independent <br/>unpolarised 
        momentum distribution</b> program</figcaption>
    </figure>
    
    <p>&emsp;This cell can create a figure of the helicity independent unpolarised momentum distribution of the
    neutrons in the deuteron $f_d(\alpha_p,\mathbf{p}_{pT})[unpol])$ in function of the proton LF momentum 
    fraction $\alpha_p$ from <b>"$\alpha_{p}$ lower bound"</b> to <b>$"\alpha_{p}$ upper bound"</b> in steps of <b>$\Delta \alpha_p$
    </b>.</p>
    
    <p>&emsp;For a range of transverse proton momentum values given at <b>$p_{pT}$</b> in MeV, seperated by 
    commas. There are 2 AV18 Wave function parametrizations, AV18a and AV18b, this can be chosen by use of the 
    <b>"Wave function"</b> radio buttons. The figure can be displayed on a logaritmic y-axis by checking the <b>
    "logaritmic y-axis"</b> checkbox. If a name is given in the box titled <b>"Figure name"</b>, the figure is 
    stored as this name.png. If no name is given the figure is not saved but there is the option to save the 
    figure after it is displayed. After displaying, and when no name was given, a box titled <b>"Figure name"</b> 
    appears in which a name of the figure can be provided as which the figure will be saved after pressing the 
    <b>"Save figure"</b> button.</p>
    
    <p>&emsp;The figure will be made after pressing the <b>"Make fd[unpol] figure"</b> button.</p><hr>""")
    
    alpha_p_min_val    =widgets.Text(value='0.6',placeholder=r'minimum proton momentum fraction',
                                     description=r'$\alpha_{p}$ lower bound',disabled=False,style=style,layout=layout)
    alpha_p_max_val    =widgets.Text(value='1.4',placeholder=r'maximum proton momentum fraction',
                                     description=r'$\alpha_{p}$ upper bound',disabled=False,style=style,
                                     layout=layout)
    step_size_val      =widgets.Text(value='0.001',placeholder=r'proton momentum fraction step size',
                                     description=r'$\Delta \alpha_p$',disabled=False,style=style,layout=layout)
    p_pt_val           =widgets.Text(value='0,100,200,300,400',placeholder=r'transverse momenta',
                                     description=r'$p_{pt} (MeV)$',disabled=False,style=style,layout=layout)
    WF_parametrization_val=widgets.RadioButtons(options=[("AV18a","first"), ("AV18b","second")],
                                                description='Wave function:',
                                                disabled=False,style = style,layout = layout)
    logaritmic    =widgets.Checkbox(value=False,description='logaritmic y axis',disabled=False,style=style,
                                    layout=layout)
    save          =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                                disabled=False,style=style,layout=layout)
    make_figure_fd=widgets.Button(description='Make fd[unpol] figure',disabled=False,button_style='',
                                  tooltip='make figure',icon='check',layout=Layout(width="initial"))
    
    display(intro)
    
    display(widgets.HBox((alpha_p_min_val,alpha_p_max_val)))
    display(step_size_val)
    display(widgets.HBox((p_pt_val,WF_parametrization_val)))
    display(widgets.HBox((logaritmic,save)))
    display(make_figure_fd)

    def fd_unpol_fig(_):
        with out:
            clear_output()
            alpha_p_min = float(alpha_p_min_val.value)
            alpha_p_max = float(alpha_p_max_val.value)
            step_size   = float(step_size_val.value)
            p_pt        = np.array([float(p_pt_i) for p_pt_i in p_pt_val.value.split(",")])
            figure      = plot_fd_unpol(alpha_p_min, alpha_p_max, step_size, p_pt, WF_parametrization_val.value, 
                                        save.value,logaritmic.value)
            if save.value:
                save.value = ""
            else:
                save_after_t=widgets.Text(value='',placeholder=r"Name of figure for saving",
                                          description=r'Figure name:',disabled=False,
                                          style={'description_width': 'initial'},layout=Layout(width="initial"))
                save_after_b=widgets.Button(description='Save figure',disabled=False,button_style='',
                                            tooltip='save',icon='check',style={'description_width': 'initial'},
                                            layout=Layout(width="initial"))
                display(widgets.HBox((save_after_t,save_after_b)))
                def on_clicked(_):
                    if not "png" in save_after_t.value and not "jpg" in save_after_t.value and\
                    not "pdf" in save_after_t.value:
                        save_after_t.value += ".png"
                    saving = os.path.join(os.getcwd(),"output",save_after_t.value)
                    figure.savefig(saving,dpi=600)
                    print("saved as {}".format(saving))
                    save_after_t.value=""
                save_after_b.on_click(on_clicked)
    make_figure_fd.on_click(fd_unpol_fig)


In [100]:
def plot_fd_tensor(alpha_p_min, alpha_p_max, step_size, p_pt, WF_parametrization="first", save = "", log = True):
    """
    plot the helicity independent tensor polarized momentum distribution in funtion of alpha_p values for 
    different fixed transvere momenta values, p_pt.
    
    alpha_p_min (float)     : start of the alpha_p axis
    alpha_p_max (float)     : end of the alpha_p axis
    step_size   (float)     : interval size for alpha_p
    p_pt        (array[float])(MeV): array of transverse momenta values p_pt
    WF_parametrization (string): "first" or "second" AV18 parametrization
    save        (bool)      : save the figure
    log         (bool)      : use a logaritmic y-axis
    
    returns None
    """
    
    #curve values______________________________________
    alpha_p_vals = np.arange(alpha_p_min, alpha_p_max+step_size,step_size)
    fd_vals =np.array([[fd_tensor(alpha_p_i,p_pt_i,WF_parametrization) for alpha_p_i in alpha_p_vals]\
                      for p_pt_i in p_pt])  #in MeV^-2

    #plotting__________________________________________
    linestyles = [(0, ()), (0, (5, 1)), (0, (5, 5)), (0, (5, 10)), (0, (1, 1)), (0, (1, 1)), (0, (1, 10))]
    fig,ax = plt.subplots(figsize=(8,4))
    for i,fd_val in enumerate(fd_vals):
        ax.plot(alpha_p_vals,fd_val*10**6,linestyle = linestyles[i],\
                 label=r"$p_{{pT}} = {} \quad MeV$".format(p_pt[i])) #*10**6 to get it into GeV^-2
    #ax.set_title("Deuteron light-front spectral function (AV18)")
    ax.set_ylabel(r"$f_d(\alpha_R,p_{RT})[tensor] \quad [GeV^{-2}]$")
    ax.legend(loc=2,bbox_to_anchor=(1.,1.))
    ax.set_xlabel(r"$\alpha_R$")
    plt.tight_layout()
    if log:
        ax.set_yscale("log")
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    plt.show()
    
    return fig

def make_figure_fd_tensor(out):
    intro = widgets.HTMLMath(r"""<center><b><h1>Helicity independent tensor polarized momentum distribution</h1>
    </b></center>
    <hr>
    <figure style="float: right;">
        <img src="figures/figure_fd_tensor.png" width ="400">
        <figcaption style="text-align:center">fig: An example figure of the <b>Helicity independent <br/>tensor 
        polarized momentum distribution</b> program</figcaption>
    </figure>
    
    <p>&emsp;This cell can create a figure of the helicity independent tensor polarized momentum distribution of 
    the neutrons in the deuteron $f_d(\alpha_p,\mathbf{p}_{pT})[tensor])$ in function of the proton LF momentum 
    fraction $\alpha_p$ from <b>"$\alpha_{p}$ lower bound"</b> to <b>$"\alpha_{p}$ upper bound"</b> in steps 
    of <b>$\Delta \alpha_p$
    </b>.</p>
    
    <p>&emsp;For a range of transverse proton momentum values given at <b>$p_{pT}$</b> in MeV, seperated by 
    commas. There are 2 AV18 Wave function parametrizations, AV18a and AV18b, this can be chosen by use of the 
    <b>"Wave function"</b> radio buttons. The figure can be displayed on a logaritmic y-axis by checking the <b>
    "logaritmic y-axis"</b> checkbox. If a name is given in the box titled <b>"Figure name"</b>, the figure is 
    stored as this name.png. If no name is given the figure is not saved but there is the option to save the 
    figure after it is displayed. After displaying, and when no name was given, a box titled <b>"Figure name"</b> 
    appears in which a name of the figure can be provided as which the figure will be saved after pressing the 
    <b>"Save figure"</b> button.</p>
    
    <p>&emsp;The figure will be made after pressing the <b>"Make fd[tensor] figure"</b> button.</p><hr>""")
    
    alpha_p_min_val    =widgets.Text(value='0.6',placeholder=r'minimum proton momentum fraction',
                                     description=r'$\alpha_{p}$ lower bound',disabled=False,style=style,layout=layout)
    alpha_p_max_val    =widgets.Text(value='1.4',placeholder=r'maximum proton momentum fraction',
                                     description=r'$\alpha_{p}$ upper bound',disabled=False,style=style,
                                     layout=layout)
    step_size_val      =widgets.Text(value='0.001',placeholder=r'proton momentum fraction step size',
                                     description=r'$\Delta \alpha_p$',disabled=False,style=style,layout=layout)
    p_pt_val           =widgets.Text(value='0,100,200,300,400',placeholder=r'transverse momenta',
                                     description=r'$p_{pt} (MeV)$',disabled=False,style=style,layout=layout)
    WF_parametrization_val=widgets.RadioButtons(options=[("AV18a","first"), ("AV18b","second")],description='Wave function:',
                                                disabled=False,style = style,layout = layout)
    logaritmic    =widgets.Checkbox(value=False,description='logaritmic y axis',disabled=False,style=style,
                                    layout=layout)
    save          =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                                disabled=False,style=style,layout=layout)
    make_figure_fd=widgets.Button(description='Make fd[tensor] figure',disabled=False,button_style='',
                                  tooltip='make figure',icon='check',layout=Layout(width="initial"))
    
    display(intro)
    
    display(widgets.HBox((alpha_p_min_val,alpha_p_max_val)))
    display(step_size_val)
    display(widgets.HBox((p_pt_val,WF_parametrization_val)))
    display(widgets.HBox((logaritmic,save)))
    display(make_figure_fd)

    def fd_tensor_fig(_):
        with out:
            clear_output()
            alpha_p_min = float(alpha_p_min_val.value)
            alpha_p_max = float(alpha_p_max_val.value)
            step_size   = float(step_size_val.value)
            p_pt        = np.array([float(p_pt_i) for p_pt_i in p_pt_val.value.split(",")])
            figure      = plot_fd_tensor(alpha_p_min, alpha_p_max, step_size, p_pt, WF_parametrization_val.value, 
                                        save.value,logaritmic.value)
            if save.value:
                save.value = ""
            else:
                save_after_t=widgets.Text(value='',placeholder=r"Name of figure for saving",
                                          description=r'Figure name:',disabled=False,
                                          style={'description_width': 'initial'},layout=Layout(width="initial"))
                save_after_b=widgets.Button(description='Save figure',disabled=False,button_style='',
                                            tooltip='save',icon='check',style={'description_width': 'initial'},
                                            layout=Layout(width="initial"))
                display(widgets.HBox((save_after_t,save_after_b)))
                def on_clicked(_):
                    if not "png" in save_after_t.value and not "jpg" in save_after_t.value and\
                    not "pdf" in save_after_t.value:
                        save_after_t.value += ".png"
                    saving = os.path.join(os.getcwd(),"output",save_after_t.value)
                    figure.savefig(saving,dpi=600)
                    print("saved as {}".format(saving))
                    save_after_t.value=""
                save_after_b.on_click(on_clicked)
    make_figure_fd.on_click(fd_tensor_fig)


In [101]:
def plot_fd_vector(alpha_p_min, alpha_p_max, step_size, p_pt, WF_parametrization="first", save = "", log = True):
    """
    plot the helicity independent tensor polarized momentum distribution in funtion of alpha_p values for 
    different fixed transvere momenta values, p_pt.
    
    alpha_p_min (float)     : start of the alpha_p axis
    alpha_p_max (float)     : end of the alpha_p axis
    step_size   (float)     : interval size for alpha_p
    p_pt        (array[float])(MeV): array of transverse momenta values p_pt
    WF_parametrization (string): "first" or "second" AV18 parametrization
    save        (bool)      : save the figure
    log         (bool)      : use a logaritmic y-axis
    
    returns None
    """
    
    #curve values______________________________________
    alpha_p_vals = np.arange(alpha_p_min, alpha_p_max+step_size,step_size)
    fd_vals =np.array([[delta_fd(alpha_p_i,p_pt_i,WF_parametrization) for alpha_p_i in alpha_p_vals]\
                      for p_pt_i in p_pt])  #in MeV^-2

    #plotting__________________________________________
    linestyles = [(0, ()), (0, (5, 1)), (0, (5, 5)), (0, (5, 10)), (0, (1, 1)), (0, (1, 1)), (0, (1, 10))]
    fig,ax = plt.subplots(figsize=(8,4))
    for i,fd_val in enumerate(fd_vals):
        ax.plot(alpha_p_vals,fd_val*10**6,linestyle = linestyles[i],\
                 label=r"$p_{{pT}} = {} \quad MeV$".format(p_pt[i])) #*10**6 to get it into GeV^-2
    #ax.set_title("Deuteron light-front spectral function (AV18)")
    ax.set_ylabel(r"$\Delta f_d(\alpha_R,p_{RT})[vector] \quad [GeV^{-2}]$")
    ax.legend(loc=2,bbox_to_anchor=(1.,1.))
    ax.set_xlabel(r"$\alpha_R$")
    plt.tight_layout()
    if log:
        ax.set_yscale("log")
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    plt.show()
    
    return fig

def make_figure_fd_vector(out):
    intro = widgets.HTMLMath(r"""<center><b><h1>Helicity dependent vector polarized momentum distribution</h1>
    </b></center>
    <hr>
    <figure style="float: right;">
        <img src="figures/figure_fd_vector.png" width ="400">
        <figcaption style="text-align:center">fig: An example figure of the <b>Helicity dependent <br/>vector 
        polarized momentum distribution</b> program</figcaption>
    </figure>
    
    <p>&emsp;This cell can create a figure of the helicity dependent vector polarized momentum distribution of 
    the neutrons in the deuteron $\Delta f_d(\alpha_p,\mathbf{p}_{pT})[vector])$ in function of the proton LF 
    momentum fraction $\alpha_p$ from <b>"$\alpha_{p}$ lower bound"</b> to <b>$"\alpha_{p}$ upper bound"</b>
    in steps of <b>$\Delta \alpha_p$</b>.</p>
    
    <p>&emsp;For a range of transverse proton momentum values given at <b>$p_{pT}$</b> in MeV, seperated by 
    commas. There are 2 AV18 Wave function parametrizations, AV18a and AV18b, this can be chosen by use of the 
    <b>"Wave function"</b> radio buttons. The figure can be displayed on a logaritmic y-axis by checking the <b>
    "logaritmic y-axis"</b> checkbox. If a name is given in the box titled <b>"Figure name"</b>, the figure is 
    stored as this name.png. If no name is given the figure is not saved but there is the option to save the 
    figure after it is displayed. After displaying, and when no name was given, a box titled <b>"Figure name"</b> 
    appears in which a name of the figure can be provided as which the figure will be saved after pressing the 
    <b>"Save figure"</b> button.</p>
    
    <p>&emsp;The figure will be made after pressing the <b>"Make fd[vector] figure"</b> button.</p><hr>""")
    
    alpha_p_min_val    =widgets.Text(value='0.6',placeholder=r'minimum proton momentum fraction',
                                     description=r'$\alpha_{p}$ lower bound',disabled=False,style=style,layout=layout)
    alpha_p_max_val    =widgets.Text(value='1.4',placeholder=r'maximum proton momentum fraction',
                                     description=r'$\alpha_{p}$ upper bound',disabled=False,style=style,
                                     layout=layout)
    step_size_val      =widgets.Text(value='0.001',placeholder=r'proton momentum fraction step size',
                                     description=r'$\Delta \alpha_p$',disabled=False,style=style,layout=layout)
    p_pt_val           =widgets.Text(value='0,100,200,300,400',placeholder=r'transverse momenta',
                                     description=r'$p_{pt} (MeV)$',disabled=False,style=style,layout=layout)
    WF_parametrization_val=widgets.RadioButtons(options=[("AV18a","first"), ("AV18b","second")],description='Wave function:',
                                                disabled=False,style = style,layout = layout)
    logaritmic    =widgets.Checkbox(value=False,description='logaritmic y axis',disabled=False,style=style,
                                    layout=layout)
    save          =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                                disabled=False,style=style,layout=layout)
    make_figure_fd=widgets.Button(description='Make fd[vector] figure',disabled=False,button_style='',
                                  tooltip='make figure',icon='check',layout=Layout(width="initial"))
    save_after_t  =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                                disabled=False,style = {'description_width': 'initial'},
                                layout=Layout(width="initial"))
    save_after_b  = widgets.Button(description='Save figure',disabled=False,button_style='',tooltip='save',
                                   icon='check',style = {'description_width': 'initial'},
                                   layout=Layout(width="initial"))
    display(intro)
    
    display(widgets.HBox((alpha_p_min_val,alpha_p_max_val)))
    display(step_size_val)
    display(widgets.HBox((p_pt_val,WF_parametrization_val)))
    display(widgets.HBox((logaritmic,save)))
    display(make_figure_fd)

    def fd_vector_fig(_):
        with out:
            clear_output()
            alpha_p_min = float(alpha_p_min_val.value)
            alpha_p_max = float(alpha_p_max_val.value)
            step_size   = float(step_size_val.value)
            p_pt        = np.array([float(p_pt_i) for p_pt_i in p_pt_val.value.split(",")])
            figure      = plot_fd_vector(alpha_p_min, alpha_p_max, step_size, p_pt, WF_parametrization_val.value, 
                                        save.value,logaritmic.value)
            if save.value:
                save.value = ""
            else:
                save_after_t=widgets.Text(value='',placeholder=r"Name of figure for saving",
                                          description=r'Figure name:',disabled=False,
                                          style={'description_width': 'initial'},layout=Layout(width="initial"))
                save_after_b=widgets.Button(description='Save figure',disabled=False,button_style='',
                                            tooltip='save',icon='check',style={'description_width': 'initial'},
                                            layout=Layout(width="initial"))
                display(widgets.HBox((save_after_t,save_after_b)))
                def on_clicked(_):
                    if not "png" in save_after_t.value and not "jpg" in save_after_t.value and\
                    not "pdf" in save_after_t.value:
                        save_after_t.value += ".png"
                    saving = os.path.join(os.getcwd(),"output",save_after_t.value)
                    figure.savefig(saving,dpi=600)
                    print("saved as {}".format(saving))
                    save_after_t.value=""
                save_after_b.on_click(on_clicked)
    make_figure_fd.on_click(fd_vector_fig)


In [102]:
def integral_check(k_start,k_stop, WF_parametrization = "first"):
    """
    prints the integral values of the momentum density which should be 1.
    
    k_start (float)(MeV): the norm of the proton 3 momentum from which to start the integration, best value is 0
    k_stop  (float)(MeV): the norm of the proton 3 momentum where to stop the integration, best a value greater
                          than 1000 Mev
    WF_parametrization (string): "first" or "second" AV18 parametrization
    
    returns the integral value and error (Integral values, error)
    """
    #AV18 parametrisation______________________________
    a = TDeuteron()
    if WF_parametrization == "first":
        WF = a.wf_av18
    elif WF_parametrization == "second":
        WF = a.wf_av18b
    else:
        raise ValueError("No parametrization called: {}".format(WF_parametrization))
    
    #integrandum_______________________________________
    def integrandum(k):
        #4*\pi is for the normalisation, integral is supposed to be 1
        return 4*np.pi*k**2*(WF.GetUp(k)**2 + WF.GetWp(k)**2)

    integral = quad(integrandum,k_start,k_stop)
    
    display(Math(r"$4\pi\int_{{{}}}^{{{}}} \left[U_{{nr}}(k)^2+W_{{nr}}(p)^2\right]k^2dk = {:.5g} \pm {:.5g} \quad [\text{{k in MeV}}]$"\
                 .format(k_start,k_stop,integral[0],integral[1])))
    
    return integral


In [106]:
#out=Output()
#make_figure_momentum_density(out)
#make_figure_fd_unpol(out)
#make_figure_fd_tensor(out)
#make_figure_fd_vector(out)
#plot_fd_unpol(0.6, 1.4,0.001, np.array([0,100,200,300,400]))
#I = integral_check(0,1000)
#out

# struc_func():
## Class that calculates the different structure functions.
***
### Input:
1. SF_parametriation: &nbsp; (string) The used structure function parametriation. Standard value is "CTEQ", other parametriations are:
    - "CB": Christy & Bosted parametrization.
    - "SLAC": SLAC paramtetrization from Bodek.
    - "Alekhin": leading twist parametrization by Alekhin [see PRD 68,014002].
    - "CTEQ": F2 based on the pdf's from CTEQ (code from Misak).
    - "HMRS": F2 from pdfs from unknown source (Shunzo Kumano is the source, not enough info in his code file).
    - "MSTW": F2 from LO (or nlo etc) from MSTW pdfs.
2. WF_parametrization: (string) The used AV18 wave function parametrization (first, second). If not specified the first one is used.
***
### Functions:
1. F1n():
<br> **Input**:
    - x_tilde        (float): The effective scaling variable for scattering on the neuteron, taken the longitudinal LF momentum in the deuteron into account.
    - Q2 &emsp;&ensp;(float): The negative of the 4 momentum transfer squared or virtual photon momentum squared in $MeV^2$.

 **Returns**
    - The conventional unpolarized nucleon structure function $F_{1n}$.
<br>
<br>
2. F2n():
<br> **Input**:
    - x_tilde        (float): The effective scaling variable for scattering on the neuteron, taken the longitudinal LF momentum in the deuteron into account.
    - Q2 &emsp;&ensp;(float): The negative of the 4 momentum transfer squared or virtual photon momentum squared in $MeV^2$.

 **Returns**
    - The conventional unpolarized nucleon structure function $F_{2n}$.
<br>
<br>
3. F1d():
<br> **Input**:
    - x: &emsp;&emsp;&emsp; (float) The effective Bjorken variable for scattering from a nucleon in the deuteron in the absence of nuclear binding.
    - Q2: &emsp;&emsp;      (float) The negative of the 4 momentum transfer squared or virtual photon momentum squared in $MeV^2$.
    - alpha_p:              (float) The proton light cone momentum fraction.
    - p_pt: &emsp;&nbsp;    (float) The transverse proton momentum in $MeV$.

 **Returns**
    - The unpolarized tagged deuteron structure function $F_{1d}$ in $MeV^{-2}$.    
<br>
<br>
4. F2d():
<br> **Input**:
    - x: &emsp;&emsp;&emsp; (float) The effective Bjorken variable for scattering from a nucleon in the deuteron in the absence of nuclear binding.
    - Q2: &emsp;&emsp;      (float) The negative of the 4 momentum transfer squared or virtual photon momentum squared in $MeV^2$.
    - alpha_p:              (float) The proton light cone momentum fraction.
    - p_pt: &emsp;&nbsp;    (float) The transverse proton momentum in $MeV$.

 **Returns**
    - The unpolarized tagged deuteron structure function $F_{2d}$ in $MeV^{-2}$.
<br>
<br>
5. FTd():
<br> **Input**:
    - x: &emsp;&emsp;&emsp; (float) The effective Bjorken variable for scattering from a nucleon in the deuteron in the absence of nuclear binding.
    - Q2: &emsp;&emsp;      (float) The negative of the 4 momentum transfer squared or virtual photon momentum squared in $MeV^2$.
    - alpha_p:              (float) The proton light cone momentum fraction.
    - p_pt: &emsp;&nbsp;    (float) The transverse proton momentum in $MeV$.

 **Returns**
    - The transverse polarized tagged deuteron structure function $F_{[UU,L]d}$ in $MeV^{-2}$.    
<br>
<br>
6. FLd():
<br> **Input**:
    - x: &emsp;&emsp;&emsp; (float) The effective Bjorken variable for scattering from a nucleon in the deuteron in the absence of nuclear binding.
    - Q2: &emsp;&emsp;      (float) The negative of the 4 momentum transfer squared or virtual photon momentum squared in $MeV^2$.
    - alpha_p:              (float) The proton light cone momentum fraction.
    - p_pt: &emsp;&nbsp;    (float) The transverse proton momentum in $MeV$.

 **Returns**
    - The longitudinal polarized tagged deuteron structure function $F_{[UU,L]d}$ in $MeV^{-2}$.
***
### Remarks:
1. The conventional Bjorken variable for scattering on the deuteron, $x_d$, is given by:
\begin{equation}
    x_d = \frac{Q^2}{2(p_dq)} \Rightarrow x = 2x_d
\end{equation}
And is related to the effective Bjorken variable for scattering from a nucleon in the deuteron in the absence of nuclear binding $x$.

2. The unpolarized structure functions of the deuteron and the neutron, $F_1$ and $F_2$, are related to each other through the unpolarized momentum distribution as follows:
\begin{equation}
    F_{1d}(x,Q^2;\alpha_p,p_{pT}) = [2(2\pi)^3]\frac{2f_d(\alpha_p,p_{pT})}{2-\alpha_p}F_{1n}(\tilde{x},Q^2) 
\end{equation}
\begin{equation}
    F_{2d}(x,Q^2;\alpha_p,p_{pT}) = [2(2\pi)^3]f_d(\alpha_p,p_{pT})F_{2n}(\tilde{x},Q^2) 
\end{equation}
Where $\tilde{x}$ is the effective scaling variable for scattering on the neuteron, taken the longitudinal LF momentum in the deuteron into account as follows:
\begin{equation}
    \tilde{x} = \frac{x}{2-\alpha_p}
\end{equation}

3. The transverse and longitudinal polarized deuteron structure functions are related to the unpolarized ones as:
\begin{equation}
    F_{[UU,T]d} = 2F_{1d} \quad F_{[UU,L]d} = (1+\gamma^2)\frac{F_{2d}}{x_d} - 2F_{1d}
\end{equation}
Where $\gamma^2$ is defined as:
\begin{equation}
    \gamma^2 = \frac{4x_d^2M_d^2}{Q^2}
\end{equation}


In [14]:
class struc_func:        

    def F1n(self,x_tilde,Q2):
        """
        the conventional dimensionlesss unpolarized nucleon structure function F_{1n}
        
        x_tilde (float)       : the effective scaling variable for scattering on the neutron, which takes into 
                                account its longitudinal LF momentum in the deuteron.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        
        returns F_{1n}
        """
        structure_funcs = subprocess.check_output("{} f {} {} {}".format(self.program, self.SF_parametriation,\
                                                                        x_tilde, Q2),shell=True).decode("ascii")
        return float(structure_funcs.split(" ")[0])
    
    def F2n(self,x_tilde,Q2):
        """
        the conventional dimensionlesss unpolarized nucleon structure function F_{2n}
        
        x_tilde (float)       : the effective scaling variable for scattering on the neutron, which takes into 
                                account its longitudinal LF momentum in the deuteron.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        
        returns F_{2n}
        """
        structure_funcs = subprocess.check_output("{} f {} {} {}".format(self.program, self.SF_parametriation,\
                                                                        x_tilde, Q2),shell=True).decode("ascii")
        return float(structure_funcs.split(" ")[1])
    
    def FTn(self,x_tilde,Q2):
        """
        the dimensionless transverse polarized neuteron structure function F_{[UU,T]n} = 2F_{1n}
        
        x_tilde (float)       : the effective scaling variable for scattering on the neutron, which takes into 
                                account its longitudinal LF momentum in the deuteron.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        
        returns F_{[UU,T]n}
        """
        return 2.*self.F1n(x_tilde,Q2)
    
    def FLn(self,x_tilde,Q2):
        """
        the dimensionless longitudinal polarized neuteron structure function 
        F_{[UU,L]n} = (1+\gamma^2)\frac{F_{2n}{\tilde{x}}- 2F_{1n}, where \gamma^2 = 4*\tilde{x}^2M_N^2/Q^2
        
        x_tilde (float)       : the effective scaling variable for scattering on the neutron, which takes into 
                                account its longitudinal LF momentum in the deuteron.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        
        returns F_{[UU,L]n}
        """
        gamma2 = (2.*x_tilde*MN)**2/Q2
        return (1.+gamma2)*self.F2n(x_tilde,Q2)/x_tilde - 2.*self.F1n(x_tilde,Q2)
    
    def F1d(self,x,Q2,alpha_p,p_pt):
        """
        the unpolarized tagged deuteron structure function F_{1d} in MeV^{-2} calculated using the unpolarized 
        momentum distribution and the conventional unpolarized nucleon structure function F_{1n}
        F_{1d}(x,Q^2;\alpha_p,p_pt) = [2(2\pi)^3]\frac{2f_d(\alpha_p,p_pt)[unpol]}{2-\alpha_p}F_{1n}(\tide{x},Q^2)
        
        x       (float)       : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                the absence of nuclear binding.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        alpha_p (float)       : the proton light cone momentum fraction.
        p_pt   (float)(MeV)  : the transverse proton momentum
        
        returns F_{1d} in MeV^{-2}
        """
        x_tilde = x/(2.-alpha_p)
        momentum_dist = fd_unpol(alpha_p, p_pt, self.WF_parametrization)
        return (2.*(2.*np.pi)**3)*((2.*momentum_dist)/(2.-alpha_p))*self.F1n(x_tilde,Q2) #in MeV^{-2}
    
    def F2d(self,x,Q2,alpha_p,p_pt):
        """
        the unpolarized tagged deuteron structure function F_{2d} in MeV^{-2} calculated using the unpolarized 
        momentum distribution and the conventional unpolarized nucleon structure function F_{2n}
        F_{2d}(x,Q^2;\alpha_p,p_pt) = [2(2\pi)^3]f_d(\alpha_p,p_pt)[unpol]F_{2n}(\tide{x},Q^2)
                
        x       (float)       : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                the absence of nuclear binding.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        alpha_p (float)       : the proton light cone momentum fraction.
        p_pt   (float)(MeV)  : the transverse proton momentum
        
        returns F_{2d} in MeV^{-2}
        """
        x_tilde = x/(2.-alpha_p)
        momentum_dist = fd_unpol(alpha_p, p_pt,self.WF_parametrization)
        return (2.*(2.*np.pi)**3)*momentum_dist*self.F2n(x_tilde,Q2)
    
    def FTd(self,x,Q2,alpha_p,p_pt):
        """
        the transverse tagged deuteron structure function F_{[UU,T]d} in MeV^{-2} calculated using the 
        conventional unpolarized deuteron structure function F_{1d}
        F_{[UU,T]d}(x,Q^2;\alpha_p,p_pt) = 2F_{1d}(x,Q^2;\alpha_p,p_pt)
                
        x       (float)       : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                the absence of nuclear binding.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        alpha_p (float)       : the proton light cone momentum fraction.
        p_pt   (float)(MeV)  : the transverse proton momentum
        
        returns F_{[UU,T]d} in MeV^{-2}
        """
        return 2.*self.F1d(x,Q2,alpha_p,p_pt)
        
    def FLd(self,x,Q2,alpha_p,p_pt):
        """
        the longitudinal tagged deuteron structure function F_{[UU,T]d} in MeV^{-2} calculated using the 
        conventional unpolarized deuteron structure functions F_{1d} and F_{2d}
        F_{[UU,L]d}(x,Q^2;\alpha_p,p_pt) = (1+\gamma^2)\frac{F_{2d}(x,Q^2;\alpha_p,p_pt)}{x_d}-
                                                                                    2F_{1d}(x,Q^2;\alpha_p,p_pt)
                
        x       (float)       : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                the absence of nuclear binding.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        alpha_p (float)       : the proton light cone momentum fraction.
        p_pt   (float)(MeV)  : the transverse proton momentum
        
        returns F_{[UU,L]d} in MeV^{-2}
        """
        x_d = x/2.
        gamma2 = (x*MD)**2/Q2 #2x_d = x
        return (1.+gamma2)*self.F2d(x,Q2,alpha_p,p_pt)/x_d - 2.*self.F1d(x,Q2,alpha_p,p_pt)
    
    def FTLLT(self,x,Q2,alpha_p,p_pt):
        """
        the transverse tensor tagged deuteron structure function T_{LL}F_{[UT_{LL},T]d} in MeV^{-2} calculated 
        using the transverse polarized neuteron structure function F_{[UU,T]n}
        T_{LL}F_{[UT_{LL},T]d}(x,Q^2;\alpha_p,p_pt) = [2(2\pi)^3]\frac{2f_d(\alpha_p,p_pt)[tensor]}{2-\alpha_p}
                                                      F_{[UU,T]n}(\tide{x},Q^2)
                
        x       (float)       : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                the absence of nuclear binding.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        alpha_p (float)       : the proton light cone momentum fraction.
        p_pt   (float)(MeV)  : the transverse proton momentum
        
        returns F_{[UT_{LL}]d} in MeV^{-2}
        """
        x_tilde = x/(2.-alpha_p)
        momentum_dist = fd_tensor(alpha_p, p_pt, self.WF_parametrization)
        return (2.*(2.*np.pi)**3)*((2.*momentum_dist)/(2.-alpha_p))*self.FTn(x_tilde,Q2) #in MeV^{-2}
                                  
    def FTLLL(self,x,Q2,alpha_p,p_pt):
        """
        the longitudinal tensor tagged deuteron structure function T_{LL}F_{[UT_{LL},L]d} in MeV^{-2} calculated 
        using the transverse polarized neuteron structure function F_{[UU,T]n}
        T_{LL}F_{[UT_{LL},L]d}(x,Q^2;\alpha_p,p_pt) = [2(2\pi)^3]\frac{2f_d(\alpha_p,p_pt)[tensor]}{2-\alpha_p}
                                                      F_{[UU,L]n}(\tide{x},Q^2)
                
        x       (float)       : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                the absence of nuclear binding.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        alpha_p (float)       : the proton light cone momentum fraction.
        p_pt   (float)(MeV)  : the transverse proton momentum
        
        returns F_{[UT_{LL}]d} in MeV^{-2}
        """
        x_tilde = x/(2.-alpha_p)
        momentum_dist = fd_tensor(alpha_p, p_pt, self.WF_parametrization)
        return (2.*(2.*np.pi)**3)*((2.*momentum_dist)/(2.-alpha_p))*self.FLn(x_tilde,Q2) #in MeV^{-2}
                                  
    def g1n(self,x_tilde,Q2):
        """
        the conventional dimensionlesss nucleon structure function g_{1n}
        
        x_tilde (float)       : the effective scaling variable for scattering on the neutron, which takes into 
                                account its longitudinal LF momentum in the deuteron.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        
        returns g_{1n}
        """
        structure_funcs = subprocess.check_output("{} g {} {} {}".format(self.program, self.SF_parametriation,\
                                                                        x_tilde, Q2),shell=True).decode("ascii")
        return float(structure_funcs.split(" ")[0])
    
    def g2n(self,x_tilde,Q2):
        """
        the conventional dimensionlesss nucleon structure function g_{2n}
        
        x_tilde (float)       : the effective scaling variable for scattering on the neutron, which takes into 
                                account its longitudinal LF momentum in the deuteron.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        
        returns g_{2n}
        """
        structure_funcs = subprocess.check_output("{} g {} {} {}".format(self.program, self.SF_parametriation,\
                                                                        x_tilde, Q2),shell=True).decode("ascii")
        return float(structure_funcs.split(" ")[1])
    
    def FSLn(self,x_tilde,Q2):
        """
        the dimensionless transverse polarized neuteron structure function F_{[UU,T]n} = 2F_{1n}
        
        x_tilde (float)       : the effective scaling variable for scattering on the neutron, which takes into 
                                account its longitudinal LF momentum in the deuteron.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        
        returns F_{[UU,T]n}
        """
        gamma2 = (2*x_tilde*MN)**2/Q2
        return 2*(self.g1n(x_tilde,Q2)-gamma2*g2n)
    
    def FSTn(self,x_tilde,Q2):
        """
        the dimensionless longitudinal polarized neuteron structure function 
        F_{[UU,L]n} = (1+\gamma^2)\frac{F_{2n}{\tilde{x}}- 2F_{1n}, where \gamma^2 = 4*\tilde{x}^2M_N^2/Q^2
        
        x_tilde (float)       : the effective scaling variable for scattering on the neutron, which takes into 
                                account its longitudinal LF momentum in the deuteron.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        
        returns F_{[UU,L]n}
        """
        gamma2 = (2*x_tilde*MN)**2/Q2
        return -2*np.sqrt(gamma2)*(g1n+g2n)
    
    def g1d(self,x,Q2,alpha_p,p_pt):
        """
        the tagged deuteron spin structure function g_{1d} in MeV^{-2} calculated using the vector polarized 
        momentum distribution and the spin nucleon structure function g_{1n}
        g_{1d}(x,Q^2;\alpha_p,p_pt) = [2(2\pi)^3]\frac{2\Delta f_d(\alpha_p,p_pt)[pure +1]}{2-\alpha_p}
                                      g_{1n}(\tide{x},Q^2)
        
        x       (float)       : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                the absence of nuclear binding.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        alpha_p (float)       : the proton light cone momentum fraction.
        p_pt   (float)(MeV)  : the transverse proton momentum
        
        returns g_{1d} in MeV^{-2}
        """
        x_tilde = x/(2.-alpha_p)
        momentum_dist = delta_fd(alpha_p, p_pt, self.WF_parametrization)
        return (2.*(2.*np.pi)**3)*((2.*momentum_dist)/(2.-alpha_p))*self.g1n(x_tilde,Q2) #in MeV^{-2}
    
    #g_2 is suppressed
    
    def FSLd(self,x,Q2,alpha_p,p_pt):
        """
        the polarized tagged deuteron structure function F_{[LS_L]d} in MeV^{-2} calculated using the 
        spin deuteron structure functions g_{1d} and g_{2d}
        g_{[LS_L]d}(x,Q^2;\alpha_p,p_pt) = 2(g_{1d}-\gamma^2g_{2d})
                
        x       (float)       : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                the absence of nuclear binding.
        Q2      (float)(MeV^2): the negative of the 4 momentum transfer squared or virtual photon momentum squared
        alpha_p (float)       : the proton light cone momentum fraction.
        p_pt   (float)(MeV)  : the transverse proton momentum
        
        returns F_{[LS_L]d} in MeV^{-2}
        """
        return 2.*self.g1d(x,Q2,alpha_p,p_pt)    
    
    #__________________________________________________
    def __init__(self, SF_parametriation = "CTEQ", WF_parametrization = "first"):
        self.program = os.path.abspath(os.path.join(os.path.abspath(""), os.pardir,"cmake_program","build","bin",
                                                    "structure_function"))
        
        self.SF_parametriation  = SF_parametriation.upper() #StructureFunction parametriation
        self.WF_parametrization = WF_parametrization.lower() #WaveFunction parametrizatian


Test zone
__________________________________________________________________________________________________________

This is still under review

# diff_cross_sec():
## Function to calculate the differential cross section.
***
### Input
1. s_ed: &emsp;&emsp;&emsp;&emsp;&ensp;&nbsp;    (float) 
2. x: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp; (float) &nbsp;    The effective Bjorken variable for scattering from a nucleon in the deuteron in the absence of nuclear binding.
3. Q2: &emsp;&emsp;&emsp;&emsp;&emsp;&ensp;      (float) &nbsp;    The negative of the 4 momentum transfer squared or virtual photon momentum squared in $MeV^2$.
4. alpha_p_range:&nbsp;                          (array)&nbsp;     Start and stop value for the interval of proton light cone momentum fraction in an array.
5. t_apo_range: &emsp;                           (array)&nbsp;     Array of invariant momentum transfer between the deuteron and the proton minus the proton mass, defined as t'.
6. parametriation: &nbsp;                        (string)&nbsp;    The name of the parametriation used to calculate the structure functions. Standard value is "CTEQ", for other parametriations see struc_func.
7. parametrization:                              (string)&nbsp;    The used AV18 parametrization (first, second). If not specified the first one is used.
8. N: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;       (int) &emsp;&ensp; The amount of values over which to average out.
***
### Returns
***
### Remarks
1. The invariant momentum transfer between the deuteron and proton, the t channel, is defined as:
\begin{equation}
    t = (p_d-p_p)^2 \quad t' = t - m
\end{equation}
2. And the transverse momentum can be calculated using this invariant momentum transfer and the proton light cone momentum fraction as follows:
\begin{equation}
    t' = \frac{\alpha_p(2-\alpha_p)M_d^2-4M_N^2}{2\alpha_p}-\frac{2|p_{pT}|^2}{\alpha_p} = t'_{min}(\alpha_p) - \frac{2|p_{pT}|^2}{\alpha_p}
\end{equation}
\begin{equation}
    \Rightarrow p_{pT}^2 = \frac{1}{4}\alpha_p(2-\alpha_p)M_d^2-M_N^2-\frac{\alpha_pt'}{2} = \frac{\alpha_p}{2}\left(t'_{min}(\alpha_p) - t'\right)
\end{equation}
3. The differential cross section is defined as:
\begin{equation}
    d\sigma = \frac{2\pi\alpha_{em}^2y^2}{Q^4(1-\epsilon)} dx_ddQ^2 \left(F_{Td}(x,Q^2;\alpha_p,p_{pT})+\epsilon F_{Ld}(x,Q^2;\alpha_p,p_{pT})\right)\frac{1}{2(2\pi)^3}\frac{d^3p_{p}}{E_p}
\end{equation}
\begin{equation}
    = \frac{1}{16\pi^2}\frac{\alpha_{em}^2y^2}{Q^4(1-\epsilon)} dxdQ^2 \left(F_{Td}(x,Q^2;\alpha_p,p_{pT})+\epsilon F_{Ld}(x,Q^2;\alpha_p,p_{pT})\right)\frac{d^3p_{p}}{E_p}
\end{equation}
\begin{equation}
    = \frac{1}{16\pi^2}\frac{\alpha_{em}^2y^2}{Q^4(1-\epsilon)} dxdQ^2 \left(F_{Td}(x,Q^2;\alpha_p,p_{pT})+\epsilon F_{Ld}(x,Q^2;\alpha_p,p_{pT})\right)\frac{d\alpha_p}{\alpha_p}d^2p_{pT}
\end{equation}
Where we used that $x = 2x_d$. Also $\alpha_{em}$ is the fine structure constant, $\epsilon$ the virtual photon polarization parameter:
\begin{equation}
    \epsilon = \frac{1-y-\gamma^2y^2/4}{1-y+y^2/2+\gamma^2y^2/4}
\end{equation}
And $y$ the electron fractional energy loss:
\begin{equation}
    y = \frac{p_dq}{p_dp_e} = \frac{Q^2}{x_d(s_{ed}-M_d^2)}
\end{equation}
With $s_{ed}$ the electron-deuteron squared CM energy $(s_{ed} = (p_e + p_d)^2)$.
4. Now using the relation between the transverse momentum, the proton light cone momentum fraction and the invariant momentum t', the derivative $d^2p_{pT}$ can be rewritten so that the differential cross section is a function of t'.
\begin{equation}
    dt' = -\frac{4}{\alpha_p}|p_{pT}|d|p_{pT}| \Rightarrow d^2p_{pT} = |p_{pT}|d|p_{pT}|d\phi = \frac{\alpha_p}{4}d(-t')d\phi \quad (\phi \in [0,2\pi[)
\end{equation}
\begin{equation}
    \Rightarrow \frac{d\alpha_p}{\alpha_p}d^2p_{pT} = \frac{1}{4}d\alpha_p d(-t')d\phi
\end{equation}
This gives for the differential cross section, after integration over $\phi$ which gives an additional 2$\pi$:
\begin{equation}
    \frac{d\sigma}{dxdQ^2} = \frac{1}{16\pi^2}\frac{\alpha_{em}^2y^2}{Q^4(1-\epsilon)} \left(F_{Td}(x,Q^2;\alpha_p,p_{pT})+\epsilon F_{Ld}(x,Q^2;\alpha_p,p_{pT})\right)\frac{\pi}{2}d\alpha_pd(-t')
\end{equation}
\begin{equation}
    = \frac{1}{32\pi}\frac{\alpha_{em}^2y^2}{Q^4(1-\epsilon)} \left(F_{Td}(x,Q^2;\alpha_p,p_{pT})+\epsilon F_{Ld}(x,Q^2;\alpha_p,p_{pT})\right)d\alpha_pd(-t')
\end{equation}

Where $x = 2x_d$ is used.


In [15]:
def diff_cross_sec(s_ed,x,Q2,alpha_p,x_val,x_parameter, SF_parametriation = "CTEQ",\
                   WF_parametrization = "first",sigma_dependency = "p", x_dependency = None):
    """
    The differential cross section in nb/MeV^4
    
    s_ed     (float)(MeV^2)    : the invariant electron deuteron squared centre of mass energy
    x        (float)           : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                 the absence of nuclear binding.
    Q2       (float)(MeV^2)    : the negative of the 4 momentum transfer squared or virtual photon momentum squared
    alpha_p  (float)           : the proton light cone momentum fraction
    x_val   (float)(MeV^2/MeV): the value that is used as the x-axis, this can either be an invariant momentum 
                                 transfer between the deuteron and the proton minus the proton mass, t', a 
                                 transverse momentum value, p_pt, or a norm of the proton/neuteron 3 momentum, k.
    x_parameter        (string): which parameter is used in x_val:
                                 - "t''": the invariant momentum transfer, t'
                                 - "p_pt" : the transverse momentum, p_{pt}
                                 - "k"    : the 3 momentum norm, k
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 If not specified the "first" one is used.
    sigma_dependency   (string): The used dependency of the differential cross section:
                                 - "p"    : \frac{d\sigma}{dxdQ^2\frac{d^3p}{E_p}} is returned
                                 - "t'": \frac{d\sigma}{dxdQ^2d\alpha_pd(-t')} is returned
                                 standard value is "t''"
    x_dependency       (string): the parameter in which x_val is returned:
                                 - "t'": the invariant momentum transfer, t'
                                 - "p_pt" : the transverse momentum, p_{pt}
                                 - "k"    : the 3 momentum norm, k
                                 if not specified x_parameter is used.
    
    returns a 2,2 numpy array with the first line the x_parameter with units and the sigma_dependency with units 
    and the second line the x_val and the differential cross section in nb/MeV^4
    e.g.:
    diff_cross_sec(s_ed= 1e9, x= 0.05, Q2= 35e6, alpha_p= 1., x_val= -5e3, x_parameter= "t'")
    [["t' [MeV^2]" '\\frac{d\\sigma}{dxdQ^2\\frac{d^3p}{E_p}} [nb/MeV^4]']
     ['-5000.0' '9.604651453371215e-10']]
    diff_cross_sec(s_ed= 1e9, x= 0.05, Q2= 35e6, alpha_p= 1., x_val= -5e3, x_parameter= "t'",x_dependency= "k")
    [['k [MeV]' '\\frac{d\\sigma}{dxdQ^2\\frac{d^3p}{E_p}} [nb/MeV^4]']
     ['20.311335077890654' '9.05219507383359e-10']]
    """
    #conversion between parameters_____________________
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_val)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_val**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def t_apo():
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*x_val**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*x_val**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def k():
        if x_parameter == "t'":
            return np.sqrt(MD**2-4*MN**2-2*x_val/(2-alpha_p))/4
        elif x_parameter == "p_pt":
            return np.sqrt((x_val**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2)
        elif x_parameter == "k":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    #calculation of transverse momentum p_pt___________
    p_pt = p_trans()
    
    #calculation of x dependency value_________________
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        x_dep_val = t_apo()
        header = [r"t' [MeV^2]"]
    elif x_dependency == "p_pt":
        x_dep_val = p_trans()
        header = [r"p_{pt} [MeV]"]
    elif x_dependency == "k":
        x_dep_val = k()
        header = [r"k [MeV]"]
    else:
        raise ValueError("no x dependency called {}, choose from t_apo, p_pt, k".format)
        
    #kinematic variables_______________________________
    s_eN    = s_ed/2
    x_d     = x/2.
    x_tilde = x/(2.-alpha_p)
    y       = Q2/(x_d*(s_ed-MD**2))
    gamma2  = (x*MD)**2/Q2 #2x_d = x
    epsilon = (1-y-gamma2*y**2/4)/(1-y+y**2/2+gamma2*y**2/4)
    k2      = (p_pt**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2
    EN      = np.sqrt(MN**2+k2)
    
    #structure function________________________________
    struc_f = struc_func(SF_parametriation, WF_parametrization)
    F = struc_f.FTd(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FLd(x,Q2,alpha_p,p_pt)
    
    #front factor______________________________________
    front_factor = alpha_em**2*y**2/(Q2**2*(1-epsilon))
    
    #norm factor relies on the used dependency_________
    if sigma_dependency == "p": #dependent on d^3p/E_p
        norm_factor = 1/(16*np.pi**2)
        header += [r"\frac{d\sigma}{dxdQ^2\frac{d^3p}{E_p}} [nb/MeV^4]"]
    elif sigma_dependency == "t'": #dependent on -t'
        norm_factor  = 1/(32*np.pi)
        header += [r"\frac{d\sigma}{dxdQ^2d\alpha_pd(-t')} [nb/MeV^4]"]
    else:
        raise ValueError("No dependecy on {}".format(sigma_dependecy))
    
    #returning in nb/MeV^4_____________________________
    d_sigma = fgHbarc**2*10**7*norm_factor*front_factor*F
    
    return np.array([header,[x_dep_val,d_sigma]])


In [16]:
def plot_diff_cross_sec(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,\
                        SF_parametriation = "CTEQ", WF_parametrization = "first",sigma_dependency="p",\
                        x_dependency=None,save="",log=True):
    """
    plots the differential cross section in nb/GeV^4 for the bin average values in function of the values in 
    x_val_interval given in the form specified by x_dependency
    
    L_int   (float)(nb^-1)       : the integrated luminosity
    s_ed    (float)(MeV^2)       : the invariant electron deuteron squared centre of mass energy
    x_range (array[float])       : begin and end value of the bin of the effective Bjorken variable for 
                                   scattering from a nucleon in the deuteronin the absence of nuclear binding.
    Q2_range(array[float])(MeV^2): begin and end value of the bin of the negative of the 4 momentum transfer 
                                   squared or virtual photon momentum squared
    alpha_p_range (array[float]) : begin and end value of the bin of the proton light cone momentum fraction
    x_val_range   (array[float])(MeV^2/MeV): an array of evenly spaced values that creates bins of x_vals which 
                                             will be used as the x-axis, these can either be an invariant momentum 
                                             transfer between the deuteron and the proton minus the proton mass, 
                                             t', a transverse momentum value, p_pt, or a norm of the
                                             proton/neuteron 3 momentum, k.
    x_parameter        (string): which parameter is used in x_val:
                                 - "t''": the invariant momentum transfer, t'
                                 - "p_pt" : the transverse momentum, p_{pt}
                                 - "k"    : the 3 momentum norm, k
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 if not specified the "first" one is used.
    sigma_dependency   (string): the used dependency of the differential cross section:
                                 - "p"    : \frac{d\sigma}{dxdQ^2\frac{d^3p}{E_p}} is returned
                                 - "t'": \frac{d\sigma}{dxdQ^2d\alpha_pd(-t')} is returned
                                 standard value is "t_apo"
    x_dependency       (string): the dependency on the x-axis of the plot. the values in x_val_interval do not 
                                 have to be the same parameters as the ones that need to be plotted:
                                 - "t'": the invariant momentum transfer, t'
                                 - "p_pt" : the transverse momentum, p_{pt}
                                 - "k"    : the 3 momentum norm, k
                                 if not specified x_parameter is used.
    save (string): if the figure need to be saved give the name of the figure
    log  (bool)  : use a logaritmic y-axis, standard value is True
    
    returns the figure
    
    plot_diff_cross_sec(L_int=1e6,s_ed=2e9,x_range=np.array([0.04,0.06]),Q2_range=np.array([30,40])*10**6,\
                        alpha_p_range=np.array([0.98,1.02]),x_val_range=np.array([-5000,-10000,-15000,-20000]),\
                        x_parameter="t'")
    [["$-t' [GeV^2]$" '$d\\sigma/dxdQ^2(d^3p_n/E_n) [nb/GeV^{4}]$' 'error [nb/GeV^4]']
     ['0.0075' '411.84845661664525' '3.20877101324107']
     ['0.012499999999999999' '138.27169687830488' '1.8592451215365926']
     ['0.017499999999999998' '65.96068089454504' '1.284140577337086']]
    """
    x_val_bins = list(zip(x_val_range[:-1],x_val_range[1:]))
    x_vals = np.mean(x_val_bins,axis=1)
    
    #bin averages______________________________________
    x       = np.mean(x_range)
    Q2      = np.mean(Q2_range)
    alpha_p = np.mean(alpha_p_range)
    x_vals  = np.mean(np.append([x_val_range[:-1]],[x_val_range[1:]],axis=0),axis=0)
    
    #bin sizes_________________________________________
    delta_x       = x_range[1]-x_range[0]
    delta_Q2      = Q2_range[1]-Q2_range[0]
    delta_alpha_p = alpha_p_range[1]-alpha_p_range[0]
    delta_x_val   = np.array([abs(x_bin[1]-x_bin[0]) for x_bin in x_val_bins])
    
    def t_apo():
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2)/(2*alpha_p)-2*x_val_range**2/alpha_p
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*x_val_range**2-4*MN**2)/2
        elif x_parameter == "t'":
            return x_val_range
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format)
    def prog_bar(i,total):
        clear_output(wait=True)
        bar = widgets.FloatProgress(value=i,min=0,max=total,description='Loading: {:.2f}%'.format(i/total*100),\
                                    bar_style='info',orientation='horizontal',\
                                    style = {'description_width': "initial"})
        display(bar)

    #differential cross sections_______________________
    values = []
    prog_bar(0,len(x_vals))
    for i in range(len(x_vals)):
        values += [diff_cross_sec(s_ed,x,Q2,alpha_p,x_vals[i],x_parameter,SF_parametriation,\
                                  WF_parametrization,sigma_dependency,x_dependency)[1]]
        prog_bar(i+1,len(x_vals))
    values = np.array(values).astype(float).T
    
    #calculating the error
    delta = delta_x*delta_Q2
    if x_parameter == "t'":
        if sigma_dependency == "p": #if the sigma is dependent on p_pt transform to dependence on t'
            delta *= np.pi/2 #conversion between sigma dependencies
        delta *= delta_alpha_p*delta_x_val
    elif x_parameter == "p_pt" or x_parameter == "k":
        t = t_apo()
        if sigma_dependency == "p":  #if the sigma is dependent on p_pt transform to dependence on t'
            delta *= np.pi/2 #conversion between sigma dependencies
        delta_t = np.array([abs(t[i+1]-t[i]) for i in range(len(t)-1)])
        delta *= delta_alpha_p*delta_t
    else:
        raise ValueError("no x_parameter called {}".format(x_parameter))
            
    delta_n     = L_int*delta*values[1]
    rel_error   = 1/np.sqrt(delta_n)
    values      = np.append(values,[values[1]*rel_error],axis=0)
    
    #differential cross section values will be shown in nb/GeV^4
    values[1]*= 10**12 #from nb/MeV^4 to nb/GeV^4
    values[2]*= 10**12 #from nb/MeV^4 to nb/GeV^4
    
    #getting x label and right units___________________
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        #momentum transfers are given in GeV^2
        values[0] *= -10**-6
        x_label = r"$-t' [GeV^2]$"
    elif x_dependency == "p_pt":
        #transverse momenta are given in GeV
        values[0] *= 10**-3
        x_label = r"$p_pt [GeV]$"
    elif x_dependency == "k":
        #momenta norm are given in GeV
        values[0] *= 10**-3
        x_label = r"$|k| [GeV]$"
    else:
        raise ValueError("no x_dependecy called {}, choose from t',p_pt,k".format(x_dependency))

    if sigma_dependency == "p":
        y_label = r"$d\sigma/dxdQ^2(d^3p_n/E_n) [nb/GeV^{4}]$"
    elif sigma_dependency == "t'":
        y_label = r"$d\sigma/dxdQ^2(d\alpha_p d(-t')) [nb/GeV^{4}]$"
    else:
        raise ValueError("no sigma_dependency called {}".format(sigma_dependency))
        
    #plotting__________________________________________
    fig, ax = plt.subplots(figsize=(8,4))
    ax.errorbar(values[0],values[1],yerr=values[2],linestyle = "",marker="+",capsize=3,\
                label = r"$\alpha_p = {}-{}$".format(alpha_p_range[0],alpha_p_range[1])) #
    ax.plot([], [], ' ', label=r"$L_{{int}} = {}\cdot 1O^6 nb^{{-1}}$""\n"r"$S_{{eN}} = {} GeV^2$""\n"\
             r"$x = {}-{}$""\n"r"$Q^2 = {}-{} GeV^2$".format(int(L_int*10**-6),int(s_ed*10**-6/2),x_range[0],
                                                      x_range[1],Q2_range[0]*10**-6,Q2_range[1]*10**-6))
    ax.set_ylabel(y_label)
    ax.set_xlabel(x_label)
    ax.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    plt.tight_layout()
    ax.grid(True)
    if log:
        ax.set_yscale("log")
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    
    return fig

In [17]:
def plot_diff_cross_sec_MC(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,\
                           SF_parametriation = "CTEQ", WF_parametrization = "first",sigma_dependency="p",\
                           x_dependency=None,save="",log=True,nitn=1,neval=1):
    """
    plots the differential cross section in nb/GeV^4 for the bin average values in function of the values in 
    x_val_interval given in the form specified by x_dependency
    
    L_int   (float)(nb^-1)       : the integrated luminosity
    s_ed    (float)(MeV^2)       : the invariant electron deuteron squared centre of mass energy
    x_range (array[float])       : begin and end value of the bin of the effective Bjorken variable for 
                                   scattering from a nucleon in the deuteron the absence of nuclear binding.
    Q2_range(array[float])(MeV^2): begin and end value of the bin of the negative of the 4 momentum transfer 
                                   squared or virtual photon momentum squared
    alpha_p_range (array[float]) : begin and end value of the bin of the proton light cone momentum fraction
    x_val_range   (array[float])(MeV^2/MeV): an array of evenly spaced values that creates bins of x_vals which 
                                             will be used as the x-axis, these can either be an invariant momentum 
                                             transfer between the deuteron and the proton minus the proton mass, 
                                             t', a transverse momentum value, p_pt, or a norm of the
                                             proton/neuteron 3 momentum, k.
    x_parameter        (string): which parameter is used in x_val:
                                 - "t''": the invariant momentum transfer, t'
                                 - "p_pt" : the transverse momentum, p_{pt}
                                 - "k"    : the 3 momentum norm, k
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 if not specified the "first" one is used.
    sigma_dependency   (string): the used dependency of the differential cross section:
                                 - "p"    : \frac{d\sigma}{dxdQ^2\frac{d^3p}{E_p}} is returned
                                 - "t'": \frac{d\sigma}{dxdQ^2d\alpha_pd(-t')} is returned
                                 standard value is "t_apo"
    x_dependency       (string): the dependency on the x-axis of the plot. the values in x_val_interval do not 
                                 have to be the same parameters as the ones that need to be plotted:
                                 - "t'": the invariant momentum transfer, t'
                                 - "p_pt" : the transverse momentum, p_{pt}
                                 - "k"    : the 3 momentum norm, k
                                 if not specified x_parameter is used.
    save (string): if the figure need to be saved give the name of the figure
    log  (bool)  : use a logaritmic y-axis, standard value is True
    
    returns the figure
    
    plot_diff_cross_sec(L_int=1e6,s_ed=2e9,x_range=np.array([0.04,0.06]),Q2_range=np.array([30,40])*10**6,\
                        alpha_p_range=np.array([0.98,1.02]),x_val_range=np.array([-5000,-10000,-15000,-20000]),\
                        x_parameter="t'")
    [["$-t' [GeV^2]$" '$d\\sigma/dxdQ^2(d^3p_n/E_n) [nb/GeV^{4}]$' 'error [nb/GeV^4]']
     ['0.0075' '411.84845661664525' '3.20877101324107']
     ['0.012499999999999999' '138.27169687830488' '1.8592451215365926']
     ['0.017499999999999998' '65.96068089454504' '1.284140577337086']]
    """
    x_val_bins = list(zip(x_val_range[:-1],x_val_range[1:]))
    x_vals = np.mean(x_val_bins,axis=1)
    
    #bin sizes_________________________________________
    delta_x       = x_range[1]-x_range[0]
    delta_Q2      = Q2_range[1]-Q2_range[0]
    delta_alpha_p = alpha_p_range[1]-alpha_p_range[0]
    delta_x_val   = np.array([abs(x_bin[1]-x_bin[0]) for x_bin in x_val_bins])
    
    values = []
    
    #conversion between parameters_____________________
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_vals)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_vals**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def t_apo():
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*x_vals**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*x_vals**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def k():
        if x_parameter == "t'":
            return np.sqrt(MD**2-4*MN**2-2*x_vals/(2-alpha_p))/4
        elif x_parameter == "p_pt":
            return np.sqrt((x_vals**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2)
        elif x_parameter == "k":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    #calculation of x dependency value_________________
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        values += [t_apo()]
        header = [r"t' [MeV^2]"]
    elif x_dependency == "p_pt":
        values += [p_trans()]
        header = [r"p_{pt} [MeV]"]
    elif x_dependency == "k":
        values += [k()]
        header = [r"k [MeV]"]
    else:
        raise ValueError("no x dependency called {}, choose from t_apo, p_pt, k".format)
    
    def prog_bar(i,total):
        clear_output(wait=True)
        bar = widgets.FloatProgress(value=i,min=0,max=total,description='Loading: {:.2f}%'.format(i/total*100),\
                                    bar_style='info',orientation='horizontal',\
                                    style = {'description_width': "initial"})
        display(bar)
    
    def sigma(data):
        x_t       = data[0]
        Q2_t      = data[1]
        alpha_p_t = data[2]
        x_val_t   = data[3]
        
        diff_cross = float(diff_cross_sec(s_ed,x_t,Q2_t,alpha_p_t,x_val_t,x_parameter,SF_parametriation,\
                                    WF_parametrization,sigma_dependency,x_dependency)[1][1])
        return diff_cross
    
    #MC integration
    prog_bar(0,len(x_val_bins))
    results = []
    for i in range(len(x_val_bins)):
        integ  = vegas.Integrator([x_range,Q2_range,alpha_p_range,x_val_bins[i]])
        result = integ(sigma,nitn=nitn,neval=neval)
        results += [result.mean]
        prog_bar(i+1,len(x_val_bins))
    values += [results]
    del results
    values = np.array(values)
    
    #calculating the error
    delta = delta_x*delta_Q2
    if x_parameter == "t'":
        if sigma_dependency == "p": #if the sigma is dependent on p_pt transform to dependence on t'
            delta *= np.pi/2 #conversion between sigma dependencies
        delta *= delta_alpha_p*delta_x_val
    elif x_parameter == "p_pt" or x_parameter == "k":
        t = t_apo()
        if sigma_dependency == "p":  #if the sigma is dependent on p_pt transform to dependence on t'
            delta *= np.pi/2 #conversion between sigma dependencies
        delta_t = np.abs(np.mean([t[i+1]-t[i] for i in range(len(t)-1)]))
        delta *= delta_alpha_p*delta_t
    else:
        raise ValueError("no x_parameter called {}".format(x_parameter))
            
    delta_n     = L_int*values[1]
    rel_error   = 1/np.sqrt(delta_n)
    values      = np.append(values,[values[1]*rel_error],axis=0)
    values[1]  /= delta
    values[2]  /= delta
    
    #differential cross section values will be shown in nb/GeV^4
    values[1]*= 10**12 #from nb/MeV^4 to nb/GeV^4
    values[2]*= 10**12 #from nb/MeV^4 to nb/GeV^4
    
    #getting x label and right units___________________
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        #momentum transfers are given in GeV^2
        values[0] *= -10**-6
        x_label = r"$-t' [GeV^2]$"
    elif x_dependency == "p_pt":
        #transverse momenta are given in GeV
        values[0] *= 10**-3
        x_label = r"$p_pt [GeV]$"
    elif x_dependency == "k":
        #momenta norm are given in GeV
        values[0] *= 10**-3
        x_label = r"$|k| [GeV]$"
    else:
        raise ValueError("no x_dependecy called {}, choose from t',p_pt,k".format(x_dependency))

    if sigma_dependency == "p":
        y_label = r"$d\sigma/dxdQ^2(d^3p_n/E_n) [nb/GeV^{4}]$"
    elif sigma_dependency == "t'":
        y_label = r"$d\sigma/dxdQ^2(d\alpha_p d(-t')) [nb/GeV^{4}]$"
    else:
        raise ValueError("no sigma_dependency called {}".format(sigma_dependency))
        
    #plotting__________________________________________
    fig, ax = plt.subplots(figsize=(8,4))
    ax.errorbar(values[0],values[1],yerr=values[2],linestyle = "",marker="+",capsize=3,\
                label = r"$\alpha_p = {}-{}$".format(alpha_p_range[0],alpha_p_range[1])) #
    ax.plot([], [], ' ', label=r"$L_{{int}} = {}\cdot 1O^6 nb^{{-1}}$""\n"r"$S_{{eN}} = {} GeV^2$""\n"\
             r"$x = {}-{}$""\n"r"$Q^2 = {}-{} GeV^2$".format(int(L_int*10**-6),int(s_ed*10**-6/2),x_range[0],
                                                      x_range[1],Q2_range[0]*10**-6,Q2_range[1]*10**-6))
    ax.set_ylabel(y_label)
    ax.set_xlabel(x_label)
    ax.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    plt.tight_layout()
    ax.grid(True)
    if log:
        ax.set_yscale("log")
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    plt.show()
    
    return fig

In [137]:
def make_figure_cross_section(out):
    intro = widgets.HTMLMath(r"""<center><b><h1>Differential cross sections.</h1></b></center>
    <hr>
    <figure style="float: right;">
        <img src="figures/figure_cross_section.png" width ="400">
        <figcaption style="text-align:center;">fig: An example figure of the <b>Differential cross section</b> 
        program</figcaption>
    </figure>
    
    <p>&emsp;This cell can create a figure of the differential cross section $d\sigma[e+d\rightarrow e'Xp]$ for a 
    certain integrated luminosity, which can be given at <b>"$L_{int}$"</b>, and an electron-Nucleon center of 
    mass energy, provided at <b>"$s_{eN}$"</b>.</p>
    
    <p>&emsp;In the given interval for the Bjorken scaling variable, from 
    <b>"Bjorken $x$ lower bound"</b> to <b>"Bjorken $x$ upper bound"</b>, for the 4 momentum transfer squared in 
    $MeV^2$, from <b>"$Q^2$ lower bound"</b> to <b>"$Q^2$ upper bound"</b>, for the proton transverse momentum
    fraction interval from <b>"$\alpha_p$ lower bound"</b> to <b>"$\alpha_p$ upper bound"</b> and for a given 
    interval of the kinematic variable that can be:<br/> 
    &emsp;- The transverse prtoton momentum, $p_{pT}$ (in MeV)<br/> 
    &emsp;- The proton 3 momentum norm, k (in MeV)<br/> 
    &emsp;- The invariant momentum transfer, t' ($in MeV^2$).<br/>
    Which kinematic variable is given by the user can be indicated at the <b>"kinematic variable"</b> radio 
    buttons. The begin 
    and end of the interval can be given, respectively, at <b>"kin. var. lower bound"</b> and <b>"kin. var. upper
    bound"</b>.
    <b>"$\Delta$ kin. var."</b> is the spacing between the values. There are different parametrizations of the 
    structure function which can be selected in the drop down menu <b>"Structure Function"</b>. There are also
    2 AV18 Wave function parametrizations, AV18a and AV18b, this can be chosen by the use of the <b>"Wave 
    function"</b> radio buttons.</p>
    
    <p>&emsp;2 different differential cross sections can be plotted on the y-axis, both in
    nb/GeV^4, either it is dependent on the proton 3 momentum or it is dependent on the LF momentum fraction 
    $\alpha_p$ and the invariant momentum transfer t':<br/>
    &emsp; $\frac{d\sigma}{dxdQ^2\frac{d^3p_p}{E_p}}$ &emsp; or &emsp; 
    $\frac{d\sigma}{dxdQ^2d\alpha_pd(-t')}$. 
    The one that has to be displayed on the y-axis can be selected using the <b>"cross section"</b> radio buttons.
    The <b>"x-axis variable"</b> is the kinematic variable that will be displayed on the x-axis of the figure. The
    differential cross section can be displayed on a logaritmic y-axis by checking the <b>"logaritmic y-axis"</b>
    checkbox.</p>
    
    <p>&emsp;If a name is given in the box titled <b>"Figure name"</b>, the figure is stored as this name.png. 
    If no name is given the figure is not saved but there is the option to save the figure after it is displayed. 
    After displaying, and when no name was given, a box titled <b>"Figure name"</b> appears in which a name of the
    figure can be provided as which the figure will be saved after pressing the <b>"Save figure"</b> button.</p>
    
    <p>&emsp;A figure using the bin average values can be made by pressing the <b>"Make bin average figure"</b>
    button. The central values of the given bins will be used to do all the calculations. The values can also
    be calculated using a Monte Carlo integration in these bins, a figure using these values will be displayed 
    after pressing the <b>"Make MC figure"</b> button.</p>
    
    <p>&emsp; For the MC integration the <a href="https://vegas.readthedocs.io/en/latest/"> vegas</a> algorithm
    is used which uses a machine learning algorithm approach to best estimate the value of the multidimensional
    integral. The integral wil be estimated using <b>"MC iterations"</b> iterations of the vegas algorithm where
    for each iteration no more than <b>"MC evaluations"</b> evaluations of the integrand to estimate the integral.
    To produce independent estimates of the integral from which the weighted average is taken. Standard values are
    10 for both values.</p><hr>""")
    
    L_int_val   =widgets.Text(value='1e6',placeholder=r'Integrated luminosity',
                              description=r'$L_{int}$ $(nb^{-1})$',disabled=False,style=style,layout=layout)
    s_en_val    =widgets.Text(value='1e9',placeholder=r'squared e-d C.O.M energy ',
                              description=r'$s_{eN}$ $(MeV^2)$',disabled=False,style=style,layout=layout)
    x_start_val =widgets.Text(value='0.04',placeholder=r'Bjorken variable',
                              description=r'Bjorken $x$ lower bound',disabled=False,style=style,layout=layout)
    x_stop_val  =widgets.Text(value='0.06',placeholder=r'Bjorken variable',
                              description=r'Bjorken $x$ upper bound',disabled=False,style=style,layout=layout)
    Q2_start_val=widgets.Text(value='30e6',placeholder=r'4 momentum transfer squared',
                              description=r'$Q^2$ lower bound $(MeV^2)$',disabled=False,style=style,layout=layout)
    Q2_stop_val =widgets.Text(value='40e6',placeholder=r'4 momentum transfer squared',
                              description=r'$Q^2$ upper bound $(MeV^2)$',disabled=False,style=style,layout=layout)
    alpha_p_start_val=widgets.Text(value='0.98',placeholder=r'proton light cone momentum fraction',
                                   description=r'$\alpha_p$ lower bound',disabled=False,style=style,layout=layout)
    alpha_p_stop_val =widgets.Text(value='1.02',placeholder=r'proton light cone momentum fraction',
                                   description=r'$\alpha_p$ upper bound',disabled=False,style=style,layout=layout)
    x_val_start_val  =widgets.Text(value='-100000',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. lower bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_stop_val   =widgets.Text(value='0',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. upper bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_step_val   =widgets.Text(value='5000',placeholder=r"t', p_{pt}, k",
                                   description=r'$\Delta$ kin. var. $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_param_val =widgets.RadioButtons(options=[("t' (MeV^2)","t'"), ("p_pt (MeV)","p_pt"), ("k (MeV)","k")],
                                       description='kinematic variable:',disabled=False,style=style,layout=layout)
    SF_parametriation_val =widgets.Dropdown(options=[("CB: Christy & Bosted parametrization", "CB"),
    ("SLAC: SLAC paramtetrization from Bodek", "SLAC"),
    ("Alekhin: leading twist parametrization by Alekhin [see PRD 68,014002]", "Alekhin"),
    ("CTEQ: F2 based on the pdf's from CTEQ (code from Misak)", "CTEQ"),
    ("HMRS: F2 from pdfs from unknown source (Shunzo Kumano is the source, not enough info in his code file)", 
     "HMRS"),
    ("MSTW: F2 from LO (or nlo etc) from MSTW pdfs", "MSTW")],value="CTEQ",description='Structure Function:',
                                            disabled=False,style = style,layout = layout)        
    WF_parametrization_val=widgets.RadioButtons(options=[("AV18A","first"), ("AV18B","second")],
                                                description='Wave function:',disabled=False,style = style,
                                                layout = layout)
    sigma_dep_val=widgets.RadioButtons(options=[(r"d\sigma/dxdQ^2d^3p_p/E_p","p"),
                                                (r"d\sigma/dxdQ^2d^\alpha_pd(-t')", "t'") ],
                                        description=r'cross section $(nb/GeV^4)$:',disabled=False,style = style,
                                        layout = layout)
    x_dep_val    =widgets.RadioButtons(options=[(r"t'(GeV^2)","t'"), (r"p_pt (GeV)","p_pt"), ("k (GeV)", "k")],
                                       description='x-axis variable:',disabled=False,style=style,layout=layout)
    logaritmic   =widgets.Checkbox(value=True,description='logaritmic y-axis',disabled=False,style=style,
                                   layout=layout)
    save =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                       disabled=False,style=style,layout=layout)
    nitn =widgets.Text(value='10',placeholder=r"nitn",description=r'MC iterations:',disabled=False,style=style,
                       layout=layout)
    neval=widgets.Text(value='10',placeholder=r"neval",description=r'MC evaluations:',disabled=False,
                       style=style,layout=layout)
    
    make_figure_av=widgets.Button(description='Make bin average figure',disabled=False,button_style='',
                                  tooltip='make figure',icon='check',layout=Layout(width="initial"))
    make_figure_mc=widgets.Button(description='Make MC figure',disabled=False,button_style='',
                                  tooltip='make figure',icon='check',layout=Layout(width="initial"))
    
    display(intro)
    
    display(widgets.HBox((L_int_val,s_en_val)))
    display(widgets.HBox((x_start_val, x_stop_val)))
    display(widgets.HBox((Q2_start_val, Q2_stop_val)))
    display(widgets.HBox((alpha_p_start_val, alpha_p_stop_val)))
    display(widgets.HBox((x_param_val,x_val_start_val)))
    display(widgets.HBox(( x_val_stop_val,x_val_step_val)))
    display(widgets.HBox((SF_parametriation_val, WF_parametrization_val)))
    display(widgets.HBox((sigma_dep_val,x_dep_val)))
    display(widgets.HBox((logaritmic,save)))
    display(widgets.HBox((nitn,neval)))
    
    display(widgets.HBox((make_figure_av,make_figure_mc)))
    
    def cross_sec_fig(b):
        with out:
            clear_output()
            L_int  = float(L_int_val.value)
            s_ed   = 2.*float(s_en_val.value)
            x_int  = np.array([float(x_start_val.value),float(x_stop_val.value)])
            Q2_int = np.array([float(Q2_start_val.value),float(Q2_stop_val.value)])
            alpha_p_int = np.array([float(alpha_p_start_val.value),float(alpha_p_stop_val.value)])
            x_val_int   = np.arange(float(x_val_start_val.value),float(x_val_stop_val.value),
                                    float(x_val_step_val.value))
            if b.description == "Make bin average figure":
                figure = plot_diff_cross_sec(L_int,s_ed,x_int,Q2_int,alpha_p_int,x_val_int,x_param_val.value,
                                             SF_parametriation_val.value,WF_parametrization_val.value,
                                             sigma_dep_val.value,x_dep_val.value,save.value,logaritmic.value)
            else:
                figure = plot_diff_cross_sec_MC(L_int,s_ed,x_int,Q2_int,alpha_p_int,x_val_int,x_param_val.value,
                                                SF_parametriation_val.value,WF_parametrization_val.value,
                                                sigma_dep_val.value,x_dep_val.value,save.value,logaritmic.value,
                                                int(nitn.value),int(neval.value))
            if save.value:
                if not "png" in save.value and not "jpg" in save.value and not "pdf" in save.value:
                    save.value += ".png"
                print("{} saved to {}".format(save.value,os.getcwd()))
                save.value = ""
            else:
                save_after_t=widgets.Text(value='',placeholder=r"Name of figure for saving",
                                          description=r'Figure name:',disabled=False,
                                          style={'description_width': 'initial'},layout=Layout(width="initial"))
                save_after_b=widgets.Button(description='Save figure',disabled=False,button_style='',
                                            tooltip='save',icon='check',style={'description_width': 'initial'},
                                            layout=Layout(width="initial"))
                display(widgets.HBox((save_after_t,save_after_b)))
                def on_clicked(_):
                    if not "png" in save_after_t.value and not "jpg" in save_after_t.value and\
                    not "pdf" in save_after_t.value:
                        save_after_t.value += ".png"
                    saving = os.path.join(os.getcwd(),"output",save_after_t.value)
                    figure.savefig(saving,dpi=600)
                    print("saved as {}".format(saving))
                    save_after_t.value=""
                save_after_b.on_click(on_clicked)

    make_figure_av.on_click(cross_sec_fig)
    make_figure_mc.on_click(cross_sec_fig)
#out = Output()
#make_figure_cross_section(out)
#out

In [19]:
def D_d_3(alpha_p,p_pt,WF_parametrization = "first"):
    momentum_dist_vector = delta_fd(alpha_p,p_pt, WF_parametrization)
    momentum_dist_unpol  = fd_unpol(alpha_p,p_pt, WF_parametrization)
    
    return momentum_dist_vector/momentum_dist_unpol

def D_d_2(alpha_p,p_pt,WF_parametrization = "first"):
    momentum_dist_vector = delta_fd(alpha_p,p_pt, WF_parametrization)
    momentum_dist_unpol  = fd_unpol(alpha_p,p_pt, WF_parametrization)
    momentum_dist_tensor = fd_tensor(alpha_p,p_pt, WF_parametrization)
    
    return momentum_dist_vector/(momentum_dist_unpol + momentum_dist_tensor)

def plot_Dd(alpha_p,x_vals,x_parameter,WF_parametrization = "first",x_dependency=None,save=""):
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_vals)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_vals**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def t_apo():
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*x_vals**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*x_vals**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def k():
        if x_parameter == "t'":
            return np.sqrt(MD**2-4*MN**2-2*x_vals/(2-alpha_p))/4
        elif x_parameter == "p_pt":
            return np.sqrt((x_vals**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2)
        elif x_parameter == "k":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    p_pt = p_trans()
    values = np.array([[D_d_3(alpha_p,p,WF_parametrization),D_d_2(alpha_p,p,WF_parametrization)] for p in p_pt])
    
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        x_dep_val = -t_apo()*10**-6
        x_label = r"-t' [$GeV^2$]"
    elif x_dependency == "p_pt":
        x_dep_val = p_trans()*10**-3
        x_label = r"$p_{pt}$ [GeV]"
    elif x_dependency == "k":
        x_dep_val = k()*10**-3
        x_label = r"k [GeV]"
    else:
        raise ValueError("no x dependency called {}, choose from t_apo, p_pt, k".format)
    
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(x_dep_val,values[:,0],label = r"$D_{d}^{(3)}$")
    ax.plot(x_dep_val,values[:,1],label = r"$D_{d}^{(2)}$")
    ax.plot([],[]," ",label=r"$\alpha_p$ = {}".format(alpha_p))
    ax.hlines(1.,abs(np.nanmin(x_dep_val)),abs(np.nanmax(x_dep_val)),linestyles="dashed")
    ax.hlines(-1.,abs(np.nanmin(x_dep_val)),abs(np.nanmax(x_dep_val)),linestyles="dashed")
    ax.hlines(0.,abs(np.nanmin(x_dep_val)),abs(np.nanmax(x_dep_val)),linestyles="dashed")
    ax.set_xlabel(x_label)
    ax.set_ylabel(r"$D_{d}^{(3)},D_{d}^{(2)}$[vector]")
    ax.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    plt.tight_layout()
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    
    plt.show()
    
    return fig

def plot_Dd_zoom(alpha_p_values,x_vals,x_parameter,WF_parametrization = "first",x_dependency=None,save=""):
    def p_trans(alpha_p):
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_vals)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_vals**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def t_apo(alpha_p):
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*x_vals**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*x_vals**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def k(alpha_p):
        if x_parameter == "t'":
            return np.sqrt(MD**2-4*MN**2-2*x_vals/(2-alpha_p))/4
        elif x_parameter == "p_pt":
            return np.sqrt((x_vals**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2)
        elif x_parameter == "k":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    p_pt = np.array([p_trans(alpha) for alpha in alpha_p_values])
    p_pt = p_pt[np.where(p_pt==np.nanmax(p_pt))[0][0]]
    values = np.array([[[D_d_3(alpha,p,WF_parametrization),D_d_2(alpha,p,WF_parametrization)] for p in p_pt]\
                        for alpha in alpha_p_values])
    
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        x_dep_val = np.array([-t_apo(alpha)*10**-6 for alpha in alpha_p_values])
        x_dep_val = x_dep_val[np.where(x_dep_val==np.nanmax(x_dep_val))[0][0]]
        x_label = r"-t' [$GeV^2$]"
    elif x_dependency == "p_pt":
        x_dep_val = np.array([p_trans(alpha)*10**-3 for alpha in alpha_p_values])
        x_dep_val = x_dep_val[np.where(x_dep_val==np.nanmax(x_dep_val))[0][0]]
        x_label = r"$p_{pt}$ [GeV]"
    elif x_dependency == "k":
        x_dep_val = np.array([k(alpha)*10**-3 for alpha in alpha_p_values])
        x_dep_val = x_dep_val[np.where(x_dep_val==np.nanmax(x_dep_val))[0][0]]
        x_label = r"k [GeV]"
    else:
        raise ValueError("no x dependency called {}, choose from t_apo, p_pt, k".format)
    
    fig, (ax1, ax2) = plt.subplots(2, 1,figsize=(8,4))
    for i,value in enumerate(values):
        ax1.plot(x_dep_val,value[:,0],label=r"$\alpha_p$ = {}".format(alpha_p_values[i]))
        ax2.plot(x_dep_val,value[:,1],label=r"$\alpha_p$ = {}".format(alpha_p_values[i]))
    
    ax1.set_ylabel(r"$D_{d}^{(3)}$[vector]")
    ax1.set_xlabel(x_label)
    ax1.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    plt.tight_layout()
    ax1.hlines(1.,abs(np.nanmin(x_dep_val)),abs(np.nanmax(x_dep_val)),linestyles="dashed")
    
    ax2.set_ylabel(r"$D_{d}^{(2)}$[vector]")
    ax2.set_xlabel(x_label)
    ax2.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    plt.tight_layout()
    ax2.hlines(1.,abs(np.nanmin(x_dep_val)),abs(np.nanmax(x_dep_val)),linestyles="dashed")
    
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    plt.show()
    
    return fig
    
#b = plot_Dd_zoom(np.array([1.,1.04,1.08]),np.linspace(0,100,100),"p_pt",save="D_d_zoom.png")

In [52]:
def D_d_3_ten(alpha_p,p_pt,WF_parametrization = "first"):
    momentum_dist_tensor = fd_tensor(alpha_p,p_pt, WF_parametrization)
    momentum_dist_unpol  = fd_unpol(alpha_p,p_pt, WF_parametrization)
    
    return momentum_dist_tensor/momentum_dist_unpol

def D_d_2_ten(alpha_p,p_pt,WF_parametrization = "first"):
    momentum_dist_unpol  = fd_unpol(alpha_p,p_pt, WF_parametrization)
    momentum_dist_tensor = fd_tensor(alpha_p,p_pt, WF_parametrization)
    
    return momentum_dist_tensor/(momentum_dist_unpol + momentum_dist_tensor)

def plot_Dd_ten(alpha_p,x_vals,x_parameter,WF_parametrization = "first",x_dependency=None,save=""):
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_vals)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_vals**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def t_apo():
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*x_vals**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*x_vals**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def k():
        if x_parameter == "t'":
            return np.sqrt(MD**2-4*MN**2-2*x_vals/(2-alpha_p))/4
        elif x_parameter == "p_pt":
            return np.sqrt((x_vals**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2)
        elif x_parameter == "k":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    p_pt = p_trans()
    values = np.array([[D_d_3_ten(alpha_p,p,WF_parametrization),D_d_2_ten(alpha_p,p,WF_parametrization)]\
                       for p in p_pt])
    
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        x_dep_val = -t_apo()*10**-6
        x_label = r"-t' [$GeV^2$]"
    elif x_dependency == "p_pt":
        x_dep_val = p_trans()*10**-3
        x_label = r"$p_{pt}$ [GeV]"
    elif x_dependency == "k":
        x_dep_val = k()*10**-3
        x_label = r"k [GeV]"
    else:
        raise ValueError("no x dependency called {}, choose from t_apo, p_pt, k".format)
    
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(x_dep_val,values[:,0],label = r"$D_{d}^{(3)}$")
    ax.plot(x_dep_val,values[:,1],label = r"$D_{d}^{(2)}$")
    ax.plot([],[]," ",label=r"$\alpha_p$ = {}".format(alpha_p))
    #ax.hlines(1.,abs(np.nanmin(x_dep_val)),abs(np.nanmax(x_dep_val)),linestyles="dashed")
    #ax.hlines(-1.,abs(np.nanmin(x_dep_val)),abs(np.nanmax(x_dep_val)),linestyles="dashed")
    ax.hlines(0.,abs(np.nanmin(x_dep_val)),abs(np.nanmax(x_dep_val)),linestyles="dashed")
    ax.set_xlabel(x_label)
    ax.set_ylabel(r"$D_{d}^{(3)},D_{d}^{(2)}$[tensor]")
    ax.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    plt.tight_layout()
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    
    plt.show()
    
    return fig

def plot_Dd_ten_zoom(alpha_p_values,x_vals,x_parameter,WF_parametrization = "first",x_dependency=None,save=""):
    def p_trans(alpha_p):
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_vals)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_vals**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def t_apo(alpha_p):
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*x_vals**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*x_vals**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def k(alpha_p):
        if x_parameter == "t'":
            return np.sqrt(MD**2-4*MN**2-2*x_vals/(2-alpha_p))/4
        elif x_parameter == "p_pt":
            return np.sqrt((x_vals**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2)
        elif x_parameter == "k":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    p_pt = np.array([p_trans(alpha) for alpha in alpha_p_values])
    p_pt = p_pt[np.where(p_pt==np.nanmax(p_pt))[0][0]]
    values = np.array([[[D_d_3_ten(alpha,p,WF_parametrization),D_d_2_ten(alpha,p,WF_parametrization)]\
                        for p in p_pt] for alpha in alpha_p_values])
    
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        x_dep_val = np.array([-t_apo(alpha)*10**-6 for alpha in alpha_p_values])
        x_dep_val = x_dep_val[np.where(x_dep_val==np.nanmax(x_dep_val))[0][0]]
        x_label = r"-t' [$GeV^2$]"
    elif x_dependency == "p_pt":
        x_dep_val = np.array([p_trans(alpha)*10**-3 for alpha in alpha_p_values])
        x_dep_val = x_dep_val[np.where(x_dep_val==np.nanmax(x_dep_val))[0][0]]
        x_label = r"$p_{pt}$ [GeV]"
    elif x_dependency == "k":
        x_dep_val = np.array([k(alpha)*10**-3 for alpha in alpha_p_values])
        x_dep_val = x_dep_val[np.where(x_dep_val==np.nanmax(x_dep_val))[0][0]]
        x_label = r"k [GeV]"
    else:
        raise ValueError("no x dependency called {}, choose from t_apo, p_pt, k".format)
    
    fig, (ax1, ax2) = plt.subplots(2, 1,figsize=(8,4))
    for i,value in enumerate(values):
        ax1.plot(x_dep_val,value[:,0],label=r"$\alpha_p$ = {}".format(alpha_p_values[i]))
        ax2.plot(x_dep_val,value[:,1],label=r"$\alpha_p$ = {}".format(alpha_p_values[i]))
    
    ax1.set_ylabel(r"$D_{d}^{(3)}$[tensor]")
    ax1.set_xlabel(x_label)
    ax1.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    ax1.hlines(0,abs(np.nanmin(x_dep_val)),abs(np.nanmax(x_dep_val)),linestyles="dashed")
    
    ax2.set_ylabel(r"$D_{d}^{(2)}$[tensor]")
    ax2.set_xlabel(x_label)
    ax2.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    plt.tight_layout()
    ax2.hlines(0.,abs(np.nanmin(x_dep_val)),abs(np.nanmax(x_dep_val)),linestyles="dashed")
    
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    plt.show()
    
    return fig


In [87]:
def make_figure_D_d(out):
    intro = widgets.HTMLMath(r"""<center><b><h1>Deuteron depolarization factors $D_d^{(3)}$ and $D_d^{(2)}$</h1>
    </b></center>
    <hr>
    <table style="float: right;"><tr><td>
        <img src="figures/figure_D_d_vector.png" width = "350" ></td></tr><tr><td>
        <img src="figures/figure_D_d_tensor.png" width = "350" ></td></tr>
        <caption style="caption-side: bottom;color:black;text-align:center">fig: 2 example figures of the 
        <b>Deuteron depolarization factors $D_d^{(3)}$ and $D_d^{(2)}$</b> program</caption>
    </table>
    
    <p>&emsp;This cell can create a figure of both $D_{d}^{3}$ and $D_{d}^{2}$ for both the vector and tensor 
    case, for a value of
    the proton light cone momentum fraction <b>"$\alpha_p$"</b> and for a given interval of a kinematic variable 
    that can be:<br/> 
    &emsp;- The transverse prtoton momentum, $p_{pT}$ (in MeV)<br/> 
    &emsp;- The proton 3 momentum norm, k (in MeV)<br/> 
    &emsp;- The invariant momentum transfer, t' ($in MeV^2$).<br/>
    Which kinematic variable is given by the user can be indicated at the <b>"kinematic variable"</b> radio 
    buttons. The begin 
    and end of the interval can be given, respectively, at <b>"kin. var. lower limit"</b> and <b>"kin. var 
    upper limit"</b>.
    <b>"$\Delta$ kin. var."</b> is the spacing between the values.</p>
    
    <p>&emsp;The <b>"x-axis variable"</b> is the kinematic variable that will be displayed on the x-axis of the 
    figure. 
    There are 2 AV18 Wave function parametrizations, AV18a and AV18b, this can be chosen by use of the <b>"Wave 
    function"</b> radio buttons. If a name is given in the box titled <b>"Figure name"</b>, the figure is stored 
    as this name.png. If no name is given the figure is not saved but there is the option to save the figure after
    it is displayed. After displaying, and when no name was given, a box titled <b>"Figure name"</b> appears in 
    which a name of the figure can be provided as which the figure will be saved after pressing the <b>"Save 
    figure"</b> button.</p>
    
    <p>&emsp;The figure will be made after pressing either the <b>"Make D_d[vector] figure"</b> button or the 
    <b>"Make D_d[tensor] figure"</b> button.</p><hr>""")
    
    alpha_p_val      =widgets.Text(value='1.',placeholder=r'proton light cone momentum fraction',
                                   description=r"$\alpha_p$",disabled=False,style=style,layout=layout)
    x_param_val      =widgets.RadioButtons(options=[("t' MeV^2","t'"), (r"p_pt (MeV)","p_pt"), ("k (MeV)","k")],
                                           description='kinematic variable:',disabled=False,style=style,
                                           layout=layout,value = "p_pt")
    x_val_start_val  =widgets.Text(value='0',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. lower limit $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_stop_val   =widgets.Text(value='600',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. upper limit $(MeV/MeV^2)$',disabled=False,
                                   style=style,layout=layout)
    x_val_step_val   =widgets.Text(value='10',placeholder=r"t', p_{pt}, k",
                                   description=r'$\Delta $kin. var.$(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    WF_parametrization_val=widgets.RadioButtons(options=[("AV18a","first"), ("AV18b","second")],
                                                description='Wave function:',disabled=False,style = style,
                                                layout = layout)
    x_dep_val    =widgets.RadioButtons(options=[(r"t'(GeV^2)","t'"), (r"p_pt (GeV)","p_pt"), ("k (GeV)", "k")],
                                       description='x-axis variable:',disabled=False,style=style,layout=layout,
                                       value = "p_pt")
    save =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                       disabled=False,style=style,layout=layout)
    
    make_figure_Dd_vec =widgets.Button(description="Make D_d[vector] figure",disabled=False,button_style='',
                                       tooltip='make figure',icon='check',layout=Layout(width="initial"))
    make_figure_Dd_ten =widgets.Button(description="Make D_d[tensor] figure",disabled=False,button_style='',
                                       tooltip='make figure',icon='check',layout=Layout(width="initial"))

    display(intro)
    
    display(alpha_p_val)
    display(widgets.HBox((x_param_val,x_val_start_val)))
    display(widgets.HBox((x_val_stop_val,x_val_step_val)))
    display(widgets.HBox((WF_parametrization_val,x_dep_val)))
    display(save)
    
    display(widgets.HBox((make_figure_Dd_vec,make_figure_Dd_ten)))
    
    def Dd_fig(b):
        with out:
            clear_output()
            alpha_p_int = float(alpha_p_val.value)
            x_val_int   = np.arange(float(x_val_start_val.value),float(x_val_stop_val.value)+\
                                    float(x_val_step_val.value),float(x_val_step_val.value))
            if b.description == "Make D_d[vector] figure":
                figure = plot_Dd(alpha_p_int,x_val_int,x_param_val.value,WF_parametrization_val.value,
                                 x_dep_val.value,save.value)
            else:
                figure = plot_Dd_ten(alpha_p_int,x_val_int,x_param_val.value,WF_parametrization_val.value,
                                     x_dep_val.value,save.value)
            if save.value:
                save.value = ""
            else:
                save_after_t=widgets.Text(value="",placeholder=r"Name of figure for saving",
                                          description=r'Figure name:',disabled=False,
                                          style={'description_width': 'initial'},layout=Layout(width="initial"))
                save_after_b=widgets.Button(description='Save figure',disabled=False,button_style='',
                                            tooltip='save',icon='check',style={'description_width':'initial'},
                                            layout=Layout(width="initial"))
                display(widgets.HBox((save_after_t,save_after_b)))
                def on_clicked(_):
                    if not "png" in save_after_t.value and not "jpg" in save_after_t.value and\
                    not "pdf" in save_after_t.value:
                        save_after_t.value += ".png"
                    saving = os.path.join(os.getcwd(),"output",save_after_t.value)
                    figure.savefig(saving,dpi=600)
                    print("saved as {}".format(saving))
                    save_after_t.value=""
                save_after_b.on_click(on_clicked)

    make_figure_Dd_vec.on_click(Dd_fig)
    make_figure_Dd_ten.on_click(Dd_fig)
   
    
#out = Output()
#make_figure_D_d(out)
#out

In [134]:
def make_figure_D_d_zoom(out):
    intro = widgets.HTMLMath(r"""<center><b><h1>Deuteron depolarization factors $D_d^{(3)}$ and $D_d^{(2)}$<br/>
    for different $\alpha_p$ values</h1></b></center>
    <hr>
    <table style="float: right;"><tr><td>
        <img src="figures/figure_D_d_zoom_vector.png" width = "350" ></td></tr><tr><td>
        <img src="figures/figure_D_d_zoom_tensor.png" width = "350" ></td></tr>
        <caption style="caption-side: bottom;color:black;text-align:center">fig: 2 example figures of the 
        <b>Deuteron depolarization factors$D_d^{(3)}$ and $D_d^{(2)}$ for different $\alpha_p$ values</b> 
        program</caption>
    </table>
    
    <p>&emsp;This cell can create a subplot with a figures of $D_{d}^{3}$ and one with a figure of
    $D_{d}^{2}$ for both the vector and tensor case, for a range of proton light cone momentum fraction values 
    <b>"$\alpha_p$"</b>, seperated by a comma and for a given interval of a kinematic variable that can be:<br/> 
    &emsp;- The transverse prtoton momentum, $p_{pT}$ (in MeV)<br/> 
    &emsp;- The proton 3 momentum norm, k (in MeV)<br/> 
    &emsp;- The invariant momentum transfer, t' ($in MeV^2$).<br/>
    Which kinematic variable is given by the user can be indicated at the <b>"kinematic variable"</b> radio 
    buttons. The begin 
    and end of the interval can be given, respectively, at <b>"kin. var. lower bound"</b> and <b>"kin. var 
    upper bound"</b>.
    <b>"$\Delta$ kin. var."</b> is the spacing between the values.</p>
    
    <p>&emsp;The <b>"x-axis variable"</b> is the kinematic variable that will be displayed on the x-axis of the 
    figure. 
    There are 2 AV18 Wave function parametrizations, AV18a and AV18b, this can be chosen by use of the <b>"Wave 
    function"</b> radio buttons. If a name is given in the box titled <b>"Figure name"</b>, the figure is stored as 
    this name.png. If no name is given the figure is not saved but there is the option to save the figure after it 
    is displayed. After displaying, and when no name was given, a box titled <b>"Figure name"</b> appears in which 
    a name of the figure can be provided as which the figure will be saved after pressing the <b>"Save figure"</b> 
    button.</p>
    
    <p>&emsp;The figure will be made after pressing either the <b>"Make D_d[vector] figure"</b> button or the
     <b>"Make D_d[tensor] figure"</b> button.</p><hr>""")
    
    alpha_p_vals     =widgets.Text(value='1, 1.04, 1.08',placeholder=r'proton light cone momentum fraction',
                                   description=r'$\alpha_p$',disabled=False,style=style,layout=layout)
    x_param_val      =widgets.RadioButtons(options=[("t' (MeV^2)","t'"), ("p_pt (MeV)","p_pt"), ("k (MeV)","k")],
                                           description='kinematic variable:',disabled=False,style=style,
                                           layout=layout,value = "p_pt")
    x_val_start_val  =widgets.Text(value='0',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. lower bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_stop_val   =widgets.Text(value='100',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. upper bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_step_val   =widgets.Text(value='1',placeholder=r"t', p_{pt}, k",
                                   description=r'$\Delta$ kin. var. $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    WF_parametrization_val=widgets.RadioButtons(options=[("AV18a","first"), ("AV18b","second")],
                                                description='Wave function:',disabled=False,style = style,
                                                layout = layout)
    x_dep_val    =widgets.RadioButtons(options=[(r"t'(GeV^2)","t'"), (r"p_pt (GeV)","p_pt"), ("k (GeV)", "k")],
                                       description='x-axis variable:',disabled=False,style=style,layout=layout,
                                       value = "p_pt")
    save =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                       disabled=False,style=style,layout=layout)
    
    make_figure_Dd_vec_zoom =widgets.Button(description='Make D_d[vector] figure',disabled=False,button_style='',
                                           tooltip='make figure',icon='check',layout=Layout(width="initial"))
    make_figure_Dd_ten_zoom =widgets.Button(description='Make D_d[tensor] figure',disabled=False,button_style='',
                                           tooltip='make figure',icon='check',layout=Layout(width="initial"))

    display(intro)
    
    display(alpha_p_vals)
    display(widgets.HBox((x_param_val,x_val_start_val)))
    display(widgets.HBox((x_val_stop_val,x_val_step_val)))
    display(widgets.HBox((WF_parametrization_val,x_dep_val)))
    display(save)
    
    display(widgets.HBox((make_figure_Dd_vec_zoom,make_figure_Dd_ten_zoom)))
    
    def Dd_zoom_fig(b):
        with out:
            clear_output()
            alpha_p_int = np.array([float(alpha) for alpha in alpha_p_vals.value.split(",")])
            x_val_int   = np.arange(float(x_val_start_val.value),float(x_val_stop_val.value)+\
                                    float(x_val_step_val.value),float(x_val_step_val.value))
            if b.description == "Make D_d[vector] figure":
                figure = plot_Dd_zoom(alpha_p_int,x_val_int,x_param_val.value,WF_parametrization_val.value,
                                      x_dep_val.value,save.value)
            else:
                figure = plot_Dd_ten_zoom(alpha_p_int,x_val_int,x_param_val.value,WF_parametrization_val.value,
                                          x_dep_val.value,save.value)
            if save.value:
                save.value = ""
            else:
                save_after_t=widgets.Text(value='',placeholder=r"Name of figure for saving",
                                          description=r'Figure name:',disabled=False,
                                          style={'description_width':'initial'},layout=Layout(width="initial"))
                save_after_b = widgets.Button(description='Save figure',disabled=False,button_style='',
                                              tooltip='save',icon='check',style={'description_width': 'initial'},
                                              layout=Layout(width="initial"))
                display(widgets.HBox((save_after_t,save_after_b)))
                def on_clicked(_):
                    if not "png" in save_after_t.value and not "jpg" in save_after_t.value and\
                    not "pdf" in save_after_t.value:
                        save_after_t.value += ".png"
                    saving = os.path.join(os.getcwd(),"output",save_after_t.value)
                    figure.savefig(saving,dpi=600)
                    print("saved as {}".format(saving))
                    save_after_t.value=""
                save_after_b.on_click(on_clicked)

    make_figure_Dd_vec_zoom.on_click(Dd_zoom_fig)
    make_figure_Dd_ten_zoom.on_click(Dd_zoom_fig)
#out = Output()
#make_figure_D_d_zoom(out)
#out

In [128]:
def full_cross_sec(s_ed,x,Q2,alpha_p,x_val,x_parameter,sigma_e,lambda_d,SF_parametriation = "CTEQ",\
                   WF_parametrization = "first"):
    
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_val)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_val**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    p_pt = p_trans()
    
    x_d     = x/2.
    y       = Q2/(x_d*(s_ed-MD**2))
    gamma2  = (x*MD)**2/Q2 #2x_d = x
    epsilon = (1-y-gamma2*y**2/4)/(1-y+y**2/2+gamma2*y**2/4)
    
    struc_f = struc_func(SF_parametriation, WF_parametrization)
    if lambda_d == 1:
        F_U = struc_f.FTd(x,Q2,alpha_p,p_pt)+struc_f.FTLLT(x,Q2,alpha_p,p_pt)+\
              epsilon*(struc_f.FLd(x,Q2,alpha_p,p_pt)+struc_f.FTLLL(x,Q2,alpha_p,p_pt))
        S_L = (1-gamma2*y/2)/(np.sqrt(1+gamma2)) 
        F_L = 2.*sigma_e*np.sqrt(1-epsilon**2)*S_L*struc_f.FSLd(x,Q2,alpha_p,p_pt)
    elif lambda_d == -1:
        F_U = struc_f.FTd(x,Q2,alpha_p,p_pt)+struc_f.FTLLT(x,Q2,alpha_p,p_pt)+\
              epsilon*(struc_f.FLd(x,Q2,alpha_p,p_pt)+struc_f.FTLLL(x,Q2,alpha_p,p_pt))
        S_L = -(1-gamma2*y/2)/(np.sqrt(1+gamma2)) 
        F_L = 2.*sigma_e*np.sqrt(1-epsilon**2)*S_L*struc_f.FSLd(x,Q2,alpha_p,p_pt)
    elif lambda_d == 0:
        F_U = struc_f.FTd(x,Q2,alpha_p,p_pt)-2.*struc_f.FTLLT(x,Q2,alpha_p,p_pt)+\
              epsilon*(struc_f.FLd(x,Q2,alpha_p,p_pt)-2.*struc_f.FTLLL(x,Q2,alpha_p,p_pt))
        F_L = 0.
    else:
        raise ValueError("deuteron spin has to be -1,1 or 0 not {}".format(lambda_d))
    
    #front factor______________________________________
    front_factor = alpha_em**2*y**2/(Q2**2*(1-epsilon))
    
    #norm factor relies on the used dependency_________
    norm_factor  = 1/(32*np.pi) #dependent on -t'
    
    #returning in nb/MeV^4_____________________________
    d_sigma = fgHbarc**2*10**7*norm_factor*front_factor*(F_U+F_L)
    
    return d_sigma

#spins = [(1./2.,1,1.),(-1./2.,-1,1.),(-1./2.,1,-1.),(1./2.,-1,-1.)]
#for spin in spins:
#    sigma_e,lambda_d,_ = spin
#    print(full_cross_sec(2e9,0.05,25e6,1,-1e4,"t'",sigma_e,lambda_d))

In [138]:
def A_d_3(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation = "CTEQ",WF_parametrization = "first"):
    """
    The parallel spin asymmetry
    
    x        (float)           : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                 the absence of nuclear binding.
    Q2       (float)(MeV^2)    : the negative of the 4 momentum transfer squared or virtual photon momentum squared
    alpha_p  (float)           : the proton light cone momentum fraction
    p_pt    (float)(MeV)      : the transverse momentum value, p_pt.
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 If not specified the "first" one is used.
    
    returns
    """
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_val)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_val**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
            
    p_pt = p_trans()
    
    x_d     = x/2.
    y       = Q2/(x_d*(s_ed-MD**2))
    gamma2  = (x*MD)**2/Q2 #2x_d = x
    epsilon = (1-y-gamma2*y**2/4)/(1-y+y**2/2+gamma2*y**2/4)
    
    #structure function________________________________
    struc_f = struc_func(SF_parametriation, WF_parametrization)
    F = struc_f.FTd(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FLd(x,Q2,alpha_p,p_pt)
    
    #spin asymmetry____________________________________
    D_L = y*(1.-y/2.)*(1.+gamma2*y/2.)/(1.-y+y**2/2.+gamma2*y**2/4.)
    asymmetry = D_L*struc_f.FSLd(x,Q2,alpha_p,p_pt)/F
    
    return asymmetry

def A_d_3_err(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,SF_parametriation = "CTEQ",\
              WF_parametrization = "first"):
    
    def t_apo(val):
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*val**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*val**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    #bin averages______________________________________
    x       = np.mean(x_range)
    Q2      = np.mean(Q2_range)
    alpha_p = np.mean(alpha_p_range)
    x_val   = np.mean(x_val_range)
    
    #bin sizes_________________________________________
    delta_x       = x_range[1]-x_range[0]
    delta_Q2      = Q2_range[1]-Q2_range[0]
    delta_alpha_p = alpha_p_range[1]-alpha_p_range[0]
    delta_x_val   = x_val_range[1]-x_val_range[0]
    delta_t_p     = abs(t_apo(x_val_range[1]-x_val_range[0]))
    
    #calculating the error
    delta = delta_x*delta_Q2*delta_alpha_p*delta_t_p
    #error of the numerator
    #spins = [(1./2.,1,1.),(-1./2.,1,-1.),(1./2.,-1,-1.),(-1./2.,-1,1.)] #(sigma_e,lambda_d,add/subtract)
    spins = [(1./2.,1,2.),(-1./2.,1,-2.)] #spins = [(1./2.,1,1.),(-1./2.,1,-1.)] #(sigma_e,lambda_d,add/subtract)
    num = 0.
    sq_err_sum = 0.
    for spin in spins:
        sigma_e,lambda_d,opperation=spin
        value = full_cross_sec(s_ed,x,Q2,alpha_p,x_val,x_parameter,sigma_e,lambda_d,\
                               SF_parametriation,WF_parametrization)
        
        sq_err_sum += opperation**2*value 
        num        += opperation*value
    sq_rel_num_err = sq_err_sum/(L_int*delta*num**2)
    
    #error of the denominator
    denom = float(diff_cross_sec(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation,\
                                 WF_parametrization,sigma_dependency="t'")[1,1])
    
    sq_rel_den_err = 1./(L_int*delta*denom)
    rel_error = np.sqrt(sq_rel_den_err + sq_rel_num_err)
    
    asymmetry = A_d_3(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation,WF_parametrization)
    error = rel_error*asymmetry
    
    return [asymmetry,error]

def A_d_3_err_MC(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,SF_parametriation = "CTEQ",\
                 WF_parametrization = "first",nitn=1,neval=1):
    """
    The parallel spin asymmetry with mc integration for error
    
    x        (float)           : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                 the absence of nuclear binding.
    Q2       (float)(MeV^2)    : the negative of the 4 momentum transfer squared or virtual photon momentum squared
    alpha_p  (float)           : the proton light cone momentum fraction
    p_pt    (float)(MeV)      : the transverse momentum value, p_pt.
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 If not specified the "first" one is used.
    
    returns
    """
    #bin averages______________________________________
    x       = np.mean(x_range)
    Q2      = np.mean(Q2_range)
    alpha_p = np.mean(alpha_p_range)
    x_val   = np.mean(x_val_range)    
    
    
    asymmetry = A_d_3(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation,WF_parametrization)
    
    #error of the numerator
    #spins = [(1./2.,1,1.),(-1./2.,1,-1.),(1./2.,-1,-1.),(-1./2.,-1,1.)] #(sigma_e,lambda_d,add/subtract)
    spins = [(1./2.,1,2.),(-1./2.,1,-2.)] #spins = [(1./2.,1,1.),(-1./2.,1,-1.)] #(sigma_e,lambda_d,add/subtract)
    num = 0.
    sq_err_sum = 0.
    for spin in spins:
        sigma_e,lambda_d,opperation=spin
        
        def integrandum_num(values):
            x_p,Q2_p,alpha_p_p,x_val_p = values
            return full_cross_sec(s_ed,x_p,Q2_p,alpha_p_p,x_val_p,x_parameter,sigma_e,lambda_d,\
                                  SF_parametriation,WF_parametrization)
        
        integ  = vegas.Integrator([x_range,Q2_range,alpha_p_range,x_val_range])
        result = integ(integrandum_num,nitn=nitn,neval=neval)
        Delta_N  = L_int*result.mean
        #value = full_cross_sec(s_ed,x,Q2,alpha_p,x_val,x_parameter,sigma_e,lambda_d,\
        #                       SF_parametriation,WF_parametrization)
        value = integrandum_num((x,Q2,alpha_p,x_val))
        sq_err_sum += (opperation*value)**2/Delta_N
        
        num += opperation*value#full_cross_sec(s_ed,x,Q2,alpha_p,x_val,x_parameter,sigma_e,lambda_d,\
                                        #SF_parametriation,WF_parametrization)
        
    sq_rel_num_err = sq_err_sum/num**2
    
    #error of the denominator
    def integrandum_den(values):
        x_p,Q2_p,alpha_p_p,x_val_p = values
        return float(diff_cross_sec(s_ed,x_p,Q2_p,alpha_p_p,x_val_p,x_parameter,SF_parametriation,\
                                    WF_parametrization,sigma_dependency="t'")[1,1])
    
    
    integ  = vegas.Integrator([x_range,Q2_range,alpha_p_range,x_val_range])
    result = integ(integrandum_den,nitn=nitn,neval=neval)
    Delta_N  = L_int*result.mean
    sq_rel_den_err = 1./Delta_N
    
    sq_rel_err = sq_rel_num_err + sq_rel_den_err
    error = asymmetry*np.sqrt(sq_rel_err)    
    
    return [asymmetry,error]

def plot_Ad3(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,SF_parametriation = "CTEQ",\
             WF_parametrization = "first",x_dependency=None,save="",MC=False,nitn=5,neval=5):
    """
    plots the parallel spin asymetry A_{\parallel d}^{(3)} for the bin average values in function of the values in 
    x_val_interval.
    
    L_int   (float)(nb^-1)       : the integrated luminosity
    s_ed    (float)(MeV^2)       : the invariant electron deuteron squared centre of mass energy
    x_range (array[float])       : begin and end value of the bin of the effective Bjorken variable for 
                                   scattering from a nucleon in the deuteronin the absence of nuclear binding.
    Q2_range(array[float])(MeV^2): array with begin and end values of the bins of the negative of the 4 momentum 
                                   transfer squared or virtual photon momentum squared
    alpha_p_range (array[float]) : begin and end value of the bin of the proton light cone momentum fraction
    x_val_range   (array[float])(MeV^2/MeV): an array of evenly spaced values that creates bins of x_vals which 
                                             will be used as the x-axis, these can either be an invariant momentum 
                                             transfer between the deuteron and the proton minus the proton mass, 
                                             t', a transverse momentum value, p_pt, or a norm of the
                                             proton/neuteron 3 momentum, k.
    x_parameter        (string): which parameter is used in x_val:
                                 - "t''": the invariant momentum transfer, t'
                                 - "p_pt" : the transverse momentum, p_{pt}
                                 - "k"    : the 3 momentum norm, k
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 if not specified the "first" one is used.
    x_dependency       (string): the dependency on the x-axis of the plot. the values in x_val_interval do not 
                                 have to be the same parameters as the ones that need to be plotted:
                                 - "t'": the invariant momentum transfer, t'
                                 - "p_pt" : the transverse momentum, p_{pt}
                                 - "k"    : the 3 momentum norm, k
                                 if not specified x_parameter is used.
    save (string): if the figure need to be saved give the name of the figure
    
    returns the figure
    e.g.
    
    """
    #bin averages______________________________________
    alpha_p = np.mean(alpha_p_range)
    x_vals  = np.mean(list(zip(x_val_range[:-1],x_val_range[1:])),axis=1)
    
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_vals)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_vals**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def t_apo():
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*x_vals**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*x_vals**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def k():
        if x_parameter == "t'":
            return np.sqrt(MD**2-4*MN**2-2*x_vals/(2-alpha_p))/4
        elif x_parameter == "p_pt":
            return np.sqrt((x_vals**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2)
        elif x_parameter == "k":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    def prog_bar(i,total):
        clear_output(wait=True)
        bar = widgets.FloatProgress(value=i,min=0,max=total,description='Loading: {:.2f}%'.format(i/total*100),\
                                    bar_style='info',orientation='horizontal',\
                                    style = {'description_width': "initial"})
        display(bar)
    
    
    values = []
    total_calculations = (len(x_val_range)-1)*(len(Q2_range))
    prog_bar(0,total_calculations)
    for j in range(len(Q2_range)):
        temp_val = []
        for i in range(len(x_val_range)-1):
            if not MC:
                temp_val  += [A_d_3_err(L_int,s_ed,x_range,Q2_range[j],alpha_p_range,x_val_range[i:i+2],
                                           x_parameter,SF_parametriation,WF_parametrization)]
            else:
                temp_val  += [A_d_3_err_MC(L_int,s_ed,x_range,Q2_range[j],alpha_p_range,x_val_range[i:i+2],
                                           x_parameter,SF_parametriation,WF_parametrization,nitn,neval)]
            prog_bar(j*(len(x_val_range)-1)+i+1,total_calculations)
        values += [temp_val]
    values = np.array(values)
    
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        x_dep_val = -t_apo()*10**-6
        x_label = r"-t' [$GeV^2$]"
    elif x_dependency == "p_pt":
        x_dep_val = p_trans()*10**-3 
        x_label = r"$p_{pt}$ [GeV]"
    elif x_dependency == "k":
        x_dep_val = k()*10**-3
        x_label = r"k [GeV]"
    else:
        raise ValueError("no x dependency called {}, choose from t_apo, p_pt, k".format)
    
    markers     = ["+","x","*","s",".","8","p","h"]
    fig,ax = plt.subplots(figsize=(8,4))
    for j,value in enumerate(values):
        ax.errorbar(x_dep_val,value[:,0],yerr=value[:,1],linestyle = "",marker=markers[j%len(markers)],\
                    capsize=3,label=r"$Q^2$ = {}-{} $GeV^2$".format(Q2_range[j][0]*10**-6,Q2_range[j][1]*10**-6))
    ax.plot([], [], ' ', label=r"$L_{{int}} = {}\cdot 1O^6 nb^{{-1}}$""\n"r"$S_{{eN}} = {} GeV^2$""\n"\
             r"x = {}-{}$""\n"r"$\alpha_p = {}-{}$".format(int(L_int*10**-6),int(s_ed*10**-6/2),x_range[0],
                                                         x_range[1],alpha_p_range[0],alpha_p_range[1]))
    #ax.set_ylim([-0.1,0.])
    ax.set_ylabel(r"$A_{\parallel d}^{(3)}$[vector]")
    ax.set_xlabel(x_label)
    ax.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    plt.tight_layout()
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    plt.show()
    
    return fig
    
def make_figure_A_d_3_vec(out):
    intro = widgets.HTMLMath(r"""<center><b><h1>Spin asymmetry $A_{\parallel d}^{(3)}$[vector]</h1></b></center>
    <hr>
    <figure style="float: right;">
        <img src="figures/figure_A_d_3_vector.png" width ="400">
        <figcaption style="text-align:center">fig: An example figure of the <b>Spin asymmetry<br/>
        $A_{\parallel d}^{(3)}$[vector]</b> program</figcaption>
    </figure>
    
    <p>&emsp;This cell can create a figure of the vector three-state parallel spin asymmetry 
    $A_{\parallel d}^{(3)}[\text{vector}]$ for a certain integrated luminosity, which can be given at <b>"$L_{int}$"</b>, and an 
    electron-Nucleon center of mass energy, provided at <b>"$s_{eN}$"</b>.</p>
    
    <p>&emsp;In the given interval for the Bjorken scaling variable, from 
    <b>"Bjorken $x$ lower bound"</b> to <b>"Bjorken $x$ upper bound"</b>, for intervals of the 4 momentum transfer 
    squared in $MeV^2$ which begin from <b>"$Q^2$ lower bound"</b> and end at <b>"$Q^2$ upper bound"</b>, for the 
    proton transverse momentum fraction interval from <b>"$\alpha_p$ lower bound"</b> to <b>"$\alpha_p$ upper 
    bound"</b> and for a range of kinematic variables that can be:<br/> 
    &emsp;- The transverse prtoton momentum, $p_{pT}$ (in MeV)<br/> 
    &emsp;- The proton 3 momentum norm, k (in MeV)<br/> 
    &emsp;- The invariant momentum transfer, t' ($in MeV^2$).<br/>
    Which kinematic variable is given by the user can be indicated at the <b>"kinematic variable"</b> radio 
    buttons. The begin 
    and end of the interval can be given, respectively, at <b>"kin. var. lower bound"</b> and <b>"kin. var upper 
    bound"</b>.
    <b>"$\Delta$ kin. var."</b> is the spacing between the values. There are different parametrizations of the 
    structure function which can be selected in the drop down menu <b>"Structure Function"</b>. There are also
    2 AV18 Wave function parametrizations, AV18a and AV18b, this can be chosen by the use of the <b>"Wave 
    function"</b> radio buttons.</p>
    
    <p>&emsp;The <b>"x-axis variable"</b> is the kinematic variable that will be displayed on the x-axis of the 
    figure. The
    If a name is given in the box titled <b>"Figure name"</b>, the figure is stored as this name.png. 
    If no name is given the figure is not saved but there is the option to save the figure after it is displayed. 
    After displaying, and when no name was given, a box titled <b>"Figure name"</b> appears in which a name of the
    figure can be provided as which the figure will be saved after pressing the <b>"Save figure"</b> button.</p>
    
    <p>&emsp;A figure using the bin average values can be made by pressing the <b>"Make A_d_3[vector] bin average 
    figure"</b>
    button. The central values of the given bins will be used to do all the calculations. The values can also
    be calculated using a Monte Carlo integration in these bins, a figure using these values will be displayed 
    after pressing the <b>"Make A_d_3[vector] MC figure"</b> button.</p>
    
    <p>&emsp; For the MC integration the <a href="https://vegas.readthedocs.io/en/latest/"> vegas</a> algorithm
    is used which uses a machine learning algorithm approach to best estimate the value of the multidimensional
    integral. The integral wil be estimated using <b>"MC iterations"</b> iterations of the vegas algorithm where
    for each iteration no more than <b>"MC evaluations"</b> evaluations of the integrand to estimate the integral.
    To produce independent estimates of the integral from which the weighted average is taken. Standard values are
    10 for both values.</p><hr>""")
    
    L_int_val   =widgets.Text(value='2e7',placeholder=r'Integrated luminosity',
                              description=r'$L_{int}$ $(nb^{-1})$',disabled=False,style=style,layout=layout)
    s_en_val    =widgets.Text(value='1e9',placeholder=r'squared e-d C.O.M energy ',
                              description=r'$s_{eN}$ $(MeV^2)$',disabled=False,style=style,layout=layout)
    x_start_val =widgets.Text(value='0.04',placeholder=r'Bjorken variable',
                              description=r'Bjorken $x$ lower bound',disabled=False,style=style,layout=layout)
    x_stop_val  =widgets.Text(value='0.06',placeholder=r'Bjorken variable',
                              description=r'Bjorken $x$ upper bound',disabled=False,style=style,layout=layout)
    Q2_start_val=widgets.Text(value='13e6,20e6,30e6',placeholder=r'4 momentum transfer squared',
                              description=r'$Q^2$ lower bound $(MeV^2)$',disabled=False,style=style,layout=layout)
    Q2_stop_val =widgets.Text(value='20e6,30e6,40e6',placeholder=r'4 momentum transfer squared',
                              description=r'$Q^2$ upper bound $(MeV^2)$',disabled=False,style=style,layout=layout)
    alpha_p_start_val =widgets.Text(value='0.98',placeholder=r'proton light cone momentum fraction',
                                   description=r'$\alpha_p$ lower bound',disabled=False,style=style,layout=layout)
    alpha_p_stop_val  =widgets.Text(value='1.02',placeholder=r'proton light cone momentum fraction',
                                   description=r'$\alpha_p$ upper bound',disabled=False,style=style,layout=layout)
    x_param_val      =widgets.RadioButtons(options=[("t' (MeV^2)","t'"), ("p_pt (MeV)","p_pt"), ("k (MeV)","k")],
                                           description='kinematic variable:',disabled=False,style=style,
                                           layout=layout,value = "t'")
    x_val_start_val  =widgets.Text(value='-6e4',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. lower bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_stop_val   =widgets.Text(value='0',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. upper bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_step_val   =widgets.Text(value='5e3',placeholder=r"t', p_{pt}, k",
                                   description=r'$\Delta$ kin. var. $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    SF_parametriation_val =widgets.Dropdown(options=[("CB: Christy & Bosted parametrization", "CB"),
    ("SLAC: SLAC paramtetrization from Bodek", "SLAC"),
    ("Alekhin: leading twist parametrization by Alekhin [see PRD 68,014002]", "Alekhin"),
    ("CTEQ: F2 based on the pdf's from CTEQ (code from Misak)", "CTEQ"),
    ("HMRS: F2 from pdfs from unknown source (Shunzo Kumano is the source, not enough info in his code file)", 
     "HMRS"),
    ("MSTW: F2 from LO (or nlo etc) from MSTW pdfs", "MSTW")],value="CTEQ",description='Structure Function:',
                                            disabled=False,style = style,layout = layout)  
    WF_parametrization_val=widgets.RadioButtons(options=[("AV18a","first"), ("AV18b","second")],
                                                description='Wave function:',disabled=False,style = style,
                                                layout = layout)
    x_dep_val    =widgets.RadioButtons(options=[(r"t'(GeV^2)","t'"), (r"p_pt (GeV)","p_pt"), ("k (GeV)", "k")],
                                       description='x-axis variable:',disabled=False,style=style,layout=layout,
                                       value = "t'")
    save =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                       disabled=False,style=style,layout=layout)
    nitn =widgets.Text(value='5',placeholder=r"nitn",description=r'MC iterations:',disabled=False,style=style,
                       layout=layout)
    neval=widgets.Text(value='5',placeholder=r"neval",description=r'MC evaluations:',disabled=False,
                       style=style,layout=layout)
    
    make_figure_Ad3_av =widgets.Button(description='Make A_d_3[vector] bin average figure',disabled=False,
                                       button_style='',tooltip='make figure',icon='check',
                                       layout=Layout(width="initial"))
    make_figure_Ad3_mc =widgets.Button(description='Make A_d_3[vector] MC figure',disabled=False,button_style='',
                                  tooltip='make figure',icon='check',layout=Layout(width="initial"))

    display(intro)
    
    display(widgets.HBox((L_int_val,s_en_val)))
    display(widgets.HBox((x_start_val, x_stop_val)))
    display(widgets.HBox((Q2_start_val, Q2_stop_val)))
    display(widgets.HBox((alpha_p_start_val, alpha_p_stop_val)))
    display(widgets.HBox((x_param_val,x_val_start_val)))
    display(widgets.HBox(( x_val_stop_val,x_val_step_val)))
    display(widgets.HBox((SF_parametriation_val, WF_parametrization_val)))
    display(widgets.HBox((x_dep_val,save)))
    display(widgets.HBox((nitn,neval)))

    display(widgets.HBox((make_figure_Ad3_av,make_figure_Ad3_mc)))
    
    def Ad3_fig(b):
        with out:
            clear_output()
            L_int  = float(L_int_val.value)
            s_ed   = 2.*float(s_en_val.value)
            x_int  = np.array([float(x_start_val.value),float(x_stop_val.value)])
            Q2_starts = Q2_start_val.value.split(",")
            Q2_stops  = Q2_stop_val.value.split(",")
            length    = min(len(Q2_starts),len(Q2_stops))
            Q2_int = np.array([[float(Q2_starts[i]),float(Q2_stops[i])] for i in range(length)])
            alpha_p_int = np.array([float(alpha_p_start_val.value),float(alpha_p_stop_val.value)])
            x_val_int   = np.arange(float(x_val_start_val.value),float(x_val_stop_val.value),
                                    float(x_val_step_val.value))
            if b.description == "Make A_d_3[vector] bin average figure":
                figure = plot_Ad3(L_int,s_ed,x_int,Q2_int,alpha_p_int,x_val_int,x_param_val.value,
                                  SF_parametriation_val.value,WF_parametrization_val.value,x_dep_val.value,
                                  save.value,False)
            else:
                figure = plot_Ad3(L_int,s_ed,x_int,Q2_int,alpha_p_int,x_val_int,x_param_val.value,
                                  SF_parametriation_val.value,WF_parametrization_val.value,x_dep_val.value,
                                  save.value,True,int(nitn.value),int(neval.value))
            if save.value:
                save.value = ""
            else:
                save_after_t=widgets.Text(value='',placeholder=r"Name of figure for saving",
                                          description=r'Figure name:',disabled=False,
                                          style={'description_width': 'initial'},layout=Layout(width="initial"))
                save_after_b=widgets.Button(description='Save figure',disabled=False,button_style='',
                                            tooltip='save',icon='check',style={'description_width': 'initial'},
                                            layout=Layout(width="initial"))
                display(widgets.HBox((save_after_t,save_after_b)))
                def on_clicked(_):
                    if not "png" in save_after_t.value and not "jpg" in save_after_t.value and\
                    not "pdf" in save_after_t.value:
                        save_after_t.value += ".png"
                    saving = os.path.join(os.getcwd(),"output",save_after_t.value)
                    figure.savefig(saving,dpi=600)
                    print("saved as {}".format(saving))
                    save_after_t.value=""
                save_after_b.on_click(on_clicked)

    make_figure_Ad3_av.on_click(Ad3_fig)
    make_figure_Ad3_mc.on_click(Ad3_fig)
    
#out = Output()
#make_figure_A_d_3_vec(out)
#out

In [144]:
def A_d_2(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation = "CTEQ",WF_parametrization = "first"):
    """
    The parallel spin asymmetry
    
    x        (float)           : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                 the absence of nuclear binding.
    Q2       (float)(MeV^2)    : the negative of the 4 momentum transfer squared or virtual photon momentum squared
    alpha_p  (float)           : the proton light cone momentum fraction
    p_pt    (float)(MeV)      : the transverse momentum value, p_pt.
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 If not specified the "first" one is used.
    
    returns
    """
    
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_val)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_val**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    p_pt = p_trans()
    
    x_d     = x/2.
    y       = Q2/(x_d*(s_ed-MD**2))
    gamma2  = (x*MD)**2/Q2 #2x_d = x
    epsilon = (1-y-gamma2*y**2/4)/(1-y+y**2/2+gamma2*y**2/4)
    
    #structure function________________________________
    struc_f = struc_func(SF_parametriation, WF_parametrization)
    F = struc_f.FTd(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FLd(x,Q2,alpha_p,p_pt)
    FT = struc_f.FTLLT(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FTLLL(x,Q2,alpha_p,p_pt)
    
    #spin asymmetry____________________________________
    D_L = y*(1.-y/2.)*(1.+gamma2*y/2.)/(1.-y+y**2/2.+gamma2*y**2/4.)
    asymmetry = D_L*struc_f.FSLd(x,Q2,alpha_p,p_pt)/(F+FT)
    
    return asymmetry

def A_d_2_err(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,SF_parametriation = "CTEQ",\
              WF_parametrization = "first"):
    
     #bin averages______________________________________
    x       = np.mean(x_range)
    Q2      = np.mean(Q2_range)
    alpha_p = np.mean(alpha_p_range)
    x_val   = np.mean(x_val_range)
    
    def t_apo(val):
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*val**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*val**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_val)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_val**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    p_pt = p_trans()
    
    #bin sizes_________________________________________
    delta_x       = x_range[1]-x_range[0]
    delta_Q2      = Q2_range[1]-Q2_range[0]
    delta_alpha_p = alpha_p_range[1]-alpha_p_range[0]
    delta_x_val   = x_val_range[1]-x_val_range[0]
    delta_t_p     = abs(t_apo(x_val_range[1]-x_val_range[0]))
    
    x_d     = x/2.
    y       = Q2/(x_d*(s_ed-MD**2))
    gamma2  = (x*MD)**2/Q2 #2x_d = x
    epsilon = (1-y-gamma2*y**2/4)/(1-y+y**2/2+gamma2*y**2/4)
    
    #calculating the error
    delta = delta_x*delta_Q2*delta_alpha_p*delta_t_p
    #error of the numerator
    #spins = [(1./2.,1,1.),(-1./2.,1,-1.),(1./2.,-1,-1.),(-1./2.,-1,1.)] #(sigma_e,lambda_d,add/subtract)
    spins = [(1./2.,1,2.),(-1./2.,1,-2.)]
    num = 0.
    sq_err_sum = 0.
    for spin in spins:
        sigma_e,lambda_d,opperation=spin
        value = full_cross_sec(s_ed,x,Q2,alpha_p,x_val,x_parameter,sigma_e,lambda_d,\
                               SF_parametriation,WF_parametrization)
        
        sq_err_sum += opperation**2*value 
        num        += opperation*value
    sq_rel_num_err = sq_err_sum/(L_int*delta*num**2)
    
    #error of the denominator
    #structure function________________________________
    struc_f = struc_func(SF_parametriation, WF_parametrization)
    F = struc_f.FTd(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FLd(x,Q2,alpha_p,p_pt)
    FT = struc_f.FTLLT(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FTLLL(x,Q2,alpha_p,p_pt)    
    
    sq_rel_den_err = 1./(L_int*delta*(F+FT))    
    rel_error = np.sqrt(sq_rel_den_err + sq_rel_num_err)
    
    asymmetry = A_d_2(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation,WF_parametrization)
    error = rel_error*asymmetry
    
    return [asymmetry,error]

def A_d_2_err_MC(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,SF_parametriation = "CTEQ",\
                 WF_parametrization = "first",nitn=1,neval=1):
    """
    The parallel spin asymmetry with mc integration for error
    
    x        (float)           : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                 the absence of nuclear binding.
    Q2       (float)(MeV^2)    : the negative of the 4 momentum transfer squared or virtual photon momentum squared
    alpha_p  (float)           : the proton light cone momentum fraction
    p_pt    (float)(MeV)      : the transverse momentum value, p_pt.
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 If not specified the "first" one is used.
    
    returns
    """
    def p_trans(alpha_p_t,x_val_t):
        if x_parameter == "t'":
            return np.sqrt((alpha_p_t*(2.-alpha_p_t)*MD**2-4*MN**2-2*alpha_p_t*x_val_t)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p_t*(2-alpha_p_t)*(x_val_t**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    #bin averages______________________________________
    x       = np.mean(x_range)
    Q2      = np.mean(Q2_range)
    alpha_p = np.mean(alpha_p_range)
    x_val   = np.mean(x_val_range)    
    
    
    asymmetry = A_d_2(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation,WF_parametrization)
    
    #error of the numerator
    #spins = [(1./2.,1,1.),(-1./2.,1,-1.),(1./2.,-1,-1.),(-1./2.,-1,1.)] #(sigma_e,lambda_d,add/subtract)
    spins = [(1./2.,1,2.),(-1./2.,1,-2.)]
    num = 0.
    sq_err_sum = 0.
    for spin in spins:
        sigma_e,lambda_d,opperation=spin
        
        def integrandum_num(values):
            x_p,Q2_p,alpha_p_p,x_val_p = values
            return full_cross_sec(s_ed,x_p,Q2_p,alpha_p_p,x_val_p,x_parameter,sigma_e,lambda_d,\
                                  SF_parametriation,WF_parametrization)
        
        integ  = vegas.Integrator([x_range,Q2_range,alpha_p_range,x_val_range])
        result = integ(integrandum_num,nitn=nitn,neval=neval)
        Delta_N  = L_int*result.mean
        value = integrandum_num((x,Q2,alpha_p,x_val))
        sq_err_sum += (opperation*value)**2/Delta_N
        
        num += opperation*value
        
    sq_rel_num_err = sq_err_sum/num**2
    
    #error of the denominator
    #structure function________________________________
    struc_f = struc_func(SF_parametriation, WF_parametrization)
    def integrandum_den(values):
        x_p,Q2_p,alpha_p_p,x_val_p = values
        p_pt_p = p_trans(alpha_p_p,x_val_p)
        
        x_d_p     = x_p/2.
        y_p       = Q2_p/(x_d_p*(s_ed-MD**2))
        gamma2_p  = (x_p*MD)**2/Q2_p #2x_d = x
        epsilon_p = (1-y_p-gamma2_p*y_p**2/4)/(1-y_p+y_p**2/2+gamma2_p*y_p**2/4)
        
        F = struc_f.FTd(x_p,Q2_p,alpha_p_p,p_pt_p)+epsilon_p*struc_f.FLd(x_p,Q2_p,alpha_p_p,p_pt_p)
        FT = struc_f.FTLLT(x_p,Q2_p,alpha_p_p,p_pt_p)+epsilon_p*struc_f.FTLLL(x_p,Q2_p,alpha_p_p,p_pt_p)
        return F+FT
    
    
    integ  = vegas.Integrator([x_range,Q2_range,alpha_p_range,x_val_range])
    result = integ(integrandum_den,nitn=nitn,neval=neval)
    Delta_N  = L_int*result.mean
    sq_rel_den_err = 1./Delta_N
    
    sq_rel_err = sq_rel_num_err + sq_rel_den_err
    error = asymmetry*np.sqrt(sq_rel_err)    
    
    return [asymmetry,error]

def plot_Ad2(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,SF_parametriation = "CTEQ",\
             WF_parametrization = "first",x_dependency=None,save="",MC=False,nitn=5,neval=5):
    
    #bin averages______________________________________
    alpha_p = np.mean(alpha_p_range)
    x_vals  = np.mean(list(zip(x_val_range[:-1],x_val_range[1:])),axis=1)
    
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_vals)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_vals**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def t_apo():
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*x_vals**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*x_vals**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def k():
        if x_parameter == "t'":
            return np.sqrt(MD**2-4*MN**2-2*x_vals/(2-alpha_p))/4
        elif x_parameter == "p_pt":
            return np.sqrt((x_vals**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2)
        elif x_parameter == "k":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    def prog_bar(i,total):
        clear_output(wait=True)
        bar = widgets.FloatProgress(value=i,min=0,max=total,description='Loading: {:.2f}%'.format(i/total*100),\
                                    bar_style='info',orientation='horizontal',\
                                    style = {'description_width': "initial"})
        display(bar)
    
    values = []
    total_calculations = (len(x_val_range)-1)*(len(Q2_range))
    prog_bar(0,total_calculations)
    for j in range(len(Q2_range)):
        temp_val = []
        for i in range(len(x_val_range)-1):
            if not MC:
                temp_val  += [A_d_2_err(L_int,s_ed,x_range,Q2_range[j],alpha_p_range,x_val_range[i:i+2],
                                           x_parameter,SF_parametriation,WF_parametrization)]
            else:
                temp_val  += [A_d_2_err_MC(L_int,s_ed,x_range,Q2_range[j],alpha_p_range,x_val_range[i:i+2],
                                           x_parameter,SF_parametriation,WF_parametrization,nitn,neval)]
            prog_bar(j*(len(x_val_range)-1)+i+1,total_calculations)
        values += [temp_val]
    values = np.array(values)
    
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        x_dep_val = -t_apo()*10**-6
        x_label = r"-t' [$GeV^2$]"
    elif x_dependency == "p_pt":
        x_dep_val = p_trans()*10**-3 
        x_label = r"$p_{pt}$ [GeV]"
    elif x_dependency == "k":
        x_dep_val = k()*10**-3
        x_label = r"k [GeV]"
    else:
        raise ValueError("no x dependency called {}, choose from t_apo, p_pt, k".format)
    
    markers     = ["+","x","*","s",".","8","p","h"]
    fig,ax = plt.subplots(figsize=(8,4))
    for j,value in enumerate(values):
        ax.errorbar(x_dep_val,value[:,0],yerr=value[:,1],linestyle = "",marker=markers[j%len(markers)],\
                    capsize=3,label=r"$Q^2$ = {}-{} $GeV^2$".format(Q2_range[j][0]*10**-6,Q2_range[j][1]*10**-6))
    ax.plot([], [], ' ', label=r"$L_{{int}} = {}\cdot 1O^6 nb^{{-1}}$""\n"r"$S_{{eN}} = {} GeV^2$""\n"\
             r"$x$ = {}-{}""\n"r"$\alpha_p$ = {}-{}".format(int(L_int*10**-6),int(s_ed*10**-6/2),x_range[0],
                                                         x_range[1],alpha_p_range[0],alpha_p_range[1]))
    #ax.set_ylim([-0.1,0.])
    ax.set_ylim(auto=True)
    ax.set_ylabel(r"$A_{\parallel d}^{(2)}$[vector]")
    ax.set_xlabel(x_label)
    ax.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    ax.set_aspect(0.62*((np.nanmax(x_dep_val)-np.nanmin(x_dep_val))/
                           (np.nanmax(values[:,:,0])-np.nanmin(values[:,:,0]))))
    plt.tight_layout()
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    plt.show()
    
    return fig
    
def make_figure_A_d_2_vec(out):
    intro = widgets.HTMLMath(r"""<center><b><h1>Spin asymmetry $A_{\parallel d}^{(2)}$[vector]</h1></b></center>
    <hr>
    <figure style="float: right;">
        <img src="figures/figure_A_d_2_vector.png" width ="400">
        <figcaption style="text-align:center">fig: An example figure of the <b>Spin asymmetry<br/>
        $A_{\parallel d}^{(2)}$[vector]</b> program</figcaption>
    </figure>
    
    <p>&emsp;This cell can create a figure of the vector two-state parallel spin asymmetry 
    $A_{\parallel d}^{(2)}[\text{vecotr}]$ for a certain integrated luminosity, which can be given at 
    <b>"$L_{int}$"</b>, and an electron-Nucleon center of mass energy, provided at <b>"$s_{eN}$"</b>.</p>
    
    <p>&emsp;In the given interval for the Bjorken scaling variable, from 
    <b>"Bjorken $x$ lower bound"</b> to <b>"Bjorken $x$ upper bound"</b>, for intervals of the 4 momentum 
    transfer squared in $MeV^2$ which begin from <b>"$Q^2$ lower bound"</b> and end at <b>"$Q^2$ upper bound"</b>,
    for the proton transverse momentum fraction interval from <b>"$\alpha_p$ lower bound"</b> to <b>"$\alpha_p$ 
    upper bound"</b> and for a range of kinematic variables that can be:<br/> 
    &emsp;- The transverse prtoton momentum, $p_{pT}$ (in MeV)<br/> 
    &emsp;- The proton 3 momentum norm, k (in MeV)<br/> 
    &emsp;- The invariant momentum transfer, t' ($in MeV^2$).<br/>
    Which kinematic variable is given by the user can be indicated at the <b>"kinematic variable"</b> radio 
    buttons. The begin 
    and end of the interval can be given, respectively, at <b>"kin. var. lower bound"</b> and <b>"kin. var. 
    upper bound"</b>.
    <b>"$\Delta$ kin. var."</b> is the spacing between the values. There are different parametrizations of the 
    structure function which can be selected in the drop down menu <b>"Structure Function"</b>. There are also
    2 AV18 Wave function parametrizations, AV18a and AV18b, this can be chosen by the use of the <b>"Wave 
    function"</b> radio buttons.</p>
    
    <p>&emsp;The <b>"x-axis variable"</b> is the kinematic variable that will be displayed on the x-axis of the 
    figure. The
    If a name is given in the box titled <b>"Figure name"</b>, the figure is stored as this name.png. 
    If no name is given the figure is not saved but there is the option to save the figure after it is displayed. 
    After displaying, and when no name was given, a box titled <b>"Figure name"</b> appears in which a name of the
    figure can be provided as which the figure will be saved after pressing the <b>"Save figure"</b> button.</p>
    
    <p>&emsp;A figure using the bin average values can be made by pressing the <b>"Make A_d_2[vector] bin average 
    figure"</b>
    button. The central values of the given bins will be used to do all the calculations. The values can also
    be calculated using a Monte Carlo integration in these bins, a figure using these values will be displayed 
    after pressing the <b>"Make A_d_2[vector] MC figure"</b> button.</p>
    
    <p>&emsp; For the MC integration the <a href="https://vegas.readthedocs.io/en/latest/"> vegas</a> algorithm
    is used which uses a machine learning algorithm approach to best estimate the value of the multidimensional
    integral. The integral wil be estimated using <b>"MC iterations"</b> iterations of the vegas algorithm where
    for each iteration no more than <b>"MC evaluations"</b> evaluations of the integrand to estimate the integral.
    To produce independent estimates of the integral from which the weighted average is taken.</p><hr>""")
    
    L_int_val   =widgets.Text(value='2e7',placeholder=r'Integrated luminosity',
                              description=r'$L_{int}$ $(nb^{-1})$',disabled=False,style=style,layout=layout)
    s_en_val    =widgets.Text(value='1e9',placeholder=r'squared e-d C.O.M energy ',
                              description=r'$s_{eN}$ $(MeV^2)$',disabled=False,style=style,layout=layout)
    x_start_val =widgets.Text(value='0.04',placeholder=r'Bjorken variable',
                              description=r'Bjorken $x$ lower bound',disabled=False,style=style,layout=layout)
    x_stop_val  =widgets.Text(value='0.06',placeholder=r'Bjorken variable',
                              description=r'Bjorken $x$ upper bound',disabled=False,style=style,layout=layout)
    Q2_start_val=widgets.Text(value='13e6,20e6,30e6',placeholder=r'4 momentum transfer squared',
                              description=r'$Q^2$ lower bound $(MeV^2)$',disabled=False,style=style,layout=layout)
    Q2_stop_val =widgets.Text(value='20e6,30e6,40e6',placeholder=r'4 momentum transfer squared',
                              description=r'$Q^2$ upper bound $(MeV^2)$',disabled=False,style=style,layout=layout)
    alpha_p_start_val =widgets.Text(value='0.98',placeholder=r'proton light cone momentum fraction',
                                   description=r'$\alpha_p$ lower bound',disabled=False,style=style,layout=layout)
    alpha_p_stop_val  =widgets.Text(value='1.02',placeholder=r'proton light cone momentum fraction',
                                   description=r'$\alpha_p$ upper bound',disabled=False,style=style,layout=layout)
    x_param_val      =widgets.RadioButtons(options=[("t' (MeV^2)","t'"), ("p_pt (MeV)","p_pt"), ("k (MeV)","k")],
                                           description='kinematic variable:',disabled=False,style=style,
                                           layout=layout,value = "t'")
    x_val_start_val  =widgets.Text(value='-6e4',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. lower bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_stop_val   =widgets.Text(value='0',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. upper bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_step_val   =widgets.Text(value='5e3',placeholder=r"t', p_{pt}, k",
                                   description=r'$\Delta$ kin. var. $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    SF_parametriation_val =widgets.Dropdown(options=[("CB: Christy & Bosted parametrization", "CB"),
    ("SLAC: SLAC paramtetrization from Bodek", "SLAC"),
    ("Alekhin: leading twist parametrization by Alekhin [see PRD 68,014002]", "Alekhin"),
    ("CTEQ: F2 based on the pdf's from CTEQ (code from Misak)", "CTEQ"),
    ("HMRS: F2 from pdfs from unknown source (Shunzo Kumano is the source, not enough info in his code file)", 
     "HMRS"),
    ("MSTW: F2 from LO (or nlo etc) from MSTW pdfs", "MSTW")],value="CTEQ",description='Structure Function:',
                                            disabled=False,style = style,layout = layout)  
    WF_parametrization_val=widgets.RadioButtons(options=[("AV18a","first"), ("AV18b","second")],
                                                description='Wave function:',disabled=False,style = style,
                                                layout = layout)
    x_dep_val    =widgets.RadioButtons(options=[(r"t'(GeV^2)","t'"), (r"p_pt (GeV)","p_pt"), ("k (GeV)", "k")],
                                       description='x-axis variable:',disabled=False,style=style,layout=layout,
                                       value = "t'")
    save =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                       disabled=False,style=style,layout=layout)
    nitn =widgets.Text(value='5',placeholder=r"nitn",description=r'MC iterations:',disabled=False,style=style,
                       layout=layout)
    neval=widgets.Text(value='5',placeholder=r"neval",description=r'MC evaluations:',disabled=False,
                       style=style,layout=layout)
    
    make_figure_Ad2_av =widgets.Button(description='Make A_d_2[vector] bin average figure',disabled=False,
                                       button_style='',
                                       tooltip='make figure',icon='check',layout=Layout(width="initial"))
    make_figure_Ad2_mc =widgets.Button(description='Make A_d_2[vector] MC figure',disabled=False,button_style='',
                                  tooltip='make figure',icon='check',layout=Layout(width="initial"))
    
    display(intro)
    
    display(widgets.HBox((L_int_val,s_en_val)))
    display(widgets.HBox((x_start_val, x_stop_val)))
    display(widgets.HBox((Q2_start_val, Q2_stop_val)))
    display(widgets.HBox((alpha_p_start_val, alpha_p_stop_val)))
    display(widgets.HBox((x_param_val,x_val_start_val)))
    display(widgets.HBox(( x_val_stop_val,x_val_step_val)))
    display(widgets.HBox((SF_parametriation_val, WF_parametrization_val)))
    display(widgets.HBox((x_dep_val,save)))
    display(widgets.HBox((nitn,neval)))

    display(widgets.HBox((make_figure_Ad2_av,make_figure_Ad2_mc)))
    
    def Ad2_fig(b):
        with out:
            clear_output()
            L_int  = float(L_int_val.value)
            s_ed   = 2.*float(s_en_val.value)
            x_int  = np.array([float(x_start_val.value),float(x_stop_val.value)])
            Q2_starts = Q2_start_val.value.split(",")
            Q2_stops  = Q2_stop_val.value.split(",")
            length    = min(len(Q2_starts),len(Q2_stops))
            Q2_int = np.array([[float(Q2_starts[i]),float(Q2_stops[i])] for i in range(length)])
            alpha_p_int = np.array([float(alpha_p_start_val.value),float(alpha_p_stop_val.value)])
            x_val_int   = np.arange(float(x_val_start_val.value),float(x_val_stop_val.value),
                                    float(x_val_step_val.value))
            if b.description == "Make bin average figure":
                figure = plot_Ad2(L_int,s_ed,x_int,Q2_int,alpha_p_int,x_val_int,x_param_val.value,
                                  SF_parametriation_val.value,WF_parametrization_val.value,x_dep_val.value,
                                  save.value,False)
            else:
                figure = plot_Ad2(L_int,s_ed,x_int,Q2_int,alpha_p_int,x_val_int,x_param_val.value,
                                  SF_parametriation_val.value,WF_parametrization_val.value,x_dep_val.value,
                                  save.value,True,int(nitn.value),int(neval.value))
                
            if save.value:
                save.value = ""
            else:
                save_after_t=widgets.Text(value='',placeholder=r"Name of figure for saving",
                                          description=r'Figure name:',disabled=False,
                                          style={'description_width': 'initial'},layout=Layout(width="initial"))
                save_after_b=widgets.Button(description='Save figure',disabled=False,button_style='',
                                            tooltip='save',icon='check',style={'description_width': 'initial'},
                                            layout=Layout(width="initial"))
                display(widgets.HBox((save_after_t,save_after_b)))
                def on_clicked(_):
                    if not "png" in save_after_t.value and not "jpg" in save_after_t.value and\
                    not "pdf" in save_after_t.value:
                        save_after_t.value += ".png"
                    saving = os.path.join(os.getcwd(),"output",save_after_t.value)
                    figure.savefig(saving,dpi=600)
                    print("saved as {}".format(saving))
                    save_after_t.value=""
                save_after_b.on_click(on_clicked)

    make_figure_Ad2_av.on_click(Ad2_fig)
    make_figure_Ad2_mc.on_click(Ad2_fig)
    
#out = Output()
#make_figure_A_d_2_vec(out)
#out

In [161]:
def A_d_3_ten(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation = "CTEQ",WF_parametrization = "first"):
    """
    The tensor parallel spin asymmetry
    
    x        (float)           : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                 the absence of nuclear binding.
    Q2       (float)(MeV^2)    : the negative of the 4 momentum transfer squared or virtual photon momentum squared
    alpha_p  (float)           : the proton light cone momentum fraction
    p_pt    (float)(MeV)      : the transverse momentum value, p_pt.
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 If not specified the "first" one is used.
    
    returns
    """
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_val)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_val**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
            
    p_pt = p_trans()
    
    x_d     = x/2.
    y       = Q2/(x_d*(s_ed-MD**2))
    gamma2  = (x*MD)**2/Q2 #2x_d = x
    epsilon = (1-y-gamma2*y**2/4)/(1-y+y**2/2+gamma2*y**2/4)
    
    #structure function________________________________
    struc_f = struc_func(SF_parametriation, WF_parametrization)
    F = struc_f.FTd(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FLd(x,Q2,alpha_p,p_pt)
    FT = struc_f.FTLLT(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FTLLL(x,Q2,alpha_p,p_pt)
    
    #spin asymmetry____________________________________
    asymmetry = (3./2.)*FT/F
    
    return asymmetry

def A_d_3_ten_err(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,SF_parametriation = "CTEQ",
              WF_parametrization = "first"):
    
    def t_apo(val):
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*val**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*val**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    #bin averages______________________________________
    x       = np.mean(x_range)
    Q2      = np.mean(Q2_range)
    alpha_p = np.mean(alpha_p_range)
    x_val   = np.mean(x_val_range)
    
    #bin sizes_________________________________________
    delta_x       = x_range[1]-x_range[0]
    delta_Q2      = Q2_range[1]-Q2_range[0]
    delta_alpha_p = alpha_p_range[1]-alpha_p_range[0]
    delta_x_val   = x_val_range[1]-x_val_range[0]
    delta_t_p     = abs(t_apo(x_val_range[1]-x_val_range[0]))
    
    #calculating the error
    delta = delta_x*delta_Q2*delta_alpha_p*delta_t_p
    #error of the numerator
    spins = [(1./2.,1,2.),(1./2.,-1,2.),(1./2.,0,-4.)] #(sigma_e,lambda_d,add/subtract)
    
    num = 0.
    sq_err_sum = 0.
    for spin in spins:
        sigma_e,lambda_d,opperation=spin
        value = full_cross_sec(s_ed,x,Q2,alpha_p,x_val,x_parameter,sigma_e,lambda_d,\
                               SF_parametriation,WF_parametrization)
        
        sq_err_sum += opperation**2*value 
        num        += opperation*value
    sq_rel_num_err = sq_err_sum/(L_int*delta*num**2)
    
    #error of the denominator
    denom = float(diff_cross_sec(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation,\
                                 WF_parametrization,sigma_dependency="t'")[1,1])
    
    sq_rel_den_err = 1./(L_int*delta*denom)
    rel_error = np.sqrt(sq_rel_den_err + sq_rel_num_err)
    
    asymmetry = A_d_3_ten(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation,WF_parametrization)
    error = rel_error*asymmetry
    
    return [asymmetry,error]

def A_d_3_ten_err_MC(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,
                     SF_parametriation = "CTEQ",WF_parametrization = "first",nitn=1,neval=1):
    """
    The parallel spin asymmetry with mc integration for error
    
    x        (float)           : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                 the absence of nuclear binding.
    Q2       (float)(MeV^2)    : the negative of the 4 momentum transfer squared or virtual photon momentum squared
    alpha_p  (float)           : the proton light cone momentum fraction
    p_pt    (float)(MeV)      : the transverse momentum value, p_pt.
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 If not specified the "first" one is used.
    
    returns
    """
    #bin averages______________________________________
    x       = np.mean(x_range)
    Q2      = np.mean(Q2_range)
    alpha_p = np.mean(alpha_p_range)
    x_val   = np.mean(x_val_range)    
    
    
    asymmetry = A_d_3_ten(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation,WF_parametrization)
    
    #error of the numerator
    spins = [(1./2.,1,2.),(1./2.,-1,2.),(1./2.,0,-4)] #(sigma_e,lambda_d,add/subtract)
    
    num = 0.
    sq_err_sum = 0.
    for spin in spins:
        sigma_e,lambda_d,opperation=spin
        
        def integrandum_num(values):
            x_p,Q2_p,alpha_p_p,x_val_p = values
            return full_cross_sec(s_ed,x_p,Q2_p,alpha_p_p,x_val_p,x_parameter,sigma_e,lambda_d,\
                                  SF_parametriation,WF_parametrization)
        
        integ  = vegas.Integrator([x_range,Q2_range,alpha_p_range,x_val_range])
        result = integ(integrandum_num,nitn=nitn,neval=neval)
        Delta_N  = L_int*result.mean
        value = integrandum_num((x,Q2,alpha_p,x_val))
        
        sq_err_sum += (opperation*value)**2/Delta_N
        num += opperation*value
        
    sq_rel_num_err = sq_err_sum/num**2
    
    #error of the denominator
    def integrandum_den(values):
        x_p,Q2_p,alpha_p_p,x_val_p = values
        return float(diff_cross_sec(s_ed,x_p,Q2_p,alpha_p_p,x_val_p,x_parameter,SF_parametriation,\
                                    WF_parametrization,sigma_dependency="t'")[1,1])
     
    integ  = vegas.Integrator([x_range,Q2_range,alpha_p_range,x_val_range])
    result = integ(integrandum_den,nitn=nitn,neval=neval)
    Delta_N  = L_int*result.mean
    sq_rel_den_err = 1./Delta_N
    
    sq_rel_err = sq_rel_num_err + sq_rel_den_err
    error = asymmetry*np.sqrt(sq_rel_err)    
    
    return [asymmetry,error]

def plot_Ad3_ten(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,SF_parametriation = "CTEQ",\
             WF_parametrization = "first",x_dependency=None,save="",MC=False,nitn=5,neval=5):
    """
    plots the parallel spin asymetry A_{\parallel d}^{(3)} for the bin average values in function of the values in 
    x_val_interval.
    
    L_int   (float)(nb^-1)       : the integrated luminosity
    s_ed    (float)(MeV^2)       : the invariant electron deuteron squared centre of mass energy
    x_range (array[float])       : begin and end value of the bin of the effective Bjorken variable for 
                                   scattering from a nucleon in the deuteronin the absence of nuclear binding.
    Q2_range(array[float])(MeV^2): array with begin and end values of the bins of the negative of the 4 momentum 
                                   transfer squared or virtual photon momentum squared
    alpha_p_range (array[float]) : begin and end value of the bin of the proton light cone momentum fraction
    x_val_range   (array[float])(MeV^2/MeV): an array of evenly spaced values that creates bins of x_vals which 
                                             will be used as the x-axis, these can either be an invariant momentum 
                                             transfer between the deuteron and the proton minus the proton mass, 
                                             t', a transverse momentum value, p_pt, or a norm of the
                                             proton/neuteron 3 momentum, k.
    x_parameter        (string): which parameter is used in x_val:
                                 - "t''": the invariant momentum transfer, t'
                                 - "p_pt" : the transverse momentum, p_{pt}
                                 - "k"    : the 3 momentum norm, k
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 if not specified the "first" one is used.
    x_dependency       (string): the dependency on the x-axis of the plot. the values in x_val_interval do not 
                                 have to be the same parameters as the ones that need to be plotted:
                                 - "t'": the invariant momentum transfer, t'
                                 - "p_pt" : the transverse momentum, p_{pt}
                                 - "k"    : the 3 momentum norm, k
                                 if not specified x_parameter is used.
    save (string): if the figure need to be saved give the name of the figure
    
    returns the figure
    e.g.
    
    """
    #bin averages______________________________________
    alpha_p = np.mean(alpha_p_range)
    x_vals  = np.mean(list(zip(x_val_range[:-1],x_val_range[1:])),axis=1)
    
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_vals)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_vals**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def t_apo():
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*x_vals**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*x_vals**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def k():
        if x_parameter == "t'":
            return np.sqrt(MD**2-4*MN**2-2*x_vals/(2-alpha_p))/4
        elif x_parameter == "p_pt":
            return np.sqrt((x_vals**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2)
        elif x_parameter == "k":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    def prog_bar(i,total):
        clear_output(wait=True)
        bar = widgets.FloatProgress(value=i,min=0,max=total,description='Loading: {:.2f}%'.format(i/total*100),\
                                    bar_style='info',orientation='horizontal',\
                                    style = {'description_width': "initial"})
        display(bar)
    
    
    values = []
    total_calculations = (len(x_val_range)-1)*(len(Q2_range))
    prog_bar(0,total_calculations)
    for j in range(len(Q2_range)):
        temp_val = []
        for i in range(len(x_val_range)-1):
            if not MC:
                temp_val  += [A_d_3_ten_err(L_int,s_ed,x_range,Q2_range[j],alpha_p_range,x_val_range[i:i+2],
                                            x_parameter,SF_parametriation,WF_parametrization)]
            else:
                temp_val  += [A_d_3_ten_err_MC(L_int,s_ed,x_range,Q2_range[j],alpha_p_range,x_val_range[i:i+2],
                                               x_parameter,SF_parametriation,WF_parametrization,nitn,neval)]
            prog_bar(j*(len(x_val_range)-1)+i+1,total_calculations)
        values += [temp_val]
    values = np.array(values)
    
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        x_dep_val = -t_apo()*10**-6
        x_label = r"-t' [$GeV^2$]"
    elif x_dependency == "p_pt":
        x_dep_val = p_trans()*10**-3 
        x_label = r"$p_{pt}$ [GeV]"
    elif x_dependency == "k":
        x_dep_val = k()*10**-3
        x_label = r"k [GeV]"
    else:
        raise ValueError("no x dependency called {}, choose from t_apo, p_pt, k".format)
    
    markers     = ["+","x","*","s",".","8","p","h"]
    fig,ax = plt.subplots(figsize=(8,4))
    for j,value in enumerate(values):
        ax.errorbar(x_dep_val,value[:,0],yerr=value[:,1],linestyle = "",marker=markers[j%len(markers)],\
                    capsize=3,label=r"$Q^2$ = {}-{} $GeV^2$".format(Q2_range[j][0]*10**-6,Q2_range[j][1]*10**-6))
    ax.plot([], [], ' ', label=r"$L_{{int}} = {}\cdot 1O^6 nb^{{-1}}$""\n"r"$S_{{eN}} = {} GeV^2$""\n"\
             r"x = {}-{}""\n"r"$\alpha_p$ = {}-{}".format(int(L_int*10**-6),int(s_ed*10**-6/2),x_range[0],
                                                         x_range[1],alpha_p_range[0],alpha_p_range[1]))
    #ax.set_ylim([-0.1,0.])
    ax.set_ylabel(r"$A_{\parallel d}^{(3)}$ [tensor]")
    ax.set_xlabel(x_label)
    ax.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    plt.tight_layout()
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving,dpi=600)
        print("saved as {}".format(saving))
    plt.show()
    
    return fig

def make_figure_A_d_3_ten(out):
    intro = widgets.HTMLMath(r"""<center><b><h1>Spin asymmetry $A_{\parallel d}^{(3)}$[tensor]</h1></b></center>
    <hr>
    <figure style="float: right;">
        <img src="figures/figure_A_d_3_tensor.png" width ="400">
        <figcaption style="text-align:center">fig: An example figure of the <b>Spin asymmetry 
        $A_{\parallel d}^{(3)}$[tensor]</b> program</figcaption>
    </figure>
    
    <p>&emsp;This cell can create a figure of the tensor three-state parallel spin asymmetry 
    $A_{\parallel d}^{(3)}$[tensor] for a certain integrated luminosity, which can be given at <b>"$L_{int}$"</b>, and an 
    electron-Nucleon center of mass energy, provided at <b>"$s_{eN}$"</b>.</p>
    
    <p>&emsp;In the given interval for the Bjorken scaling variable, from 
    <b>"Bjorken $x$ lower bound"</b> to <b>"Bjorken $x$ upper bound"</b>, for intervals of the 4 momentum transfer 
    squared in $MeV^2$ which begin from <b>"$Q^2$ lower bound"</b> and end at <b>"$Q^2$ upper bound"</b>, for the 
    proton transverse momentum fraction interval from <b>"$\alpha_p$ lower bound"</b> to <b>"$\alpha_p$ upper 
    bound"</b> and for a range of kinematic variables that can be:<br/> 
    &emsp;- The transverse prtoton momentum, $p_{pT}$ (in MeV)<br/> 
    &emsp;- The proton 3 momentum norm, k (in MeV)<br/> 
    &emsp;- The invariant momentum transfer, t' ($in MeV^2$).<br/>
    Which kinematic variable is given by the user can be indicated at the <b>"kinematic variable"</b> radio 
    buttons. The begin 
    and end of the interval can be given, respectively, at <b>"kin. var. lower bound"</b> and <b>"kin. var upper 
    bound"</b>.
    <b>"$\Delta$ var."</b> is the spacing between the values. There are different parametrizations of the 
    structure function which can be selected in the drop down menu <b>"Structure Function"</b>. There are also
    2 AV18 Wave function parametrizations, AV18a and AV18b, this can be chosen by the use of the <b>"Wave 
    function"</b> radio buttons.</p>
    
    <p>&emsp;The <b>"x-axis variable"</b> is the kinematic variable that will be displayed on the x-axis of the 
    figure. The
    If a name is given in the box titled <b>"Figure name"</b>, the figure is stored as this name.png. 
    If no name is given the figure is not saved but there is the option to save the figure after it is displayed. 
    After displaying, and when no name was given, a box titled <b>"Figure name"</b> appears in which a name of the
    figure can be provided as which the figure will be saved after pressing the <b>"Save figure"</b> button.</p>
    
    <p>&emsp;A figure using the bin average values can be made by pressing the <b>"Make A_d_3[tensor] bin average 
    figure"</b>
    button. The central values of the given bins will be used to do all the calculations. The values can also
    be calculated using a Monte Carlo integration in these bins, a figure using these values will be displayed 
    after pressing the <b>"Make A_d_3[tensor] MC figure"</b> button.</p>
    
    <p>&emsp; For the MC integration the <a href="https://vegas.readthedocs.io/en/latest/"> vegas</a> algorithm
    is used which uses a machine learning algorithm approach to best estimate the value of the multidimensional
    integral. The integral wil be estimated using <b>"MC iterations"</b> iterations of the vegas algorithm where
    for each iteration no more than <b>"MC evaluations"</b> evaluations of the integrand to estimate the integral.
    To produce independent estimates of the integral from which the weighted average is taken.</p><hr>""")
    
    L_int_val   =widgets.Text(value='2e7',placeholder=r'Integrated luminosity',
                              description=r'$L_{int}$ $(nb^{-1})$',disabled=False,style=style,layout=layout)
    s_en_val    =widgets.Text(value='1e9',placeholder=r'squared e-d C.O.M energy ',
                              description=r'$s_{eN}$ $(MeV^2)$',disabled=False,style=style,layout=layout)
    x_start_val =widgets.Text(value='0.04',placeholder=r'Bjorken variable',
                              description=r'Bjorken $x$ lower bound',disabled=False,style=style,layout=layout)
    x_stop_val  =widgets.Text(value='0.06',placeholder=r'Bjorken variable',
                              description=r'Bjorken $x$ upper bound',disabled=False,style=style,layout=layout)
    Q2_start_val=widgets.Text(value='20e6',placeholder=r'4 momentum transfer squared',
                              description=r'$Q^2$ lower bound $(MeV^2)$',disabled=False,style=style,layout=layout)
    Q2_stop_val =widgets.Text(value='30e6',placeholder=r'4 momentum transfer squared',
                              description=r'$Q^2$ upper bound $(MeV^2)$',disabled=False,style=style,layout=layout)
    alpha_p_start_val =widgets.Text(value='0.98',placeholder=r'proton light cone momentum fraction',
                                   description=r'$\alpha_p$ lower bound',disabled=False,style=style,layout=layout)
    alpha_p_stop_val  =widgets.Text(value='1.02',placeholder=r'proton light cone momentum fraction',
                                   description=r'$\alpha_p$ upper bound',disabled=False,style=style,layout=layout)
    x_param_val      =widgets.RadioButtons(options=[("t' (MeV^2)","t'"), ("p_pt (MeV)","p_pt"), ("k (MeV)","k")],
                                           description='kinematic variable:',disabled=False,style=style,
                                           layout=layout,value = "t'")
    x_val_start_val  =widgets.Text(value='-5e5',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. lower bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_stop_val   =widgets.Text(value='0',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. upper bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_step_val   =widgets.Text(value='2e4',placeholder=r"t', p_{pt}, k",
                                   description=r'$\Delta$ kin. var. $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    SF_parametriation_val =widgets.Dropdown(options=[("CB: Christy & Bosted parametrization", "CB"),
    ("SLAC: SLAC paramtetrization from Bodek", "SLAC"),
    ("Alekhin: leading twist parametrization by Alekhin [see PRD 68,014002]", "Alekhin"),
    ("CTEQ: F2 based on the pdf's from CTEQ (code from Misak)", "CTEQ"),
    ("HMRS: F2 from pdfs from unknown source (Shunzo Kumano is the source, not enough info in his code file)", 
     "HMRS"),
    ("MSTW: F2 from LO (or nlo etc) from MSTW pdfs", "MSTW")],value="CTEQ",description='Structure Function:',
                                            disabled=False,style = style,layout = layout)  
    WF_parametrization_val=widgets.RadioButtons(options=[("AV18a","first"), ("AV18b","second")],
                                                description='Wave function:',disabled=False,style = style,
                                                layout = layout)
    x_dep_val    =widgets.RadioButtons(options=[(r"t'(GeV^2)","t'"), (r"p_pt (GeV)","p_pt"), ("k (GeV)", "k")],
                                       description='x-axis variable:',disabled=False,style=style,layout=layout,
                                       value = "t'")
    save =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                       disabled=False,style=style,layout=layout)
    nitn =widgets.Text(value='5',placeholder=r"nitn",description=r'MC iterations:',disabled=False,style=style,
                       layout=layout)
    neval=widgets.Text(value='5',placeholder=r"neval",description=r'MC evaluations:',disabled=False,
                       style=style,layout=layout)
    
    make_figure_Ad3_av =widgets.Button(description='Make A_d_3[tensor] bin average figure',disabled=False,
                                       button_style='',tooltip='make A_d_3[tensor] MC figure',icon='check',
                                       layout=Layout(width="initial"))
    make_figure_Ad3_mc =widgets.Button(description='Make A_d_3[tensor] MC figure',disabled=False,button_style='',
                                       tooltip='make figure',icon='check',layout=Layout(width="initial"))

    display(intro)
    
    display(widgets.HBox((L_int_val,s_en_val)))
    display(widgets.HBox((x_start_val, x_stop_val)))
    display(widgets.HBox((Q2_start_val, Q2_stop_val)))
    display(widgets.HBox((alpha_p_start_val, alpha_p_stop_val)))
    display(widgets.HBox((x_param_val,x_val_start_val)))
    display(widgets.HBox(( x_val_stop_val,x_val_step_val)))
    display(widgets.HBox((SF_parametriation_val, WF_parametrization_val)))
    display(widgets.HBox((x_dep_val,save)))
    display(widgets.HBox((nitn,neval)))

    display(widgets.HBox((make_figure_Ad3_av,make_figure_Ad3_mc)))
    
    def Ad3_fig(b):
        with out:
            clear_output()
            L_int  = float(L_int_val.value)
            s_ed   = 2.*float(s_en_val.value)
            x_int  = np.array([float(x_start_val.value),float(x_stop_val.value)])
            Q2_starts = Q2_start_val.value.split(",")
            Q2_stops  = Q2_stop_val.value.split(",")
            length    = min(len(Q2_starts),len(Q2_stops))
            Q2_int = np.array([[float(Q2_starts[i]),float(Q2_stops[i])] for i in range(length)])
            alpha_p_int = np.array([float(alpha_p_start_val.value),float(alpha_p_stop_val.value)])
            x_val_int   = np.arange(float(x_val_start_val.value),float(x_val_stop_val.value),
                                    float(x_val_step_val.value))
            if b.description == "Make A_d_3[tensor] bin average figure":
                figure = plot_Ad3_ten(L_int,s_ed,x_int,Q2_int,alpha_p_int,x_val_int,x_param_val.value,
                                      SF_parametriation_val.value,WF_parametrization_val.value,x_dep_val.value,
                                      save.value,False)
            else:
                figure = plot_Ad3_ten(L_int,s_ed,x_int,Q2_int,alpha_p_int,x_val_int,x_param_val.value,
                                      SF_parametriation_val.value,WF_parametrization_val.value,x_dep_val.value,
                                      save.value,True,int(nitn.value),int(neval.value))
            if save.value:
                save.value = ""
            else:
                save_after_t=widgets.Text(value='',placeholder=r"Name of figure for saving",
                                          description=r'Figure name:',disabled=False,
                                          style={'description_width': 'initial'},layout=Layout(width="initial"))
                save_after_b=widgets.Button(description='Save figure',disabled=False,button_style='',
                                            tooltip='save',icon='check',style={'description_width': 'initial'},
                                            layout=Layout(width="initial"))
                display(widgets.HBox((save_after_t,save_after_b)))
                def on_clicked(_):
                    if not "png" in save_after_t.value and not "jpg" in save_after_t.value and\
                    not "pdf" in save_after_t.value:
                        save_after_t.value += ".png"
                    saving = os.path.join(os.getcwd(),"output",save_after_t.value)
                    figure.savefig(saving,dpi=600)
                    print("saved as {}".format(saving))
                    save_after_t.value=""
                save_after_b.on_click(on_clicked)

    make_figure_Ad3_av.on_click(Ad3_fig)
    make_figure_Ad3_mc.on_click(Ad3_fig)
    
#out = Output()
#make_figure_A_d_3_ten(out)
#out

In [162]:
def A_d_2_ten(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation = "CTEQ",WF_parametrization = "first"):
    """
    The parallel spin asymmetry
    
    x        (float)           : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                 the absence of nuclear binding.
    Q2       (float)(MeV^2)    : the negative of the 4 momentum transfer squared or virtual photon momentum squared
    alpha_p  (float)           : the proton light cone momentum fraction
    p_pt    (float)(MeV)      : the transverse momentum value, p_pt.
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 If not specified the "first" one is used.
    
    returns
    """
    
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_val)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_val**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    p_pt = p_trans()
    
    x_d     = x/2.
    y       = Q2/(x_d*(s_ed-MD**2))
    gamma2  = (x*MD)**2/Q2 #2x_d = x
    epsilon = (1-y-gamma2*y**2/4)/(1-y+y**2/2+gamma2*y**2/4)
    
    #structure function________________________________
    struc_f = struc_func(SF_parametriation, WF_parametrization)
    F = struc_f.FTd(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FLd(x,Q2,alpha_p,p_pt)
    FT = struc_f.FTLLT(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FTLLL(x,Q2,alpha_p,p_pt)
    
    #spin asymmetry____________________________________
    asymmetry = (3./2.)*FT/(F+FT)
    
    return asymmetry

def A_d_2_ten_err(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,SF_parametriation = "CTEQ",\
              WF_parametrization = "first"):
    
     #bin averages______________________________________
    x       = np.mean(x_range)
    Q2      = np.mean(Q2_range)
    alpha_p = np.mean(alpha_p_range)
    x_val   = np.mean(x_val_range)
    
    def t_apo(val):
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*val**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*val**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_val)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_val**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    p_pt = p_trans()
    
    #bin sizes_________________________________________
    delta_x       = x_range[1]-x_range[0]
    delta_Q2      = Q2_range[1]-Q2_range[0]
    delta_alpha_p = alpha_p_range[1]-alpha_p_range[0]
    delta_x_val   = x_val_range[1]-x_val_range[0]
    delta_t_p     = abs(t_apo(x_val_range[1]-x_val_range[0]))
    
    x_d     = x/2.
    y       = Q2/(x_d*(s_ed-MD**2))
    gamma2  = (x*MD)**2/Q2 #2x_d = x
    epsilon = (1-y-gamma2*y**2/4)/(1-y+y**2/2+gamma2*y**2/4)
    
    #calculating the error
    delta = delta_x*delta_Q2*delta_alpha_p*delta_t_p
    #error of the numerator
    #spins = [(1./2.,1,1.),(-1./2.,1,-1.),(1./2.,-1,-1.),(-1./2.,-1,1.)] #(sigma_e,lambda_d,add/subtract)
    spins = [(1./2.,1,2.),(-1./2.,1,2.),(1./2.,0,-4)]
    num = 0.
    sq_err_sum = 0.
    for spin in spins:
        sigma_e,lambda_d,opperation=spin
        value = full_cross_sec(s_ed,x,Q2,alpha_p,x_val,x_parameter,sigma_e,lambda_d,\
                               SF_parametriation,WF_parametrization)
        
        sq_err_sum += opperation**2*value
        num        += opperation*value
    sq_rel_num_err = sq_err_sum/(L_int*delta*num**2)
    
    #error of the denominator
    #structure function________________________________
    struc_f = struc_func(SF_parametriation, WF_parametrization)
    F = struc_f.FTd(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FLd(x,Q2,alpha_p,p_pt)
    FT = struc_f.FTLLT(x,Q2,alpha_p,p_pt)+epsilon*struc_f.FTLLL(x,Q2,alpha_p,p_pt)    
    
    sq_rel_den_err = 1./(L_int*delta*(F+FT))    
    rel_error = np.sqrt(sq_rel_den_err + sq_rel_num_err)
    
    asymmetry = A_d_2_ten(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation,WF_parametrization)
    error = rel_error*asymmetry
    
    return [asymmetry,error]

def A_d_2_ten_err_MC(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,SF_parametriation = "CTEQ",\
                 WF_parametrization = "first",nitn=1,neval=1):
    """
    The parallel spin asymmetry with mc integration for error
    
    x        (float)           : the effective Bjorken variable for scattering from a nucleon in the deuteron in 
                                 the absence of nuclear binding.
    Q2       (float)(MeV^2)    : the negative of the 4 momentum transfer squared or virtual photon momentum squared
    alpha_p  (float)           : the proton light cone momentum fraction
    p_pt    (float)(MeV)      : the transverse momentum value, p_pt.
    SF_parametriation  (string): the structure function parametriation. Standard value is "CTEQ", 
                                 for other parametriations see struc_func.
    WF_parametrization (string): the AV18 wave function parametrization ("first", "second"). 
                                 If not specified the "first" one is used.
    
    returns
    """
    def p_trans(alpha_p_t,x_val_t):
        if x_parameter == "t'":
            return np.sqrt((alpha_p_t*(2.-alpha_p_t)*MD**2-4*MN**2-2*alpha_p_t*x_val_t)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p_t*(2-alpha_p_t)*(x_val_t**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_val
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    #bin averages______________________________________
    x       = np.mean(x_range)
    Q2      = np.mean(Q2_range)
    alpha_p = np.mean(alpha_p_range)
    x_val   = np.mean(x_val_range)    
    
    
    asymmetry = A_d_2_ten(s_ed,x,Q2,alpha_p,x_val,x_parameter,SF_parametriation,WF_parametrization)
    
    #error of the numerator
    #spins = [(1./2.,1,1.),(-1./2.,1,-1.),(1./2.,-1,-1.),(-1./2.,-1,1.)] #(sigma_e,lambda_d,add/subtract)
    spins = [(1./2.,1,2.),(-1./2.,1,2.),(1./2.,0,-4.)]
    num = 0.
    sq_err_sum = 0.
    for spin in spins:
        sigma_e,lambda_d,opperation=spin
        
        def integrandum_num(values):
            x_p,Q2_p,alpha_p_p,x_val_p = values
            return full_cross_sec(s_ed,x_p,Q2_p,alpha_p_p,x_val_p,x_parameter,sigma_e,lambda_d,\
                                  SF_parametriation,WF_parametrization)
        
        integ  = vegas.Integrator([x_range,Q2_range,alpha_p_range,x_val_range])
        result = integ(integrandum_num,nitn=nitn,neval=neval)
        Delta_N  = L_int*result.mean
        value = integrandum_num((x,Q2,alpha_p,x_val))
        sq_err_sum += (opperation*value)**2/Delta_N
        
        num += opperation*value
        
    sq_rel_num_err = sq_err_sum/num**2
    
    #error of the denominator
    #structure function________________________________
    struc_f = struc_func(SF_parametriation, WF_parametrization)
    def integrandum_den(values):
        x_p,Q2_p,alpha_p_p,x_val_p = values
        p_pt_p = p_trans(alpha_p_p,x_val_p)
        
        x_d_p     = x_p/2.
        y_p       = Q2_p/(x_d_p*(s_ed-MD**2))
        gamma2_p  = (x_p*MD)**2/Q2_p #2x_d = x
        epsilon_p = (1-y_p-gamma2_p*y_p**2/4)/(1-y_p+y_p**2/2+gamma2_p*y_p**2/4)
        
        F = struc_f.FTd(x_p,Q2_p,alpha_p_p,p_pt_p)+epsilon_p*struc_f.FLd(x_p,Q2_p,alpha_p_p,p_pt_p)
        FT = struc_f.FTLLT(x_p,Q2_p,alpha_p_p,p_pt_p)+epsilon_p*struc_f.FTLLL(x_p,Q2_p,alpha_p_p,p_pt_p)
        return F+FT
    
    
    integ  = vegas.Integrator([x_range,Q2_range,alpha_p_range,x_val_range])
    result = integ(integrandum_den,nitn=nitn,neval=neval)
    Delta_N  = L_int*result.mean
    sq_rel_den_err = 1./Delta_N
    
    sq_rel_err = sq_rel_num_err + sq_rel_den_err
    error = asymmetry*np.sqrt(sq_rel_err)    
    
    return [asymmetry,error]

def plot_Ad2_ten(L_int,s_ed,x_range,Q2_range,alpha_p_range,x_val_range,x_parameter,SF_parametriation = "CTEQ",
                 WF_parametrization = "first",x_dependency=None,save="",MC=False,nitn=5,neval=5):
    
    #bin averages______________________________________
    alpha_p = np.mean(alpha_p_range)
    x_vals  = np.mean(list(zip(x_val_range[:-1],x_val_range[1:])),axis=1)
    
    def p_trans():
        if x_parameter == "t'":
            return np.sqrt((alpha_p*(2.-alpha_p)*MD**2-4*MN**2-2*alpha_p*x_vals)/4)
        elif x_parameter == "k":
            return np.sqrt(alpha_p*(2-alpha_p)*(x_vals**2+MN**2)-MN**2)
        elif x_parameter == "p_pt":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def t_apo():
        if x_parameter == "p_pt":
            return (alpha_p*(2-alpha_p)*MD**2-4*MN**2-4*x_vals**2)/(2*alpha_p)
        elif x_parameter == "k":
            return (2-alpha_p)*(MD**2-4*x_vals**2-4*MN**2)/2.
        elif x_parameter == "t'":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    def k():
        if x_parameter == "t'":
            return np.sqrt(MD**2-4*MN**2-2*x_vals/(2-alpha_p))/4
        elif x_parameter == "p_pt":
            return np.sqrt((x_vals**2+MN**2)/(alpha_p*(2-alpha_p))-MN**2)
        elif x_parameter == "k":
            return x_vals
        else:
            raise ValueError("no x parameter called {}, choose from t_apo, p_pt, k".format(x_parameter))
    
    def prog_bar(i,total):
        clear_output(wait=True)
        bar = widgets.FloatProgress(value=i,min=0,max=total,description='Loading: {:.2f}%'.format(i/total*100),\
                                    bar_style='info',orientation='horizontal',\
                                    style = {'description_width': "initial"})
        display(bar)
    
    values = []
    total_calculations = (len(x_val_range)-1)*(len(Q2_range))
    prog_bar(0,total_calculations)
    for j in range(len(Q2_range)):
        temp_val = []
        for i in range(len(x_val_range)-1):
            if not MC:
                temp_val  += [A_d_2_ten_err(L_int,s_ed,x_range,Q2_range[j],alpha_p_range,x_val_range[i:i+2],
                                            x_parameter,SF_parametriation,WF_parametrization)]
            else:
                temp_val  += [A_d_2_ten_err_MC(L_int,s_ed,x_range,Q2_range[j],alpha_p_range,x_val_range[i:i+2],
                                               x_parameter,SF_parametriation,WF_parametrization,nitn,neval)]
            prog_bar(j*(len(x_val_range)-1)+i+1,total_calculations)
        values += [temp_val]
    values = np.array(values)
    
    if not x_dependency:
        x_dependency = x_parameter
    if x_dependency == "t'":
        x_dep_val = -t_apo()*10**-6
        x_label = r"-t' [$GeV^2$]"
    elif x_dependency == "p_pt":
        x_dep_val = p_trans()*10**-3 
        x_label = r"$p_{pt}$ [GeV]"
    elif x_dependency == "k":
        x_dep_val = k()*10**-3
        x_label = r"k [GeV]"
    else:
        raise ValueError("no x dependency called {}, choose from t_apo, p_pt, k".format)
    
    markers     = ["+","x","*","s",".","8","p","h"]
    fig,ax = plt.subplots(figsize=(8,4))
    for j,value in enumerate(values):
        ax.errorbar(x_dep_val,value[:,0],yerr=value[:,1],linestyle = "",marker=markers[j%len(markers)],\
                    capsize=3,label=r"$Q^2$ = {}-{} $GeV^2$".format(Q2_range[j][0]*10**-6,Q2_range[j][1]*10**-6))
    ax.plot([], [], ' ', label=r"$L_{{int}} = {}\cdot 1O^6 nb^{{-1}}$""\n"r"$S_{{eN}} = {} GeV^2$""\n"\
             r"x = {}-{}""\n"r"$\alpha_p$ = {}-{}".format(int(L_int*10**-6),int(s_ed*10**-6/2),x_range[0],
                                                         x_range[1],alpha_p_range[0],alpha_p_range[1]))
    #ax.set_ylim([-0.1,0.])
    ax.set_ylim(auto=True)
    ax.set_ylabel(r"$A_{\parallel d}^{(2)}$[tensor]")
    ax.set_xlabel(x_label)
    ax.legend(loc=2,bbox_to_anchor= (1.0, 1.0))
    plt.tight_layout()
    if save:
        if not "png" in save and not "jpg" in save and not "pdf" in save:
            save += ".png"
        saving = os.path.join(os.getcwd(),"output",save)
        fig.savefig(saving, dpi=600)
        print("saved as {}".format(saving))
    plt.show()
    
    return fig
    
def make_figure_A_d_2_ten(out):
    intro = widgets.HTMLMath(r"""<center><b><h1>Spin asymmetry $A_{\parallel d}^{(2)}$[tensor]</h1></b></center>
    <hr>
    <figure style="float: right;">
        <img src="figures/figure_A_d_2_tensor.png" width ="400">
        <figcaption style="text-align:center">fig: An example figure of the <b>Spin asymmetry <br/>
        $A_{\parallel d}^{(2)}$[tensor]</b> program</figcaption>
    </figure>
    
    <p>&emsp;This cell can create a figure of the tensor two-state parallel spin asymmetry 
    $A_{\parallel d}^{(2)}$[tensor] for a certain integrated luminosity, which can be given at <b>"$L_{int}$"</b>, and an 
    electron-Nucleon center of mass energy, provided at <b>"$s_{eN}$"</b>.</p>
    
    <p>&emsp;In the given interval for the Bjorken scaling variable, from 
    <b>"Bjorken $x$ lower bound"</b> to <b>"Bjorken $x$ upper bound"</b>, for intervals of the 4 momentum 
    transfer squared in $MeV^2$ which begin from <b>"$Q^2$ lower bound"</b> and end at <b>"$Q^2$ upper bound"</b>, 
    for the proton transverse momentum fraction interval from <b>"$\alpha_p$ lower bound"</b> to <b>"$\alpha_p$ 
    upper bound"</b> and for a range of kinematic variables that can be:<br/> 
    &emsp;- The transverse prtoton momentum, $p_{pT}$ (in MeV)<br/> 
    &emsp;- The proton 3 momentum norm, k (in MeV)<br/> 
    &emsp;- The invariant momentum transfer, t' ($in MeV^2$).<br/>
    Which kinematic variable is given by the user can be indicated at the <b>"kinematic variable"</b> radio 
    buttons. The begin 
    and end of the interval can be given, respectively, at <b>"kin. var. lower bound"</b> and <b>"kin. var 
    upper bound"</b>.
    <b>"$\Delta$ var."</b> is the spacing between the values. There are different parametrizations of the 
    structure function which can be selected in the drop down menu <b>"Structure Function"</b>. There are also
    2 AV18 Wave function parametrizations, AV18a and AV18b, this can be chosen by the use of the <b>"Wave 
    function"</b> radio buttons.</p>
    
    <p>&emsp;The <b>"x-axis variable"</b> is the kinematic variable that will be displayed on the x-axis of the 
    figure. The
    If a name is given in the box titled <b>"Figure name"</b>, the figure is stored as this name.png. 
    If no name is given the figure is not saved but there is the option to save the figure after it is displayed. 
    After displaying, and when no name was given, a box titled <b>"Figure name"</b> appears in which a name of the
    figure can be provided as which the figure will be saved after pressing the <b>"Save figure"</b> button.</p>
    
    <p>&emsp;A figure using the bin average values can be made by pressing the <b>"Make A_d_2[tensor] bin average 
    figure"</b>
    button. The central values of the given bins will be used to do all the calculations. The values can also
    be calculated using a Monte Carlo integration in these bins, a figure using these values will be displayed 
    after pressing the <b>"Make A_d_2[tensor] MC figure"</b> button.</p>
    
    <p>&emsp; For the MC integration the <a href="https://vegas.readthedocs.io/en/latest/"> vegas</a> algorithm
    is used which uses a machine learning algorithm approach to best estimate the value of the multidimensional
    integral. The integral wil be estimated using <b>"MC iterations"</b> iterations of the vegas algorithm where
    for each iteration no more than <b>"MC evaluations"</b> evaluations of the integrand to estimate the integral.
    To produce independent estimates of the integral from which the weighted average is taken.</p><hr>""")
    
    L_int_val   =widgets.Text(value='2e7',placeholder=r'Integrated luminosity',
                              description=r'$L_{int}$ $(nb^{-1})$',disabled=False,style=style,layout=layout)
    s_en_val    =widgets.Text(value='1e9',placeholder=r'squared e-d C.O.M energy ',
                              description=r'$s_{eN}$ $(MeV^2)$',disabled=False,style=style,layout=layout)
    x_start_val =widgets.Text(value='0.04',placeholder=r'Bjorken variable',
                              description=r'Bjorken $x$ lower bound',disabled=False,style=style,layout=layout)
    x_stop_val  =widgets.Text(value='0.06',placeholder=r'Bjorken variable',
                              description=r'Bjorken $x$ upper bound',disabled=False,style=style,layout=layout)
    Q2_start_val=widgets.Text(value='20e6',placeholder=r'4 momentum transfer squared',
                              description=r'$Q^2$ lower bound $(MeV^2)$',disabled=False,style=style,layout=layout)
    Q2_stop_val =widgets.Text(value='30e6',placeholder=r'4 momentum transfer squared',
                              description=r'$Q^2$ upper bound $(MeV^2)$',disabled=False,style=style,layout=layout)
    alpha_p_start_val =widgets.Text(value='0.98',placeholder=r'proton light cone momentum fraction',
                                   description=r'$\alpha_p$ lower bound',disabled=False,style=style,layout=layout)
    alpha_p_stop_val  =widgets.Text(value='1.02',placeholder=r'proton light cone momentum fraction',
                                   description=r'$\alpha_p$ upper bound',disabled=False,style=style,layout=layout)
    x_param_val      =widgets.RadioButtons(options=[("t' (MeV^2)","t'"), ("p_pt (MeV)","p_pt"), ("k (MeV)","k")],
                                           description='kinematic variable:',disabled=False,style=style,
                                           layout=layout,value = "t'")
    x_val_start_val  =widgets.Text(value='-5e5',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. lower bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_stop_val   =widgets.Text(value='0',placeholder=r"t', p_{pt}, k",
                                   description=r'kin. var. upper bound $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    x_val_step_val   =widgets.Text(value='2e4',placeholder=r"t', p_{pt}, k",
                                   description=r'$\Delta$ kin. var. $(MeV/MeV^2)$',disabled=False,style=style,
                                   layout=layout)
    SF_parametriation_val =widgets.Dropdown(options=[("CB: Christy & Bosted parametrization", "CB"),
    ("SLAC: SLAC paramtetrization from Bodek", "SLAC"),
    ("Alekhin: leading twist parametrization by Alekhin [see PRD 68,014002]", "Alekhin"),
    ("CTEQ: F2 based on the pdf's from CTEQ (code from Misak)", "CTEQ"),
    ("HMRS: F2 from pdfs from unknown source (Shunzo Kumano is the source, not enough info in his code file)", 
     "HMRS"),
    ("MSTW: F2 from LO (or nlo etc) from MSTW pdfs", "MSTW")],value="CTEQ",description='Structure Function:',
                                            disabled=False,style = style,layout = layout)  
    WF_parametrization_val=widgets.RadioButtons(options=[("AV18a","first"), ("AV18b","second")],
                                                description='Wave function:',disabled=False,style = style,
                                                layout = layout)
    x_dep_val    =widgets.RadioButtons(options=[(r"t'(GeV^2)","t'"), (r"p_pt (GeV)","p_pt"), ("k (GeV)", "k")],
                                       description='x-axis variable:',disabled=False,style=style,layout=layout,
                                       value = "t'")
    save =widgets.Text(value='',placeholder=r"Name of figure for saving",description=r'Figure name:',
                       disabled=False,style=style,layout=layout)
    nitn =widgets.Text(value='5',placeholder=r"nitn",description=r'MC iterations:',disabled=False,style=style,
                       layout=layout)
    neval=widgets.Text(value='5',placeholder=r"neval",description=r'MC evaluations:',disabled=False,
                       style=style,layout=layout)
    
    make_figure_Ad2_av =widgets.Button(description='Make A_d_2[tensor] bin average figure',disabled=False,
                                       button_style='',
                                       tooltip='make figure',icon='check',layout=Layout(width="initial"))
    make_figure_Ad2_mc =widgets.Button(description='Make A_d_2[tensor] MC figure',disabled=False,button_style='',
                                  tooltip='make figure',icon='check',layout=Layout(width="initial"))
    
    display(intro)
    
    display(widgets.HBox((L_int_val,s_en_val)))
    display(widgets.HBox((x_start_val, x_stop_val)))
    display(widgets.HBox((Q2_start_val, Q2_stop_val)))
    display(widgets.HBox((alpha_p_start_val, alpha_p_stop_val)))
    display(widgets.HBox((x_param_val,x_val_start_val)))
    display(widgets.HBox(( x_val_stop_val,x_val_step_val)))
    display(widgets.HBox((SF_parametriation_val, WF_parametrization_val)))
    display(widgets.HBox((x_dep_val,save)))
    display(widgets.HBox((nitn,neval)))

    display(widgets.HBox((make_figure_Ad2_av,make_figure_Ad2_mc)))
    
    def Ad2_fig(b):
        with out:
            clear_output()
            L_int  = float(L_int_val.value)
            s_ed   = 2.*float(s_en_val.value)
            x_int  = np.array([float(x_start_val.value),float(x_stop_val.value)])
            Q2_starts = Q2_start_val.value.split(",")
            Q2_stops  = Q2_stop_val.value.split(",")
            length    = min(len(Q2_starts),len(Q2_stops))
            Q2_int = np.array([[float(Q2_starts[i]),float(Q2_stops[i])] for i in range(length)])
            alpha_p_int = np.array([float(alpha_p_start_val.value),float(alpha_p_stop_val.value)])
            x_val_int   = np.arange(float(x_val_start_val.value),float(x_val_stop_val.value),
                                    float(x_val_step_val.value))
            if b.description == "Make Ad2[tensor] bin average figure":
                figure = plot_Ad2_ten(L_int,s_ed,x_int,Q2_int,alpha_p_int,x_val_int,x_param_val.value,
                                      SF_parametriation_val.value,WF_parametrization_val.value,
                                      x_dep_val.value,save.value,False)
            else:
                figure = plot_Ad2_ten(L_int,s_ed,x_int,Q2_int,alpha_p_int,x_val_int,x_param_val.value,
                                  SF_parametriation_val.value,WF_parametrization_val.value,
                                  x_dep_val.value,save.value,True,int(nitn.value),int(neval.value))
            if save.value:
                save.value = ""
            else:
                save_after_t  =widgets.Text(value='',placeholder=r"Name of figure for saving",
                                            description=r'Figure name:',disabled=False,
                                            style = {'description_width': 'initial'},
                                            layout=Layout(width="initial"))
                save_after_b = widgets.Button(description='Save figure',disabled=False,button_style='',
                                              tooltip='save',icon='check',style={'description_width':'initial'},
                                              layout=Layout(width="initial"))
                display(widgets.HBox((save_after_t,save_after_b)))
                def on_clicked(_):
                    if not "png" in save_after_t.value and not "jpg" in save_after_t.value and\
                    not "pdf" in save_after_t.value:
                        save_after_t.value += ".png"
                    saving = os.path.join(os.getcwd(),"output",save_after_t.value)
                    figure.savefig(saving,dpi=600)
                    print("saved as {}".format(saving))
                    save_after_t.value=""
                save_after_b.on_click(on_clicked)
                
    make_figure_Ad2_av.on_click(Ad2_fig)
    make_figure_Ad2_mc.on_click(Ad2_fig)
    
#out = Output()
#make_figure_A_d_2_ten(out)
#out